In [1]:
import selenium
import pandas as pd
import numpy as np
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.common.by import By
import time
import re 
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time


# AUSTIN

In [10]:
driver=webdriver.Chrome()
driver.get('https://rentberry.com/apartments/s/austin-tx')
slct_type=driver.find_element(By.XPATH,'//*[@id="search-sidebar-wrapper"]/div[1]/rb-filter-bar/div[1]/div/div/div[2]/rb-filter-selection/rb-filter-selection-item/div/button')
slct_type.click()
time.sleep(2)

In [11]:
slct_apt=driver.find_element(By.XPATH,'//div[@class="filter-panel ng-star-inserted"]/div[1]/rb-filter-options/rb-filter-option[1]')
slct_apt.click()
time.sleep(1)
slct_save=driver.find_element(By.XPATH,'//button[@class="rb-button w-100"]')
slct_save.click()
time.sleep(2)

In [12]:

urls = []
start_page = 0
end_page = 17  # Adjust the end page number as needed

for page in range(start_page, end_page):
    elements = driver.find_elements(By.XPATH, '//a[@class="property-card-bottom-address__text link-ghost font-xs one-line-string"]')
    for element in elements:
        urls.append(element.get_attribute('href'))
        
    try:
        next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//a[@class="pagination-item"]/span[@class="pagination-arrow-next"]')))
        next_button.click()
        time.sleep(2)
    except Exception as e:
        print("Exception occurred:", e)
        break  # Break the loop if unable to click next button

In [13]:

city = []
country = []
name = []
address=[]
date=[]
pet = []
bath = []
bed = []
area = []
deposit=[]
price = []
code = []
ac=[]
parking=[]
dishwasher=[]
fireplace=[]
Storage=[]
hardwood=[]

def measurement(sq_m):
    sq_ft = float(sq_m) * 10.7639  # 1 square meter = 10.7639 square feet
    return round(sq_ft,2)

for i in urls: # loop for each property card
    e = []
    driver.get(i)
    time.sleep(1.5)
    # Append city and country
    city.append('Austin')
    country.append('US')
    try:
        # Find price
        price_element = driver.find_element(By.XPATH, '//div[@class="price-value-container"]/div')
        price_text = price_element.text
        match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', price_text)
        if match:
            price_str = match.group().replace(',', '')
            price_usd = float(price_str)
            price_inr = price_usd * 83.12
            rounded = round(price_inr)
            price.append(f"{rounded}")
        else:
            price.append('-')
    except NoSuchElementException:
        price.append('-')
    try:
        # Find date
        date_element = driver.find_element(By.XPATH, '//div[@class="property-data-date font-xs"]')
        date_text = date_element.text
        match = re.search(r'\b\d{2}/\d{2}/\d{2}\b', date_text)
        if match:
            date_format = match.group()
            date.append(date_format)
        else:
            date.append('-')
    except NoSuchElementException:
        date.append('-')
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
        code.append('-')
    try:
        address_code = re.search(r'\b(?:AL|AK|AZ|AR|CA|CO|CT|DE|FL|GA|HI|ID|IL|IN|IA|KS|KY|LA|ME|MD|MA|MI|MN|MS|MO|MT|NE|NV|NH|NJ|NM|NY|NC|ND|OH|OK|OR|PA|RI|SC|SD|TN|TX|UT|VT|VA|WA|WV|WI|WY)\b', address_text)
        if address_code:
            code_format = address_code.group()
            code.append(code_format)
        else:
            code.append('-')
    except NoSuchElementException:
        address.append('-')
        code.append('-')
        
    a = driver.find_element(By.XPATH, '//*[@id="main-info"]/section/div/div/rb-property-facilities/section')
    elements = a.find_elements(By.TAG_NAME, 'span')
    
    for i in elements:
        e.append(i.text)
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    
    try:
        if e.index('Bed')>=0:
            bed.append(e[e.index('Bed')-1])
    except ValueError:
        bed.append('-')
    
    try:
        if e.index('Bath')>=0:
            bath.append(e[e.index('Bath')-1])
    except ValueError:
        bath.append('-')
    try:
        if 'Sq Ft' in e:
            area_index = e.index('Sq Ft')
            area_value = e[area_index - 1]
            area.append(area_value)
        elif 'Sq M' in e:
            area_index = e.index('Sq M')
            area_value_sq_m = e[area_index - 1]
            area_value = measurement(area_value_sq_m)
            area.append(area_value)
        else:
            area.append('-')
        
    except ValueError:
        area.append('-')
    try:
        # Check if 'Deposit' is in the list of property facilities
        if 'Deposit' in e:
            # Find the index of 'Deposit' and get the preceding element which contains the deposit amount
            deposit_index = e.index('Deposit')
            deposit_text = e[deposit_index - 1]
            
            # Extract deposit amount
            match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', deposit_text)
            if match:
                price_str = match.group().replace(',', '')
                price_usd = float(price_str)
                price_inr = price_usd * 83.12
                rounded = round(price_inr)
                deposit.append(f"{rounded}")
            else:
                deposit.append('-')
        else:
            deposit.append('-')  # If 'Deposit' is not found, append '-'
    except ValueError:
        deposit.append('-')    
    try:
        if e.index('Dogs') or e.index('Cats') >=0:
            pet.append(e[e.index('Allowed')])
    except ValueError:
        pet.append('-')
# capturing the amenities
    try:
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-rb-amenities/div/rb-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-mls-amenities/div/rb-mls-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
            
        try :
            if e.index('Air Conditioning')>=0:
                ac.append('Available')
        except ValueError:
            ac.append('Unavailable')

        try :
            if e.index('Parking')>=0:
                parking.append('Available')
        except ValueError:
            parking.append('Unavailable')

        try :
            if e.index('Dishwasher')>=0:
                dishwasher.append('Available')
        except ValueError:
            dishwasher.append('Unavailable')

        try :
            if e.index('Fireplace')>=0:
                fireplace.append('Available')
        except ValueError:
            fireplace.append('Unavailable')
        try :
            if e.index('Storage')>=0:
                Storage.append('Available')
        except ValueError:
            Storage.append('Unavailable')
        try :
            if e.index('Hardwood Floors')>=0:
                hardwood.append('Available')
        except ValueError:
            hardwood.append('Unavailable')
            
    except NoSuchElementException:
        ac.append('Unavailable')
        parking.append('Unavailable')
        dishwasher.append('Unavailable')
        fireplace.append('Unavailable')
        Storage.append('Unavailable')
        hardwood.append('Unavailable')
    
#     print(e)
    e.clear()# clear e for each itirations
        

In [14]:
austin=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'bed':bed,'bath':bath,'area':area,'pet':pet,
                 'ac': ac,'parking': parking,'state_code':code,'price':price,'date':date,
                  'dishwasher': dishwasher,'fireplace': fireplace,'storage':Storage,'hardwood_floor':hardwood,'deposit':deposit})
austin


,city,country,name,address,bed,bath,area,pet,ac,parking,state_code,price,date,dishwasher,fireplace,storage,hardwood_floor,deposit
0,Austin,US,"Douglas Street, Austin, TX 78741 - 1 BR 1 BA A...","Austin, East Riverside - Oltorf, TX, US",1,1,672,Allowed,Unavailable,Unavailable,TX,80211,02/11/24,Unavailable,Unavailable,Unavailable,Unavailable,-
1,Austin,US,"507 Pressler Street Apt #A5A Aspen, Austin, TX...",507 Pressler Street,1,1,985,Allowed,Unavailable,Unavailable,-,188017,-,Unavailable,Unavailable,Unavailable,Unavailable,-
2,Austin,US,"Anderson Mill Road, Austin, TX 78729 - 1 BR 1 ...","TX, US",1,1,673,Allowed,Unavailable,Unavailable,TX,84616,02/11/24,Unavailable,Unavailable,Unavailable,Unavailable,-
3,Austin,US,"Fabulous Downtown Studio W/ Deck & Gym, Nr Off...","Austin, TX",Studio,1,507,Allowed,Available,Available,TX,251854,10/30/23,Available,Unavailable,Unavailable,Unavailable,-
4,Austin,US,"Hip Downtown Austin 1BR W/ Gym, Pool & W/D, Nr...","Austin, TX",1,1,727,Allowed,Available,Available,TX,428899,04/25/23,Available,Unavailable,Unavailable,Unavailable,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,Austin,US,"Flat For Rent In Austin, Texas","Patton Courtyard Apartments, 1708 Patton Lane,...",Studio,1,"15,492",-,Unavailable,Unavailable,TX,54028,02/17/24,Unavailable,Unavailable,Unavailable,Unavailable,-
370,Austin,US,"Property For Rent In Austin, Texas","4805 Evans Avenue, Austin, TX 78751, USA",Studio,-,-,-,Unavailable,Unavailable,TX,241048,01/17/24,Unavailable,Unavailable,Unavailable,Unavailable,-
371,Austin,US,"Property For Rent In Austin, Texas","4016 Lewis Lane, Austin, TX 78756, USA",Studio,-,-,-,Unavailable,Unavailable,TX,130914,01/10/24,Unavailable,Unavailable,Unavailable,Unavailable,-
372,Austin,US,"East 4th Street, Downtown Austin, Austin, TX 7...","Austin, East Sixth, TX, US",1,1,774,Allowed,Unavailable,Unavailable,TX,129501,02/11/24,Unavailable,Unavailable,Unavailable,Unavailable,-


# BUFFALO

In [42]:
driver=webdriver.Chrome()
driver.get('https://rentberry.com/apartments/s/buffalo-ny')
slct_type=driver.find_element(By.XPATH,'//*[@id="search-sidebar-wrapper"]/div[1]/rb-filter-bar/div[1]/div/div/div[2]/rb-filter-selection/rb-filter-selection-item/div/button')
slct_type.click()
time.sleep(2)

In [43]:
slct_apt=driver.find_element(By.XPATH,'//div[@class="filter-panel ng-star-inserted"]/div[1]/rb-filter-options/rb-filter-option[1]')
slct_apt.click()
time.sleep(2)

In [44]:
slct_save=driver.find_element(By.XPATH,'//button[@class="rb-button w-100"]')
slct_save.click()
time.sleep(2)

In [46]:
urls = []
start_page = 0
end_page =12  # Adjust the end page number as needed

for page in range(start_page, end_page):
    elements = driver.find_elements(By.XPATH, '//a[@class="property-card-bottom-address__text link-ghost font-xs one-line-string"]')
    for element in elements:
        urls.append(element.get_attribute('href'))
        
    next_button = driver.find_element(By.XPATH, '//a[@class="pagination-item"]/span[@class="pagination-arrow-next"]')
    next_button.click()
    time.sleep(2)

In [47]:

city = []
country = []
name = []
address=[]
date=[]
pet = []
bath = []
bed = []
area = []
deposit=[]
price = []
code = []
ac=[]
parking=[]
dishwasher=[]
fireplace=[]
Storage=[]
hardwood=[]

def measurement(sq_m):
    sq_ft = float(sq_m) * 10.7639  # 1 square meter = 10.7639 square feet
    return round(sq_ft,2)

for i in urls: # loop for each property card
    e = []
    driver.get(i)
    time.sleep(1.5)
    # Append city and country
    city.append('buffalo')
    country.append('US')
    try:
        # Find price
        price_element = driver.find_element(By.XPATH, '//div[@class="price-value-container"]/div')
        price_text = price_element.text
        match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', price_text)
        if match:
            price_str = match.group().replace(',', '')
            price_usd = float(price_str)
            price_inr = price_usd * 83.12
            rounded = round(price_inr)
            price.append(f"{rounded}")
        else:
            price.append('-')
    except NoSuchElementException:
        price.append('-')
    try:
        # Find date
        date_element = driver.find_element(By.XPATH, '//div[@class="property-data-date font-xs"]')
        date_text = date_element.text
        match = re.search(r'\b\d{2}/\d{2}/\d{2}\b', date_text)
        if match:
            date_format = match.group()
            date.append(date_format)
        else:
            date.append('-')
    except NoSuchElementException:
        date.append('-')
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
        code.append('-')
    try:
        address_code = re.search(r'\b(?:AL|AK|AZ|AR|CA|CO|CT|DE|FL|GA|HI|ID|IL|IN|IA|KS|KY|LA|ME|MD|MA|MI|MN|MS|MO|MT|NE|NV|NH|NJ|NM|NY|NC|ND|OH|OK|OR|PA|RI|SC|SD|TN|TX|UT|VT|VA|WA|WV|WI|WY)\b', address_text)
        if address_code:
            code_format = address_code.group()
            code.append(code_format)
        else:
            code.append('-')
    except NoSuchElementException:
        address.append('-')
        code.append('-')
        
    a = driver.find_element(By.XPATH, '//*[@id="main-info"]/section/div/div/rb-property-facilities/section')
    elements = a.find_elements(By.TAG_NAME, 'span')
    
    for i in elements:
        e.append(i.text)
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    
    try:
        if e.index('Bed')>=0:
            bed.append(e[e.index('Bed')-1])
    except ValueError:
        bed.append('-')
    
    try:
        if e.index('Bath')>=0:
            bath.append(e[e.index('Bath')-1])
    except ValueError:
        bath.append('-')
    try:
        if 'Sq Ft' in e:
            area_index = e.index('Sq Ft')
            area_value = e[area_index - 1]
            area.append(area_value)
        elif 'Sq M' in e:
            area_index = e.index('Sq M')
            area_value_sq_m = e[area_index - 1]
            area_value = measurement(area_value_sq_m)
            area.append(area_value)
        else:
            area.append('-')
        
    except ValueError:
        area.append('-')
    try:
        # Check if 'Deposit' is in the list of property facilities
        if 'Deposit' in e:
            # Find the index of 'Deposit' and get the preceding element which contains the deposit amount
            deposit_index = e.index('Deposit')
            deposit_text = e[deposit_index - 1]
            
            # Extract deposit amount
            match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', deposit_text)
            if match:
                price_str = match.group().replace(',', '')
                price_usd = float(price_str)
                price_inr = price_usd * 83.12
                rounded = round(price_inr)
                deposit.append(f"{rounded}")
            else:
                deposit.append('-')
        else:
            deposit.append('-')  # If 'Deposit' is not found, append '-'
    except ValueError:
        deposit.append('-')    
    try:
        if e.index('Dogs') or e.index('Cats') >=0:
            pet.append(e[e.index('Allowed')])
    except ValueError:
        pet.append('-')
# capturing the amenities
    try:
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-rb-amenities/div/rb-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-mls-amenities/div/rb-mls-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
            
        try :
            if e.index('Air Conditioning')>=0:
                ac.append('Available')
        except ValueError:
            ac.append('Unavailable')

        try :
            if e.index('Parking')>=0:
                parking.append('Available')
        except ValueError:
            parking.append('Unavailable')

        try :
            if e.index('Dishwasher')>=0:
                dishwasher.append('Available')
        except ValueError:
            dishwasher.append('Unavailable')

        try :
            if e.index('Fireplace')>=0:
                fireplace.append('Available')
        except ValueError:
            fireplace.append('Unavailable')
        try :
            if e.index('Storage')>=0:
                Storage.append('Available')
        except ValueError:
            Storage.append('Unavailable')
        try :
            if e.index('Hardwood Floors')>=0:
                hardwood.append('Available')
        except ValueError:
            hardwood.append('Unavailable')
            
    except NoSuchElementException:
        ac.append('Unavailable')
        parking.append('Unavailable')
        dishwasher.append('Unavailable')
        fireplace.append('Unavailable')
        Storage.append('Unavailable')
        hardwood.append('Unavailable')
    
#     print(e)
    e.clear()# clear e for each itirations
        

In [49]:
buffalo=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'bed':bed,'bath':bath,'area':area,'pet':pet,
                 'ac': ac,'parking': parking,'state_code':code,'price':price,'date':date,
                  'dishwasher': dishwasher,'fireplace': fireplace,'storage':Storage,'hardwood_floor':hardwood,'deposit':deposit})
buffalo


,city,country,name,address,bed,bath,area,pet,ac,parking,state_code,price,date,dishwasher,fireplace,storage,hardwood_floor,deposit
0,buffalo,US,PRIME ELMOOD VILLAGE/ABLRIGHT KNOX PET FRIENDL...,"Buffalo, NY",2,1,"1,300",Allowed,Available,Available,NY,421418,04/30/23,Available,Unavailable,Unavailable,Unavailable,-
1,buffalo,US,"Flat For Rent In Buffalo, New York","82 Hazelwood Avenue, Buffalo, NY 14215, USA",2,1,"1,903",-,Unavailable,Unavailable,NY,83120,01/17/24,Unavailable,Unavailable,Unavailable,Unavailable,-
2,buffalo,US,Ready Apartment For Rent,"439 University Avenue, Unit Upper Unit",3,1,"1,300",Allowed,Unavailable,Unavailable,-,128836,11/08/23,Unavailable,Unavailable,Unavailable,Available,128836
3,buffalo,US,Cool Urban Stay: Downtown Loft,"Buffalo, NY",4,2,"10,763",-,Available,Available,NY,314194,05/31/23,Available,Unavailable,Unavailable,Unavailable,-
4,buffalo,US,"Flat For Rent In Buffalo, New York","1017 Lafayette Avenue, Buffalo, NY 14209, USA",2,1.5,"1,400",-,Unavailable,Unavailable,NY,191176,01/14/24,Unavailable,Unavailable,Unavailable,Unavailable,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,buffalo,US,"Flat For Rent In Buffalo, New York","392 Emslie Street, Buffalo, NY 14206, USA",4,1.5,"2,816",-,Unavailable,Unavailable,NY,124680,01/14/24,Unavailable,Unavailable,Unavailable,Unavailable,-
260,buffalo,US,"Flat For Rent In Buffalo, New York","49 Progressive Avenue, Buffalo, NY 14207, USA",1,1,"1,242",-,Unavailable,Unavailable,NY,66496,01/25/24,Unavailable,Unavailable,Unavailable,Unavailable,-
261,buffalo,US,"$1,500 - 2 Bedroom 1 Bathroom Single Family Ho...","29 Lord Street, Unit n/a",2,1,879,Allowed,Unavailable,Unavailable,-,124680,02/13/24,Available,Unavailable,Unavailable,Unavailable,-
262,buffalo,US,"Flat For Rent In Buffalo, New York","42 Lincoln Boulevard, Buffalo, NY 14217, USA",3,1,"2,648",-,Unavailable,Unavailable,NY,128836,01/21/24,Unavailable,Unavailable,Unavailable,Unavailable,-


In [48]:
len(urls)

264

# DALLAS

In [2]:
driver=webdriver.Chrome()
driver.get('https://rentberry.com/apartments/s/dallas-tx')
slct_type=driver.find_element(By.XPATH,'//*[@id="search-sidebar-wrapper"]/div[1]/rb-filter-bar/div[1]/div/div/div[2]/rb-filter-selection/rb-filter-selection-item/div/button')
slct_type.click()
time.sleep(2)

In [3]:
slct_apt=driver.find_element(By.XPATH,'//div[@class="filter-panel ng-star-inserted"]/div[1]/rb-filter-options/rb-filter-option[1]')
slct_apt.click()
time.sleep(1)
slct_save=driver.find_element(By.XPATH,'//button[@class="rb-button w-100"]')
slct_save.click()
time.sleep(1)

In [6]:

urls = []
start_page = 0
end_page = 20  # Adjust the end page number as needed

for page in range(start_page, end_page):
    elements = driver.find_elements(By.XPATH, '//a[@class="property-card-bottom-address__text link-ghost font-xs one-line-string"]')
    for element in elements:
        urls.append(element.get_attribute('href'))
        
    try:
        next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//a[@class="pagination-item"]/span[@class="pagination-arrow-next"]')))
        next_button.click()
        time.sleep(2)
    except Exception as e:
        print("Exception occurred:", e)
        break  # Break the loop if unable to click next button


In [7]:
len(urls)

440

In [8]:

city = []
country = []
name = []
address=[]
date=[]
pet = []
bath = []
bed = []
area = []
deposit=[]
price = []
code = []
ac=[]
parking=[]
dishwasher=[]
fireplace=[]
Storage=[]
hardwood=[]

def measurement(sq_m):
    sq_ft = float(sq_m) * 10.7639  # 1 square meter = 10.7639 square feet
    return round(sq_ft,2)

for i in urls: # loop for each property card
    e = []
    driver.get(i)
    time.sleep(1.5)
    # Append city and country
    city.append('dallas')
    country.append('US')
    try:
        # Find price
        price_element = driver.find_element(By.XPATH, '//div[@class="price-value-container"]/div')
        price_text = price_element.text
        match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', price_text)
        if match:
            price_str = match.group().replace(',', '')
            price_usd = float(price_str)
            price_inr = price_usd * 83.12
            rounded = round(price_inr)
            price.append(f"{rounded}")
        else:
            price.append('-')
    except NoSuchElementException:
        price.append('-')
    try:
        # Find date
        date_element = driver.find_element(By.XPATH, '//div[@class="property-data-date font-xs"]')
        date_text = date_element.text
        match = re.search(r'\b\d{2}/\d{2}/\d{2}\b', date_text)
        if match:
            date_format = match.group()
            date.append(date_format)
        else:
            date.append('-')
    except NoSuchElementException:
        date.append('-')
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
        code.append('-')
    try:
        address_code = re.search(r'\b(?:AL|AK|AZ|AR|CA|CO|CT|DE|FL|GA|HI|ID|IL|IN|IA|KS|KY|LA|ME|MD|MA|MI|MN|MS|MO|MT|NE|NV|NH|NJ|NM|NY|NC|ND|OH|OK|OR|PA|RI|SC|SD|TN|TX|UT|VT|VA|WA|WV|WI|WY)\b', address_text)
        if address_code:
            code_format = address_code.group()
            code.append(code_format)
        else:
            code.append('-')
    except NoSuchElementException:
        address.append('-')
        code.append('-')
        
    a = driver.find_element(By.XPATH, '//*[@id="main-info"]/section/div/div/rb-property-facilities/section')
    elements = a.find_elements(By.TAG_NAME, 'span')
    
    for i in elements:
        e.append(i.text)
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    
    try:
        if e.index('Bed')>=0:
            bed.append(e[e.index('Bed')-1])
    except ValueError:
        bed.append('-')
    
    try:
        if e.index('Bath')>=0:
            bath.append(e[e.index('Bath')-1])
    except ValueError:
        bath.append('-')
    try:
        if 'Sq Ft' in e:
            area_index = e.index('Sq Ft')
            area_value = e[area_index - 1]
            area.append(area_value)
        elif 'Sq M' in e:
            area_index = e.index('Sq M')
            area_value_sq_m = e[area_index - 1]
            area_value = measurement(area_value_sq_m)
            area.append(area_value)
        else:
            area.append('-')
        
    except ValueError:
        area.append('-')
    try:
        # Check if 'Deposit' is in the list of property facilities
        if 'Deposit' in e:
            # Find the index of 'Deposit' and get the preceding element which contains the deposit amount
            deposit_index = e.index('Deposit')
            deposit_text = e[deposit_index - 1]
            
            # Extract deposit amount
            match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', deposit_text)
            if match:
                price_str = match.group().replace(',', '')
                price_usd = float(price_str)
                price_inr = price_usd * 83.12
                rounded = round(price_inr)
                deposit.append(f"{rounded}")
            else:
                deposit.append('-')
        else:
            deposit.append('-')  # If 'Deposit' is not found, append '-'
    except ValueError:
        deposit.append('-')    
    try:
        if e.index('Dogs') or e.index('Cats') >=0:
            pet.append(e[e.index('Allowed')])
    except ValueError:
        pet.append('-')
# capturing the amenities
    try:
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-rb-amenities/div/rb-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-mls-amenities/div/rb-mls-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
            
        try :
            if e.index('Air Conditioning')>=0:
                ac.append('Available')
        except ValueError:
            ac.append('Unavailable')

        try :
            if e.index('Parking')>=0:
                parking.append('Available')
        except ValueError:
            parking.append('Unavailable')

        try :
            if e.index('Dishwasher')>=0:
                dishwasher.append('Available')
        except ValueError:
            dishwasher.append('Unavailable')

        try :
            if e.index('Fireplace')>=0:
                fireplace.append('Available')
        except ValueError:
            fireplace.append('Unavailable')
        try :
            if e.index('Storage')>=0:
                Storage.append('Available')
        except ValueError:
            Storage.append('Unavailable')
        try :
            if e.index('Hardwood Floors')>=0:
                hardwood.append('Available')
        except ValueError:
            hardwood.append('Unavailable')
            
    except NoSuchElementException:
        ac.append('Unavailable')
        parking.append('Unavailable')
        dishwasher.append('Unavailable')
        fireplace.append('Unavailable')
        Storage.append('Unavailable')
        hardwood.append('Unavailable')
    
#     print(e)
    e.clear()# clear e for each itirations
        

In [9]:
dallas=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'bed':bed,'bath':bath,'area':area,'pet':pet,
                 'ac': ac,'parking': parking,'state_code':code,'price':price,'date':date,
                  'dishwasher': dishwasher,'fireplace': fireplace,'storage':Storage,'hardwood_floor':hardwood,'deposit':deposit})
dallas


,city,country,name,address,bed,bath,area,pet,ac,parking,state_code,price,date,dishwasher,fireplace,storage,hardwood_floor,deposit
0,dallas,US,Cozy Studio Apartment - Great Area,"4638 San Jacinto St, Dallas, TX 75204, USA",Studio,1,400,Allowed,Unavailable,Unavailable,TX,91432,02/17/24,Unavailable,Unavailable,Unavailable,Unavailable,91432
1,dallas,US,"$1,411 - Modern Studio In Dallas With Great Am...","4003 Ross Ave, Unit 0000",Studio,1,624,Allowed,Unavailable,Unavailable,-,117282,05/11/23,Available,Unavailable,Available,Available,8312
2,dallas,US,"Stylish 2BR Suite | Pool, Parking + Gym F","Richardson, TX",2,2,"1,200",-,Available,Available,TX,201982,04/25/23,Available,Unavailable,Unavailable,Unavailable,-
3,dallas,US,"Central 1BR King Suite | Pool, Gym, Parking B","Irving, TX",1,1,700,-,Available,Available,TX,361572,12/16/23,Available,Available,Unavailable,Unavailable,-
4,dallas,US,Large Modern Loft Style 3bd In Downtown Dallas,"Dallas, TX",2,2,"1,683",-,Available,Available,TX,713170,04/25/23,Available,Unavailable,Unavailable,Unavailable,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,dallas,US,"Flat For Rent In Dallas, Texas","5701 Harvest Hill Road, Dallas, TX 75230, USA",1,1,658,-,Unavailable,Unavailable,TX,155850,02/17/24,Unavailable,Unavailable,Unavailable,Unavailable,-
436,dallas,US,"Flat For Rent In Dallas, Texas","811 North Bishop Avenue, Dallas, TX 75208, USA",1,1,500,-,Unavailable,Unavailable,TX,103900,01/14/24,Unavailable,Unavailable,Unavailable,Unavailable,-
437,dallas,US,"Luxury DT Highrise City Views, Gym, Pool, WFH","Dallas, TX",1,1,-,-,Available,Available,TX,418925,08/01/23,Available,Unavailable,Unavailable,Unavailable,-
438,dallas,US,"Flat For Rent In Dallas, Texas","3919 Prescott Avenue, Dallas, TX 75219, USA",1,1,700,-,Unavailable,Unavailable,TX,91016,01/14/24,Unavailable,Unavailable,Unavailable,Unavailable,-


# New Jersey

In [13]:
driver=webdriver.Chrome()
driver.get('https://rentberry.com/apartments/s/jersey-city-nj')
slct_type=driver.find_element(By.XPATH,'//*[@id="search-sidebar-wrapper"]/div[1]/rb-filter-bar/div[1]/div/div/div[2]/rb-filter-selection/rb-filter-selection-item/div/button')
slct_type.click()
time.sleep(2)

In [14]:
slct_apt=driver.find_element(By.XPATH,'//div[@class="filter-panel ng-star-inserted"]/div[1]/rb-filter-options/rb-filter-option[1]')
slct_apt.click()
time.sleep(1)
slct_save=driver.find_element(By.XPATH,'//button[@class="rb-button w-100"]')
slct_save.click()
time.sleep(1)

In [15]:

urls = []
start_page = 0
end_page = 20  # Adjust the end page number as needed

for page in range(start_page, end_page):
    elements = driver.find_elements(By.XPATH, '//a[@class="property-card-bottom-address__text link-ghost font-xs one-line-string"]')
    for element in elements:
        urls.append(element.get_attribute('href'))
        
    try:
        next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//a[@class="pagination-item"]/span[@class="pagination-arrow-next"]')))
        next_button.click()
        time.sleep(2)
    except Exception as e:
        print("Exception occurred:", e)
        break  # Break the loop if unable to click next button

In [16]:

city = []
country = []
name = []
address=[]
date=[]
pet = []
bath = []
bed = []
area = []
deposit=[]
price = []
code = []
ac=[]
parking=[]
dishwasher=[]
fireplace=[]
Storage=[]
hardwood=[]

def measurement(sq_m):
    sq_ft = float(sq_m) * 10.7639  # 1 square meter = 10.7639 square feet
    return round(sq_ft,2)

for i in urls: # loop for each property card
    e = []
    driver.get(i)
    time.sleep(1.5)
    # Append city and country
    city.append('new-jersey')
    country.append('US')
    try:
        # Find price
        price_element = driver.find_element(By.XPATH, '//div[@class="price-value-container"]/div')
        price_text = price_element.text
        match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', price_text)
        if match:
            price_str = match.group().replace(',', '')
            price_usd = float(price_str)
            price_inr = price_usd * 83.12
            rounded = round(price_inr)
            price.append(f"{rounded}")
        else:
            price.append('-')
    except NoSuchElementException:
        price.append('-')
    try:
        # Find date
        date_element = driver.find_element(By.XPATH, '//div[@class="property-data-date font-xs"]')
        date_text = date_element.text
        match = re.search(r'\b\d{2}/\d{2}/\d{2}\b', date_text)
        if match:
            date_format = match.group()
            date.append(date_format)
        else:
            date.append('-')
    except NoSuchElementException:
        date.append('-')
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
        code.append('-')
    try:
        address_code = re.search(r'\b(?:AL|AK|AZ|AR|CA|CO|CT|DE|FL|GA|HI|ID|IL|IN|IA|KS|KY|LA|ME|MD|MA|MI|MN|MS|MO|MT|NE|NV|NH|NJ|NM|NY|NC|ND|OH|OK|OR|PA|RI|SC|SD|TN|TX|UT|VT|VA|WA|WV|WI|WY)\b', address_text)
        if address_code:
            code_format = address_code.group()
            code.append(code_format)
        else:
            code.append('-')
    except NoSuchElementException:
        address.append('-')
        code.append('-')
        
    a = driver.find_element(By.XPATH, '//*[@id="main-info"]/section/div/div/rb-property-facilities/section')
    elements = a.find_elements(By.TAG_NAME, 'span')
    
    for i in elements:
        e.append(i.text)
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    
    try:
        if e.index('Bed')>=0:
            bed.append(e[e.index('Bed')-1])
    except ValueError:
        bed.append('-')
    
    try:
        if e.index('Bath')>=0:
            bath.append(e[e.index('Bath')-1])
    except ValueError:
        bath.append('-')
    try:
        if 'Sq Ft' in e:
            area_index = e.index('Sq Ft')
            area_value = e[area_index - 1]
            area.append(area_value)
        elif 'Sq M' in e:
            area_index = e.index('Sq M')
            area_value_sq_m = e[area_index - 1]
            area_value = measurement(area_value_sq_m)
            area.append(area_value)
        else:
            area.append('-')
        
    except ValueError:
        area.append('-')
    try:
        # Check if 'Deposit' is in the list of property facilities
        if 'Deposit' in e:
            # Find the index of 'Deposit' and get the preceding element which contains the deposit amount
            deposit_index = e.index('Deposit')
            deposit_text = e[deposit_index - 1]
            
            # Extract deposit amount
            match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', deposit_text)
            if match:
                price_str = match.group().replace(',', '')
                price_usd = float(price_str)
                price_inr = price_usd * 83.12
                rounded = round(price_inr)
                deposit.append(f"{rounded}")
            else:
                deposit.append('-')
        else:
            deposit.append('-')  # If 'Deposit' is not found, append '-'
    except ValueError:
        deposit.append('-')    
    try:
        if e.index('Dogs') or e.index('Cats') >=0:
            pet.append(e[e.index('Allowed')])
    except ValueError:
        pet.append('-')
# capturing the amenities
    try:
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-rb-amenities/div/rb-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-mls-amenities/div/rb-mls-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
            
        try :
            if e.index('Air Conditioning')>=0:
                ac.append('Available')
        except ValueError:
            ac.append('Unavailable')

        try :
            if e.index('Parking')>=0:
                parking.append('Available')
        except ValueError:
            parking.append('Unavailable')

        try :
            if e.index('Dishwasher')>=0:
                dishwasher.append('Available')
        except ValueError:
            dishwasher.append('Unavailable')

        try :
            if e.index('Fireplace')>=0:
                fireplace.append('Available')
        except ValueError:
            fireplace.append('Unavailable')
        try :
            if e.index('Storage')>=0:
                Storage.append('Available')
        except ValueError:
            Storage.append('Unavailable')
        try :
            if e.index('Hardwood Floors')>=0:
                hardwood.append('Available')
        except ValueError:
            hardwood.append('Unavailable')
            
    except NoSuchElementException:
        ac.append('Unavailable')
        parking.append('Unavailable')
        dishwasher.append('Unavailable')
        fireplace.append('Unavailable')
        Storage.append('Unavailable')
        hardwood.append('Unavailable')
    
#     print(e)
    e.clear()# clear e for each itirations
        

In [17]:
nj=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'bed':bed,'bath':bath,'area':area,'pet':pet,
                 'ac': ac,'parking': parking,'state_code':code,'price':price,'date':date,
                  'dishwasher': dishwasher,'fireplace': fireplace,'storage':Storage,'hardwood_floor':hardwood,'deposit':deposit})
nj


,city,country,name,address,bed,bath,area,pet,ac,parking,state_code,price,date,dishwasher,fireplace,storage,hardwood_floor,deposit
0,new-jersey,US,"$3,191 - Studio In Jersey City With Great Amen...","127 Marin Blvd, Unit 413",Studio,1,461,Allowed,Unavailable,Unavailable,-,265236,02/22/24,Available,Unavailable,Unavailable,Available,83120
1,new-jersey,US,"Flat For Rent In Jersey City, New Jersey","Lin's Gourmet, 1768 John F. Kennedy Boulevard,...",3,1,"1,000",-,Unavailable,Unavailable,NJ,157928,01/09/24,Unavailable,Unavailable,Unavailable,Unavailable,-
2,new-jersey,US,"$2,425 - Studio In Bayonne With Great Amenities","196 W 43rd St, Unit 1203",Studio,1,-,Allowed,Unavailable,Unavailable,-,201566,02/01/24,Available,Unavailable,Unavailable,Available,201566
3,new-jersey,US,"Flat For Rent In Jersey City, New Jersey","295 Beacon Avenue, Croxton, Jersey City, NJ 07...",2,2,"1,040",-,Unavailable,Unavailable,NJ,265984,01/29/24,Unavailable,Unavailable,Unavailable,Unavailable,-
4,new-jersey,US,"Flat For Rent In Jersey City, New Jersey","92 Woodward Street, Communipaw, Jersey City, N...",3,1,-,-,Unavailable,Unavailable,NJ,207800,02/17/24,Unavailable,Unavailable,Unavailable,Unavailable,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,new-jersey,US,Beautiful 1BR | Business Center | Jersey City ...,"Jersey City, NJ",1,1,733,Allowed,Available,Available,NJ,546098,12/21/23,Available,Unavailable,Unavailable,Unavailable,-
436,new-jersey,US,"Residential Rental, Split Level, Multi-Level","Summit Avenue at Fairmount Avenue, Summit Aven...",2,1,-,-,Available,Unavailable,NJ,178708,02/14/23,Unavailable,Unavailable,Unavailable,Available,-
437,new-jersey,US,1 Bedroom In The Heart Of Downtown Jersey City,"Jersey City, NJ",1,1,-,Allowed,Unavailable,Unavailable,NJ,536124,09/30/23,Available,Unavailable,Unavailable,Unavailable,-
438,new-jersey,US,"Residential Rental, Contemporary","3 Colonial Drive, Bayonne, NJ 07002, USA",1,1,-,-,Available,Available,NJ,116368,02/13/23,Unavailable,Unavailable,Unavailable,Unavailable,-


# las-vegas

In [18]:
driver=webdriver.Chrome()
driver.get('https://rentberry.com/apartments/s/las-vegas-nv')
slct_type=driver.find_element(By.XPATH,'//*[@id="search-sidebar-wrapper"]/div[1]/rb-filter-bar/div[1]/div/div/div[2]/rb-filter-selection/rb-filter-selection-item/div/button')
slct_type.click()
time.sleep(2)

In [20]:
slct_apt=driver.find_element(By.XPATH,'//div[@class="filter-panel ng-star-inserted"]/div[1]/rb-filter-options/rb-filter-option[1]')
slct_apt.click()
time.sleep(1)
slct_save=driver.find_element(By.XPATH,'//button[@class="rb-button w-100"]')
slct_save.click()
time.sleep(1)

In [21]:

urls = []
start_page = 0
end_page = 9  # Adjust the end page number as needed

for page in range(start_page, end_page):
    elements = driver.find_elements(By.XPATH, '//a[@class="property-card-bottom-address__text link-ghost font-xs one-line-string"]')
    for element in elements:
        urls.append(element.get_attribute('href'))
        
    try:
        next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//a[@class="pagination-item"]/span[@class="pagination-arrow-next"]')))
        next_button.click()
        time.sleep(2)
    except Exception as e:
        print("Exception occurred:", e)
        break  # Break the loop if unable to click next button


In [22]:
len(urls)

198

In [23]:

city = []
country = []
name = []
address=[]
date=[]
pet = []
bath = []
bed = []
area = []
deposit=[]
price = []
code = []
ac=[]
parking=[]
dishwasher=[]
fireplace=[]
Storage=[]
hardwood=[]

def measurement(sq_m):
    sq_ft = float(sq_m) * 10.7639  # 1 square meter = 10.7639 square feet
    return round(sq_ft,2)

for i in urls: # loop for each property card
    e = []
    driver.get(i)
    time.sleep(1.5)
    # Append city and country
    city.append('las-vegas')
    country.append('US')
    try:
        # Find price
        price_element = driver.find_element(By.XPATH, '//div[@class="price-value-container"]/div')
        price_text = price_element.text
        match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', price_text)
        if match:
            price_str = match.group().replace(',', '')
            price_usd = float(price_str)
            price_inr = price_usd * 83.12
            rounded = round(price_inr)
            price.append(f"{rounded}")
        else:
            price.append('-')
    except NoSuchElementException:
        price.append('-')
    try:
        # Find date
        date_element = driver.find_element(By.XPATH, '//div[@class="property-data-date font-xs"]')
        date_text = date_element.text
        match = re.search(r'\b\d{2}/\d{2}/\d{2}\b', date_text)
        if match:
            date_format = match.group()
            date.append(date_format)
        else:
            date.append('-')
    except NoSuchElementException:
        date.append('-')
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
        code.append('-')
    try:
        address_code = re.search(r'\b(?:AL|AK|AZ|AR|CA|CO|CT|DE|FL|GA|HI|ID|IL|IN|IA|KS|KY|LA|ME|MD|MA|MI|MN|MS|MO|MT|NE|NV|NH|NJ|NM|NY|NC|ND|OH|OK|OR|PA|RI|SC|SD|TN|TX|UT|VT|VA|WA|WV|WI|WY)\b', address_text)
        if address_code:
            code_format = address_code.group()
            code.append(code_format)
        else:
            code.append('-')
    except NoSuchElementException:
        address.append('-')
        code.append('-')
        
    a = driver.find_element(By.XPATH, '//*[@id="main-info"]/section/div/div/rb-property-facilities/section')
    elements = a.find_elements(By.TAG_NAME, 'span')
    
    for i in elements:
        e.append(i.text)
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    
    try:
        if e.index('Bed')>=0:
            bed.append(e[e.index('Bed')-1])
    except ValueError:
        bed.append('-')
    
    try:
        if e.index('Bath')>=0:
            bath.append(e[e.index('Bath')-1])
    except ValueError:
        bath.append('-')
    try:
        if 'Sq Ft' in e:
            area_index = e.index('Sq Ft')
            area_value = e[area_index - 1]
            area.append(area_value)
        elif 'Sq M' in e:
            area_index = e.index('Sq M')
            area_value_sq_m = e[area_index - 1]
            area_value = measurement(area_value_sq_m)
            area.append(area_value)
        else:
            area.append('-')
        
    except ValueError:
        area.append('-')
    try:
        # Check if 'Deposit' is in the list of property facilities
        if 'Deposit' in e:
            # Find the index of 'Deposit' and get the preceding element which contains the deposit amount
            deposit_index = e.index('Deposit')
            deposit_text = e[deposit_index - 1]
            
            # Extract deposit amount
            match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', deposit_text)
            if match:
                price_str = match.group().replace(',', '')
                price_usd = float(price_str)
                price_inr = price_usd * 83.12
                rounded = round(price_inr)
                deposit.append(f"{rounded}")
            else:
                deposit.append('-')
        else:
            deposit.append('-') 
    except ValueError:
        deposit.append('-')    
    try:
        if e.index('Dogs') or e.index('Cats') >=0:
            pet.append(e[e.index('Allowed')])
    except ValueError:
        pet.append('-')
# capturing the amenities
    try:
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-rb-amenities/div/rb-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-mls-amenities/div/rb-mls-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
            
        try :
            if e.index('Air Conditioning')>=0:
                ac.append('Available')
        except ValueError:
            ac.append('Unavailable')

        try :
            if e.index('Parking')>=0:
                parking.append('Available')
        except ValueError:
            parking.append('Unavailable')

        try :
            if e.index('Dishwasher')>=0:
                dishwasher.append('Available')
        except ValueError:
            dishwasher.append('Unavailable')

        try :
            if e.index('Fireplace')>=0:
                fireplace.append('Available')
        except ValueError:
            fireplace.append('Unavailable')
        try :
            if e.index('Storage')>=0:
                Storage.append('Available')
        except ValueError:
            Storage.append('Unavailable')
        try :
            if e.index('Hardwood Floors')>=0:
                hardwood.append('Available')
        except ValueError:
            hardwood.append('Unavailable')
            
    except NoSuchElementException:
        ac.append('Unavailable')
        parking.append('Unavailable')
        dishwasher.append('Unavailable')
        fireplace.append('Unavailable')
        Storage.append('Unavailable')
        hardwood.append('Unavailable')
    
#     print(e)
    e.clear()# clear e for each itirations
        

In [24]:
lv=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'bed':bed,'bath':bath,'area':area,'pet':pet,
                 'ac': ac,'parking': parking,'state_code':code,'price':price,'date':date,
                  'dishwasher': dishwasher,'fireplace': fireplace,'storage':Storage,'hardwood_floor':hardwood,'deposit':deposit})
lv


,city,country,name,address,bed,bath,area,pet,ac,parking,state_code,price,date,dishwasher,fireplace,storage,hardwood_floor,deposit
0,las-vegas,US,"Flat For Rent In Las Vegas, Nevada","Parc Haven, 250 City Parkway, Las Vegas, NV 89...",2,3,"1,384",-,Unavailable,Unavailable,NV,188350,01/17/24,Unavailable,Unavailable,Unavailable,Unavailable,-
1,las-vegas,US,Fantastic Suite,"Nevada Ave, Las Vegas, NV, USA",1,1,-,-,Available,Unavailable,NV,324168,02/20/24,Unavailable,Unavailable,Unavailable,Unavailable,-
2,las-vegas,US,"Flat For Rent In Las Vegas, Nevada","Parc Haven, 250 City Parkway, Las Vegas, NV 89...",1,1,698,-,Unavailable,Unavailable,NV,125428,01/21/24,Unavailable,Unavailable,Unavailable,Unavailable,-
3,las-vegas,US,"Flat For Rent In Las Vegas, Nevada","1248 Lindell Road, Las Vegas, NV 89146, USA",2,2,"1,163",-,Unavailable,Unavailable,NV,199488,01/21/24,Unavailable,Unavailable,Unavailable,Unavailable,-
4,las-vegas,US,"Flat For Rent In Las Vegas, Nevada","1183 East Ogden Avenue, Las Vegas, NV 89101, USA",1,1,"14,676",-,Unavailable,Unavailable,NV,78133,02/11/24,Unavailable,Unavailable,Unavailable,Unavailable,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,las-vegas,US,"9029 Tumblewood Avenue, Spring Mountain Ranch,...","9029 Tumblewood Avenue, Spring Mountain Ranch,...",3,2,"2,091",-,Unavailable,Unavailable,-,210294,09/22/23,Unavailable,Unavailable,Unavailable,Unavailable,-
194,las-vegas,US,"#4204,222 Karen Avenue, Las Vegas","#4204,222 Karen Avenue, Las Vegas",3,3,"1,646",-,Unavailable,Unavailable,-,349104,09/22/23,Unavailable,Unavailable,Unavailable,Unavailable,-
195,las-vegas,US,Master Suite,"Spring Valley, NV, US",1,1,-,-,Unavailable,Unavailable,NV,253017,06/08/22,Unavailable,Unavailable,Unavailable,Unavailable,37986
196,las-vegas,US,"11580 Evergreen Creek Lane, Red Rock Country C...","11580 Evergreen Creek Lane, Red Rock Country C...",4,5,"3,823",-,Unavailable,Unavailable,-,623400,09/22/23,Unavailable,Unavailable,Unavailable,Unavailable,-


# miami

In [52]:
driver=webdriver.Chrome()
driver.get('https://rentberry.com/apartments/s/miami-fl')
slct_type=driver.find_element(By.XPATH,'//*[@id="search-sidebar-wrapper"]/div[1]/rb-filter-bar/div[1]/div/div/div[2]/rb-filter-selection/rb-filter-selection-item/div/button')
slct_type.click()
time.sleep(2)

In [53]:
slct_apt=driver.find_element(By.XPATH,'//div[@class="filter-panel ng-star-inserted"]/div[1]/rb-filter-options/rb-filter-option[1]')
slct_apt.click()
time.sleep(2)

In [54]:
slct_save=driver.find_element(By.XPATH,'//button[@class="rb-button w-100"]')
slct_save.click()
time.sleep(2)

In [55]:

urls = []
start_page = 0
end_page = 16  # Adjust the end page number as needed

for page in range(start_page, end_page):
    elements = driver.find_elements(By.XPATH, '//a[@class="property-card-bottom-address__text link-ghost font-xs one-line-string"]')
    for element in elements:
        urls.append(element.get_attribute('href'))
        
    try:
        next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//a[@class="pagination-item"]/span[@class="pagination-arrow-next"]')))
        next_button.click()
        time.sleep(2)
    except Exception as e:
        print("Exception occurred:", e)
        break  # Break the loop if unable to click next button


In [59]:
len(urls)

352

In [57]:
# Table 2 code

city = []
country = []
name = []
address=[]
date=[]
pet = []
bath = []
bed = []
area = []
deposit=[]
price = []
code = []
ac=[]
parking=[]
dishwasher=[]
fireplace=[]
Storage=[]
hardwood=[]

def measurement(sq_m):
    sq_ft = float(sq_m) * 10.7639  # 1 square meter = 10.7639 square feet
    return round(sq_ft,2)

for i in urls: # loop for each property card
    e = []
    driver.get(i)
    time.sleep(1.5)
    # Append city and country
    city.append('miami')
    country.append('US')
    try:
        # Find price
        price_element = driver.find_element(By.XPATH, '//div[@class="price-value-container"]/div')
        price_text = price_element.text
        match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', price_text)
        if match:
            price_str = match.group().replace(',', '')
            price_usd = float(price_str)
            price_inr = price_usd * 83.12
            rounded = round(price_inr)
            price.append(f"{rounded}")
        else:
            price.append('-')
    except NoSuchElementException:
        price.append('-')
    try:
        # Find date
        date_element = driver.find_element(By.XPATH, '//div[@class="property-data-date font-xs"]')
        date_text = date_element.text
        match = re.search(r'\b\d{2}/\d{2}/\d{2}\b', date_text)
        if match:
            date_format = match.group()
            date.append(date_format)
        else:
            date.append('-')
    except NoSuchElementException:
        date.append('-')
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
        code.append('-')
    try:
        address_code = re.search(r'\b(?:AL|AK|AZ|AR|CA|CO|CT|DE|FL|GA|HI|ID|IL|IN|IA|KS|KY|LA|ME|MD|MA|MI|MN|MS|MO|MT|NE|NV|NH|NJ|NM|NY|NC|ND|OH|OK|OR|PA|RI|SC|SD|TN|TX|UT|VT|VA|WA|WV|WI|WY)\b', address_text)
        if address_code:
            code_format = address_code.group()
            code.append(code_format)
        else:
            code.append('-')
    except NoSuchElementException:
        address.append('-')
        code.append('-')
        
    a = driver.find_element(By.XPATH, '//*[@id="main-info"]/section/div/div/rb-property-facilities/section')
    elements = a.find_elements(By.TAG_NAME, 'span')
    
    for i in elements:
        e.append(i.text)
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    
    try:
        if e.index('Bed')>=0:
            bed.append(e[e.index('Bed')-1])
    except ValueError:
        bed.append('-')
    
    try:
        if e.index('Bath')>=0:
            bath.append(e[e.index('Bath')-1])
    except ValueError:
        bath.append('-')
    try:
        if 'Sq Ft' in e:
            area_index = e.index('Sq Ft')
            area_value = e[area_index - 1]
            area.append(area_value)
        elif 'Sq M' in e:
            area_index = e.index('Sq M')
            area_value_sq_m = e[area_index - 1]
            area_value = measurement(area_value_sq_m)
            area.append(area_value)
        else:
            area.append('-')
        
    except ValueError:
        area.append('-')
    try:
        # Check if 'Deposit' is in the list of property facilities
        if 'Deposit' in e:
            # Find the index of 'Deposit' and get the preceding element which contains the deposit amount
            deposit_index = e.index('Deposit')
            deposit_text = e[deposit_index - 1]
            
            # Extract deposit amount
            match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', deposit_text)
            if match:
                price_str = match.group().replace(',', '')
                price_usd = float(price_str)
                price_inr = price_usd * 83.12
                rounded = round(price_inr)
                deposit.append(f"{rounded}")
            else:
                deposit.append('-')
        else:
            deposit.append('-')  # If 'Deposit' is not found, append '-'
    except ValueError:
        deposit.append('-')    
    try:
        if e.index('Dogs') or e.index('Cats') >=0:
            pet.append(e[e.index('Allowed')])
    except ValueError:
        pet.append('-')
# capturing the amenities
    try:
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-rb-amenities/div/rb-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-mls-amenities/div/rb-mls-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
            
        try :
            if e.index('Air Conditioning')>=0:
                ac.append('Available')
        except ValueError:
            ac.append('Unavailable')

        try :
            if e.index('Parking')>=0:
                parking.append('Available')
        except ValueError:
            parking.append('Unavailable')

        try :
            if e.index('Dishwasher')>=0:
                dishwasher.append('Available')
        except ValueError:
            dishwasher.append('Unavailable')

        try :
            if e.index('Fireplace')>=0:
                fireplace.append('Available')
        except ValueError:
            fireplace.append('Unavailable')
        try :
            if e.index('Storage')>=0:
                Storage.append('Available')
        except ValueError:
            Storage.append('Unavailable')
        try :
            if e.index('Hardwood Floors')>=0:
                hardwood.append('Available')
        except ValueError:
            hardwood.append('Unavailable')
            
    except NoSuchElementException:
        ac.append('Unavailable')
        parking.append('Unavailable')
        dishwasher.append('Unavailable')
        fireplace.append('Unavailable')
        Storage.append('Unavailable')
        hardwood.append('Unavailable')
    
#     print(e)
    e.clear()# clear e for each itirations
        

In [60]:
miami=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'bed':bed,'bath':bath,'area':area,'pet':pet,
                 'ac': ac,'parking': parking,'state_code':code,'price':price,'date':date,
                  'dishwasher': dishwasher,'fireplace': fireplace,'storage':Storage,'hardwood_floor':hardwood,'deposit':deposit})
miami.head(3)


,city,country,name,address,bed,bath,area,pet,ac,parking,state_code,price,date,dishwasher,fireplace,storage,hardwood_floor,deposit
0,miami,US,"1536 Northwest 15th Avenue Apt #103, Miami, FL...",1536 Northwest 15th Avenue,Studio,1,455,-,Unavailable,Unavailable,-,149616,-,Unavailable,Unavailable,Unavailable,Unavailable,-
1,miami,US,"Flat For Rent In Miami, Florida","651 Northeast 86th Street, Miami, FL 33138, USA",1,1,583,-,Unavailable,Unavailable,FL,137148,01/17/24,Unavailable,Unavailable,Unavailable,Unavailable,-
2,miami,US,2 Bedroom Apartment In Miami,"744 Northwest 3rd Street, Miami, FL 33128, USA",2,2,"1,175",Allowed,Unavailable,Unavailable,FL,250191,-,Unavailable,Unavailable,Unavailable,Unavailable,-


# orlando

In [40]:
driver=webdriver.Chrome()
driver.get('https://rentberry.com/apartments/s/orlando-fl')
slct_type=driver.find_element(By.XPATH,'//*[@id="search-sidebar-wrapper"]/div[1]/rb-filter-bar/div[1]/div/div/div[2]/rb-filter-selection/rb-filter-selection-item/div/button')
slct_type.click()
time.sleep(2)

In [41]:
slct_apt=driver.find_element(By.XPATH,'//div[@class="filter-panel ng-star-inserted"]/div[1]/rb-filter-options/rb-filter-option[1]')
slct_apt.click()
time.sleep(1)
slct_save=driver.find_element(By.XPATH,'//button[@class="rb-button w-100"]')
slct_save.click()
time.sleep(2)

In [42]:

urls = []
start_page = 0
end_page = 20  # Adjust the end page number as needed

for page in range(start_page, end_page):
    elements = driver.find_elements(By.XPATH, '//a[@class="property-card-bottom-address__text link-ghost font-xs one-line-string"]')
    for element in elements:
        urls.append(element.get_attribute('href'))
        
    try:
        next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//a[@class="pagination-item"]/span[@class="pagination-arrow-next"]')))
        next_button.click()
        time.sleep(2)
    except Exception as e:
        print("Exception occurred:", e)
        break  # Break the loop if unable to click next button


In [82]:
len(urls)

440

In [43]:
# Table 2 code

city = []
country = []
name = []
address=[]
date=[]
pet = []
bath = []
bed = []
area = []
deposit=[]
price = []
code = []
ac=[]
parking=[]
dishwasher=[]
fireplace=[]
Storage=[]
hardwood=[]

def measurement(sq_m):
    sq_ft = float(sq_m) * 10.7639  # 1 square meter = 10.7639 square feet
    return round(sq_ft,2)

for i in urls: # loop for each property card
    e = []
    driver.get(i)
    time.sleep(1.5)
    # Append city and country
    city.append('orlando')
    country.append('US')
    try:
        # Find price
        price_element = driver.find_element(By.XPATH, '//div[@class="price-value-container"]/div')
        price_text = price_element.text
        match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', price_text)
        if match:
            price_str = match.group().replace(',', '')
            price_usd = float(price_str)
            price_inr = price_usd * 83.12
            rounded = round(price_inr)
            price.append(f"{rounded}")
        else:
            price.append('-')
    except NoSuchElementException:
        price.append('-')
    try:
        # Find date
        date_element = driver.find_element(By.XPATH, '//div[@class="property-data-date font-xs"]')
        date_text = date_element.text
        match = re.search(r'\b\d{2}/\d{2}/\d{2}\b', date_text)
        if match:
            date_format = match.group()
            date.append(date_format)
        else:
            date.append('-')
    except NoSuchElementException:
        date.append('-')
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
        code.append('-')
    try:
        address_code = re.search(r'\b(?:AL|AK|AZ|AR|CA|CO|CT|DE|FL|GA|HI|ID|IL|IN|IA|KS|KY|LA|ME|MD|MA|MI|MN|MS|MO|MT|NE|NV|NH|NJ|NM|NY|NC|ND|OH|OK|OR|PA|RI|SC|SD|TN|TX|UT|VT|VA|WA|WV|WI|WY)\b', address_text)
        if address_code:
            code_format = address_code.group()
            code.append(code_format)
        else:
            code.append('-')
    except NoSuchElementException:
        address.append('-')
        code.append('-')
        
    a = driver.find_element(By.XPATH, '//*[@id="main-info"]/section/div/div/rb-property-facilities/section')
    elements = a.find_elements(By.TAG_NAME, 'span')
    
    for i in elements:
        e.append(i.text)
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    
    try:
        if e.index('Bed')>=0:
            bed.append(e[e.index('Bed')-1])
    except ValueError:
        bed.append('-')
    
    try:
        if e.index('Bath')>=0:
            bath.append(e[e.index('Bath')-1])
    except ValueError:
        bath.append('-')
    try:
        if 'Sq Ft' in e:
            area_index = e.index('Sq Ft')
            area_value = e[area_index - 1]
            area.append(area_value)
        elif 'Sq M' in e:
            area_index = e.index('Sq M')
            area_value_sq_m = e[area_index - 1]
            area_value = measurement(area_value_sq_m)
            area.append(area_value)
        else:
            area.append('-')
        
    except ValueError:
        area.append('-')
    try:
        # Check if 'Deposit' is in the list of property facilities
        if 'Deposit' in e:
            # Find the index of 'Deposit' and get the preceding element which contains the deposit amount
            deposit_index = e.index('Deposit')
            deposit_text = e[deposit_index - 1]
            
            # Extract deposit amount
            match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', deposit_text)
            if match:
                price_str = match.group().replace(',', '')
                price_usd = float(price_str)
                price_inr = price_usd * 83.12
                rounded = round(price_inr)
                deposit.append(f"{rounded}")
            else:
                deposit.append('-')
        else:
            deposit.append('-')  # If 'Deposit' is not found, append '-'
    except ValueError:
        deposit.append('-')    
    try:
        if e.index('Dogs') or e.index('Cats') >=0:
            pet.append(e[e.index('Allowed')])
    except ValueError:
        pet.append('-')
# capturing the amenities
    try:
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-rb-amenities/div/rb-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-mls-amenities/div/rb-mls-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
            
        try :
            if e.index('Air Conditioning')>=0:
                ac.append('Available')
        except ValueError:
            ac.append('Unavailable')

        try :
            if e.index('Parking')>=0:
                parking.append('Available')
        except ValueError:
            parking.append('Unavailable')

        try :
            if e.index('Dishwasher')>=0:
                dishwasher.append('Available')
        except ValueError:
            dishwasher.append('Unavailable')

        try :
            if e.index('Fireplace')>=0:
                fireplace.append('Available')
        except ValueError:
            fireplace.append('Unavailable')
        try :
            if e.index('Storage')>=0:
                Storage.append('Available')
        except ValueError:
            Storage.append('Unavailable')
        try :
            if e.index('Hardwood Floors')>=0:
                hardwood.append('Available')
        except ValueError:
            hardwood.append('Unavailable')
            
    except NoSuchElementException:
        ac.append('Unavailable')
        parking.append('Unavailable')
        dishwasher.append('Unavailable')
        fireplace.append('Unavailable')
        Storage.append('Unavailable')
        hardwood.append('Unavailable')
    
#     print(e)
    e.clear()# clear e for each itirations
        

In [44]:
orlando=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'bed':bed,'bath':bath,'area':area,'pet':pet,
                 'ac': ac,'parking': parking,'state_code':code,'price':price,'date':date,
                  'dishwasher': dishwasher,'fireplace': fireplace,'storage':Storage,'hardwood_floor':hardwood,'deposit':deposit})
orlando.head()


,city,country,name,address,bed,bath,area,pet,ac,parking,state_code,price,date,dishwasher,fireplace,storage,hardwood_floor,deposit
0,orlando,US,Comfortable Home With Balcony Near Universal P...,"The Isles at Cay Commons, Destination Parkway,...",3,-,"1,195",Allowed,Available,Available,FL,563886,12/27/20,Available,Unavailable,Unavailable,Unavailable,-
1,orlando,US,"$1,700 2 Bedroom 2 Bathroom Apartment In Orlan...","5736 S Texas Ave, Unit 2 BEDROOM",2,2,"1,215",Allowed,Unavailable,Unavailable,-,137148,05/11/23,Available,Unavailable,Available,Available,137148
2,orlando,US,"Flat For Rent In Orlando, Florida","2072 Page Avenue, Orlando, FL 32806, USA",1,1,650,-,Unavailable,Unavailable,FL,191176,01/09/24,Unavailable,Unavailable,Unavailable,Unavailable,-
3,orlando,US,Marriott Cypress Habour 2 Bds 2 Baths Condo/Ap...,"Orlando, FL",2,2,"1,345",-,Available,Available,FL,795458,09/16/23,Available,Unavailable,Unavailable,Unavailable,-
4,orlando,US,Historic District Of Lake Eola | Orlando| Parking,"Orlando, FL",1,1,-,Allowed,Available,Unavailable,FL,249360,04/25/23,Available,Unavailable,Unavailable,Unavailable,-


# ottawa

In [2]:
driver=webdriver.Chrome()
driver.get('https://rentberry.com/apartments/s/ottawa-on-canada')
slct_type=driver.find_element(By.XPATH,'//*[@id="search-sidebar-wrapper"]/div[1]/rb-filter-bar/div[1]/div/div/div[2]/rb-filter-selection/rb-filter-selection-item/div/button')
slct_type.click()
time.sleep(2)

In [3]:
slct_apt=driver.find_element(By.XPATH,'//div[@class="filter-panel ng-star-inserted"]/div[1]/rb-filter-options/rb-filter-option[1]')
slct_apt.click()
time.sleep(2)

In [4]:
slct_save=driver.find_element(By.XPATH,'//button[@class="rb-button w-100"]')
slct_save.click()
time.sleep(2)

In [10]:

urls = []
start_page = 0
end_page = 18  # Adjust the end page number as needed

for page in range(start_page, end_page):
    elements = driver.find_elements(By.XPATH, '//a[@class="property-card-bottom-address__text link-ghost font-xs one-line-string"]')
    for element in elements:
        urls.append(element.get_attribute('href'))
        
    try:
        next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//a[@class="pagination-item"]/span[@class="pagination-arrow-next"]')))
        next_button.click()
        time.sleep(2)
    except Exception as e:
        print("Exception occurred:", e)
        break  # Break the loop if unable to click next button


In [114]:
len(urls)

396

In [11]:

city = []
country = []
name = []
address=[]
date=[]
pet = []
bath = []
bed = []
area = []
deposit=[]
price = []
code = []
ac=[]
parking=[]
dishwasher=[]
fireplace=[]
Storage=[]
hardwood=[]

def measurement(sq_m):
    sq_ft = float(sq_m) * 10.7639  # 1 square meter = 10.7639 square feet
    return round(sq_ft,2)

for i in urls: # loop for each property card
    e = []
    driver.get(i)
    time.sleep(1.5)
    # Append city and country
    city.append('ottawa')
    country.append('Canada')
    try:
        # Find price
        price_element = driver.find_element(By.XPATH, '//div[@class="price-value-container"]/div')
        price_text = price_element.text
        match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', price_text)
        if match:
            price_str = match.group().replace(',', '')
            price_usd = float(price_str)
            price_inr = price_usd * 83.12
            rounded = round(price_inr)
            price.append(f"{rounded}")
        else:
            price.append('-')
    except NoSuchElementException:
        price.append('-')
    try:
        # Find date
        date_element = driver.find_element(By.XPATH, '//div[@class="property-data-date font-xs"]')
        date_text = date_element.text
        match = re.search(r'\b\d{2}/\d{2}/\d{2}\b', date_text)
        if match:
            date_format = match.group()
            date.append(date_format)
        else:
            date.append('-')
    except NoSuchElementException:
        date.append('-')
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
        code.append('-')
    try:
        address_code = re.search(r'\b(?:AB|BC|MB|NB|NL|NS|NT|NU|ON|PE|QC|SK|YT)\b', address_text)
        if address_code:
            code_format = address_code.group()
            code.append(code_format)
        else:
            code.append('-')
    except NoSuchElementException:
        address.append('-')
        code.append('-')
        
    a = driver.find_element(By.XPATH, '//*[@id="main-info"]/section/div/div/rb-property-facilities/section')
    elements = a.find_elements(By.TAG_NAME, 'span')
    
    for i in elements:
        e.append(i.text)
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    
    try:
        if e.index('Bed')>=0:
            bed.append(e[e.index('Bed')-1])
    except ValueError:
        bed.append('-')
    
    try:
        if e.index('Bath')>=0:
            bath.append(e[e.index('Bath')-1])
    except ValueError:
        bath.append('-')
    try:
        if 'Sq Ft' in e:
            area_index = e.index('Sq Ft')
            area_value = e[area_index - 1]
            area.append(area_value)
        elif 'Sq M' in e:
            area_index = e.index('Sq M')
            area_value_sq_m = e[area_index - 1]
            area_value = measurement(area_value_sq_m)
            area.append(area_value)
        else:
            area.append('-')
        
    except ValueError:
        area.append('-')
    try:
        # Check if 'Deposit' is in the list of property facilities
        if 'Deposit' in e:
            # Find the index of 'Deposit' and get the preceding element which contains the deposit amount
            deposit_index = e.index('Deposit')
            deposit_text = e[deposit_index - 1]
            
            # Extract deposit amount
            match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', deposit_text)
            if match:
                price_str = match.group().replace(',', '')
                price_usd = float(price_str)
                price_inr = price_usd * 83.12
                rounded = round(price_inr)
                deposit.append(f"{rounded}")
            else:
                deposit.append('-')
        else:
            deposit.append('-')  # If 'Deposit' is not found, append '-'
    except ValueError:
        deposit.append('-')    
    try:
        if e.index('Dogs') or e.index('Cats') >=0:
            pet.append(e[e.index('Allowed')])
    except ValueError:
        pet.append('-')
# capturing the amenities
    try:
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-rb-amenities/div/rb-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-mls-amenities/div/rb-mls-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
            
        try :
            if e.index('Air Conditioning')>=0:
                ac.append('Available')
        except ValueError:
            ac.append('Unavailable')

        try :
            if e.index('Parking')>=0:
                parking.append('Available')
        except ValueError:
            parking.append('Unavailable')

        try :
            if e.index('Dishwasher')>=0:
                dishwasher.append('Available')
        except ValueError:
            dishwasher.append('Unavailable')

        try :
            if e.index('Fireplace')>=0:
                fireplace.append('Available')
        except ValueError:
            fireplace.append('Unavailable')
        try :
            if e.index('Storage')>=0:
                Storage.append('Available')
        except ValueError:
            Storage.append('Unavailable')
        try :
            if e.index('Hardwood Floors')>=0:
                hardwood.append('Available')
        except ValueError:
            hardwood.append('Unavailable')
            
    except NoSuchElementException:
        ac.append('Unavailable')
        parking.append('Unavailable')
        dishwasher.append('Unavailable')
        fireplace.append('Unavailable')
        Storage.append('Unavailable')
        hardwood.append('Unavailable')
    
#     print(e)
    e.clear()# clear e for each itirations
        

In [31]:
ottawa=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'bed':bed,'bath':bath,'area':area,'pet':pet,
                 'ac': ac,'parking': parking,'state_code':code,'price':price,'date':date,
                  'dishwasher': dishwasher,'fireplace': fireplace,'storage':Storage,'hardwood_floor':hardwood,'deposit':deposit})
ottawa


,city,country,name,address,bed,bath,area,pet,ac,parking,state_code,price,date,dishwasher,fireplace,storage,hardwood_floor,deposit
0,ottawa,Canada,Bachelor/Studio Apartment For Rent,"255 York Street, Ottawa, ON K1N 5T9, Canada",1,1,-,-,Unavailable,Unavailable,ON,98497,02/01/24,Unavailable,Unavailable,Unavailable,Unavailable,-
1,ottawa,Canada,The Charming 3 Bedrooms River View Apartment G...,"Gatineau, QC J8P 1P3, Canada",3,1,"1,200",Allowed,Unavailable,Available,QC,325249,01/08/24,Available,Available,Unavailable,Unavailable,-
2,ottawa,Canada,"Rental, Apartment","250 Presland Road, Ottawa, ON K1J 9L9, Canada",3,3,-,-,Unavailable,Available,ON,259584,02/13/23,Available,Available,Unavailable,Available,-
3,ottawa,Canada,Great Room For Rent! Downtown Ottaw,"(Old) Ottawa, Capital, ON, CA",1,1,-,-,Unavailable,Available,ON,94757,02/26/19,Unavailable,Unavailable,Unavailable,Unavailable,14214
4,ottawa,Canada,"Sophisticated 4 Beds, 2 Baths - Nepean - Ottawa","Whitehaven, Ottawa, ON K2C 2W1, Canada",4,2,-,-,Available,Available,ON,291419,02/12/24,Available,Unavailable,Unavailable,Unavailable,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,ottawa,Canada,61 Parent Avenue - 310 Ottawa On K1n 7a7,"59 Parent Avenue, (Old) Ottawa, ON K1N 5C8, Ca...",1,1,-,-,Unavailable,Unavailable,ON,112960,01/14/24,Unavailable,Unavailable,Unavailable,Unavailable,-
392,ottawa,Canada,House In Stittsville,"1023 Curraglass Walk, Ottawa, ON K2S 0H4, Canada",3,1,-,-,Unavailable,Unavailable,ON,221764,02/21/24,Unavailable,Unavailable,Unavailable,Unavailable,-
393,ottawa,Canada,1 Bedroom Residential For Rent In Ottawa,"178 Clarence Street, Ottawa, ON K1N 8Y3, Canada",1,1,-,-,Unavailable,Unavailable,ON,140140,02/07/24,Unavailable,Unavailable,Unavailable,Unavailable,-
394,ottawa,Canada,45 Somerset,"Somerset Apartments, 45 Somerset Street West, ...",1,1,-,-,Unavailable,Unavailable,ON,141304,12/24/23,Unavailable,Unavailable,Unavailable,Unavailable,-


# vancouver

In [39]:
driver=webdriver.Chrome()
driver.get('https://rentberry.com/apartments/s/vancouver-bc-canada')
slct_type=driver.find_element(By.XPATH,'//*[@id="search-sidebar-wrapper"]/div[1]/rb-filter-bar/div[1]/div/div/div[2]/rb-filter-selection/rb-filter-selection-item/div/button')
slct_type.click()
time.sleep(2)



In [40]:
slct_apt=driver.find_element(By.XPATH,'//div[@class="filter-panel ng-star-inserted"]/div[1]/rb-filter-options/rb-filter-option[1]')
slct_apt.click()
time.sleep(2)
slct_save=driver.find_element(By.XPATH,'//button[@class="rb-button w-100"]')
slct_save.click()
time.sleep(2)

In [48]:

urls = []
start_page = 0
end_page = 20  # Adjust the end page number as needed

for page in range(start_page, end_page):
    elements = driver.find_elements(By.XPATH, '//a[@class="property-card-bottom-address__text link-ghost font-xs one-line-string"]')
    for element in elements:
        urls.append(element.get_attribute('href'))
        
    try:
        next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//a[@class="pagination-item"]/span[@class="pagination-arrow-next"]')))
        next_button.click()
        time.sleep(2)
    except Exception as e:
        print("Exception occurred:", e)
        break  # Break the loop if unable to click next button


In [49]:
# Table 2 code

city = []
country = []
name = []
address=[]
pet = []
bath = []
bed = []
area = []
deposit=[]
price = []
code = []
ac=[]
parking=[]
dishwasher=[]
fireplace=[]
Storage=[]
hardwood=[]

def measurement(sq_m):
    sq_ft = float(sq_m) * 10.7639  # 1 square meter = 10.7639 square feet
    return round(sq_ft,2)

for i in urls: # loop for each property card
    e = []
    driver.get(i)
    time.sleep(2)
    # Append city and country
    city.append('vancouver')
    country.append('Canada')
    try:
        # Find price
        price_element = driver.find_element(By.XPATH, '//div[@class="price-value-container"]/div')
        price_text = price_element.text
        match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', price_text)
        if match:
            price_str = match.group().replace(',', '')
            price_usd = float(price_str)
            price_inr = price_usd * 83.12
            rounded = round(price_inr)
            price.append(f"{rounded} INR")
        else:
            price.append('-')
    except NoSuchElementException:
        price.append('-')
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
        code.append('-')
    try:
        address_code = re.search(r'\b(?:AB|BC|MB|NB|NL|NS|NT|NU|ON|PE|QC|SK|YT)\b', address_text)
        if address_code:
            code_format = address_code.group()
            code.append(code_format)
        else:
            code.append('-')
    except NoSuchElementException:
        address.append('-')
        code.append('-')
        
    a = driver.find_element(By.XPATH, '//*[@id="main-info"]/section/div/div/rb-property-facilities/section')
    elements = a.find_elements(By.TAG_NAME, 'span')
    
    for i in elements:
        e.append(i.text)
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
#     try:
#         # Find address
#         address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
#         address_text = address_element.text
#         address.append(address_text)
#     except NoSuchElementException:
#         address.append('-')
    
    try:
        if e.index('Bed')>=0:
            bed.append(e[e.index('Bed')-1])
    except ValueError:
        bed.append('-')
    
    try:
        if e.index('Bath')>=0:
            bath.append(e[e.index('Bath')-1])
    except ValueError:
        bath.append('-')
    try:
        if 'Sq Ft' in e:
            area_index = e.index('Sq Ft')
            area_value = e[area_index - 1]
            area.append(area_value)
        elif 'Sq M' in e:
            area_index = e.index('Sq M')
            area_value_sq_m = e[area_index - 1]
            area_value = measurement(area_value_sq_m)
            area.append(area_value)
        else:
            area.append('-')
        
    except ValueError:
        area.append('-')
    try:
        # Check if 'Deposit' is in the list of property facilities
        if 'Deposit' in e:
            # Find the index of 'Deposit' and get the preceding element which contains the deposit amount
            deposit_index = e.index('Deposit')
            deposit_text = e[deposit_index - 1]
            
            # Extract deposit amount
            match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', deposit_text)
            if match:
                price_str = match.group().replace(',', '')
                price_usd = float(price_str)
                price_inr = price_usd * 83.12
                rounded = round(price_inr)
                deposit.append(f"{rounded}")
            else:
                deposit.append('-')
        else:
            deposit.append('-')  # If 'Deposit' is not found, append '-'
    except ValueError:
        deposit.append('-')    
    try:
        if e.index('Dogs') or e.index('Cats') >=0:
            pet.append(e[e.index('Allowed')])
    except ValueError:
        pet.append('-')
# capturing the amenities
    try:
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-rb-amenities/div/rb-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-mls-amenities/div/rb-mls-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
            
        try :
            if e.index('Air Conditioning')>=0:
                ac.append('Available')
        except ValueError:
            ac.append('Unavailable')

        try :
            if e.index('Parking')>=0:
                parking.append('Available')
        except ValueError:
            parking.append('Unavailable')

        try :
            if e.index('Dishwasher')>=0:
                dishwasher.append('Available')
        except ValueError:
            dishwasher.append('Unavailable')

        try :
            if e.index('Fireplace')>=0:
                fireplace.append('Available')
        except ValueError:
            fireplace.append('Unavailable')
        try :
            if e.index('Storage')>=0:
                Storage.append('Available')
        except ValueError:
            Storage.append('Unavailable')
        try :
            if e.index('Hardwood Floors')>=0:
                hardwood.append('Available')
        except ValueError:
            hardwood.append('Unavailable')
            
    except NoSuchElementException:
        ac.append('Unavailable')
        parking.append('Unavailable')
        dishwasher.append('Unavailable')
        fireplace.append('Unavailable')
        Storage.append('Unavailable')
        hardwood.append('Unavailable')
    
#     print(e)
    e.clear()# clear e for each itirations
        

In [9]:
u9= pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'ac': ac,'parking': parking,
                  'dishwasher': dishwasher,'fireplace': fireplace,'Storage':Storage,'hardwood':hardwood})
u9.head(10)

,city,country,name,address,ac,parking,dishwasher,fireplace,Storage,hardwood
0,vancouver,Canada,Heritage Suite In Central Kitsilano Close To E...,"Kitsilano, Vancouver, BC V6K 2M5, Canada",Unavailable,Available,Available,Available,Unavailable,Unavailable
1,vancouver,Canada,"D52 - Brand New, Two Bedroom Apartment At Aven...","Yaletown, Vancouver, BC V5Y 3J6, Canada",Unavailable,Available,Unavailable,Unavailable,Unavailable,Unavailable
2,vancouver,Canada,Beautiful & Stylish Apartment In Vancouver Wit...,"Vancouver, BC V6B 0G5, Canada",Unavailable,Unavailable,Available,Unavailable,Unavailable,Unavailable
3,vancouver,Canada,"Private, Bright Suite With Private Entrance In...","Kerrisdale, Vancouver, BC V6N 3P7, Canada",Unavailable,Available,Unavailable,Available,Unavailable,Unavailable
4,vancouver,Canada,2BD Modern Condo In Downtown Vancouver!,"Vancouver, BC V6Z 1C1, Canada",Unavailable,Available,Available,Unavailable,Unavailable,Unavailable
5,vancouver,Canada,Walking Distance To Arenas/ 1 Bed & 1 Bath Ful...,"Chinatown, Vancouver, BC V6A 1Z6, Canada",Available,Unavailable,Available,Unavailable,Unavailable,Unavailable
6,vancouver,Canada,Fantastic Location 1 BDR Charming Apartment In...,"Vancouver, BC V6Z 3E1, Canada",Available,Available,Available,Available,Unavailable,Unavailable
7,vancouver,Canada,The 18th Ave Garden Suite,"Cedar Cottage, Vancouver, BC V5N 2H2, Canada",Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable
8,vancouver,Canada,Unique Downtown Urban Oasis!,"Robson Street, Vancouver, BC V6E 4P5, Canada",Available,Available,Available,Available,Unavailable,Unavailable
9,vancouver,Canada,"1 BR In The Heart Of Downtown: Parking, Pool &...","Gastown, Vancouver, BC V6B 0B9, Canada",Available,Available,Available,Unavailable,Unavailable,Unavailable


# toronto

In [59]:
driver=webdriver.Chrome()
driver.get('https://rentberry.com/apartments/s/toronto-on-canada')
slct_type=driver.find_element(By.XPATH,'//*[@id="search-sidebar-wrapper"]/div[1]/rb-filter-bar/div[1]/div/div/div[2]/rb-filter-selection/rb-filter-selection-item/div/button')
slct_type.click()
time.sleep(2)
slct_apt=driver.find_element(By.XPATH,'//div[@class="filter-panel ng-star-inserted"]/div[1]/rb-filter-options/rb-filter-option[1]')
slct_apt.click()
time.sleep(2)
slct_save=driver.find_element(By.XPATH,'//button[@class="rb-button w-100"]')
slct_save.click()
time.sleep(2)


In [61]:

urls = []
start_page = 0
end_page = 15  # Adjust the end page number as needed

for page in range(start_page, end_page):
    elements = driver.find_elements(By.XPATH, '//a[@class="property-card-bottom-address__text link-ghost font-xs one-line-string"]')
    for element in elements:
        urls.append(element.get_attribute('href'))
        
    try:
        next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//a[@class="pagination-item"]/span[@class="pagination-arrow-next"]')))
        next_button.click()
        time.sleep(2)
    except Exception as e:
        print("Exception occurred:", e)
        break  # Break the loop if unable to click next button


In [62]:
city = []
country = []
name = []
date = []
address = []
price = []
code = []

for i in urls:  # loop for each property card
    driver.get(i)
    time.sleep(1)
    
    # Append city and country
    city.append('toronto')
    country.append('Canada')
    
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
    
    try:
        # Find date
        date_element = driver.find_element(By.XPATH, '//div[@class="property-data-date font-xs"]')
        date_text = date_element.text
        match = re.search(r'\b\d{2}/\d{2}/\d{2}\b', date_text)
        if match:
            date_format = match.group()
            date.append(date_format)
        else:
            date.append('-')
    except NoSuchElementException:
        date.append('-')
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
        code.append('-')
    try:
        address_code = re.search(r'\b(?:AB|BC|MB|NB|NL|NS|NT|NU|ON|PE|QC|SK|YT)\b', address_text)
        if address_code:
            code_format = address_code.group()
            code.append(code_format)
        else:
            code.append('-')
    except NoSuchElementException:
        address.append('-')
        code.append('-')

 
    try:
        # Find price
        price_element = driver.find_element(By.XPATH, '//div[@class="price-value-container"]/div')
        price_text = price_element.text
        match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', price_text)
        if match:
            price_str = match.group().replace(',', '')
            price_usd = float(price_str)
            price_inr = price_usd * 83.12
            rounded = round(price_inr)
            price.append(f"{rounded} INR")
        else:
            price.append('-')
    except NoSuchElementException:
        price.append('-')


In [63]:
t10=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'date':date,'state_code':code,'price':price})
t10

,city,country,name,address,date,state_code,price
0,toronto,Canada,1 Bedroom Condo With Parking,"165 Legion Rd N, Etobicoke, ON M8Y 0B3, Canada",02/06/24,ON,182864 INR
1,toronto,Canada,Holistic Zen Home Share,"20 Brockton Ave, Toronto, ON M6K 1S5, Canada",01/31/24,ON,99744 INR
2,toronto,Canada,"Cozy Studio Apartment In Toronto, Near Osgoode...","Studio 2, 199 Richmond Street West, Old Toront...",08/26/20,ON,211956 INR
3,toronto,Canada,Sunny & Spacious 1-Bedroom Apartment In Toront...,"University Plaza, Richmond Street West, Old To...",08/26/20,ON,224424 INR
4,toronto,Canada,"Nice & Bright 1-Bedroom In Toronto, Near Osgoo...","University Plaza, Richmond Street West, Old To...",08/26/20,ON,224424 INR
...,...,...,...,...,...,...,...
325,toronto,Canada,Ravine View Getaway To Toronto,"Bendale, Scarborough, ON M1J 2B1, Canada",08/21/23,ON,187768 INR
326,toronto,Canada,Great Room With Gym Near Downtown,"Toronto, Parkview Hills, ON, CA",06/02/22,ON,98913 INR
327,toronto,Canada,Sunny Beaches Home - Kingston Rd Village,"East End Danforth, Toronto, ON M4E 3G4, Canada",04/24/23,ON,213452 INR
328,toronto,Canada,Executive Monthly 2BR/2Bath Luxury Condo - Dow...,"Niagara, Toronto, ON M5V 1T4, Canada",07/19/23,ON,311700 INR


In [64]:
# Table 2 code

city = []
country = []
name = []
address=[]
pet = []
bath = []
bed = []
area = []
deposit=[]
def measurement(sq_m):
    sq_ft = float(sq_m) * 10.7639  # 1 square meter = 10.7639 square feet
    return round(sq_ft,2)

for i in urls: # loop for each property card
    e = []
    driver.get(i)
    time.sleep(1)
    # Append city and country
    city.append('ottawa')
    country.append('Canada')
    a = driver.find_element(By.XPATH, '//*[@id="main-info"]/section/div/div/rb-property-facilities/section')
    elements = a.find_elements(By.TAG_NAME, 'span')
    
    for i in elements:
        e.append(i.text)
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
    
    try:
        if e.index('Bed')>=0:
            bed.append(e[e.index('Bed')-1])
    except ValueError:
        bed.append('-')
    
    try:
        if e.index('Bath')>=0:
            bath.append(e[e.index('Bath')-1])
    except ValueError:
        bath.append('-')
    try:
        if 'Sq Ft' in e:
            area_index = e.index('Sq Ft')
            area_value = e[area_index - 1]
            area.append(area_value)
        elif 'Sq M' in e:
            area_index = e.index('Sq M')
            area_value_sq_m = e[area_index - 1]
            area_value = measurement(area_value_sq_m)
            area.append(area_value)
        else:
            area.append('-')
        
    except ValueError:
        area.append('-')
    try:
        # Check if 'Deposit' is in the list of property facilities
        if 'Deposit' in e:
            # Find the index of 'Deposit' and get the preceding element which contains the deposit amount
            deposit_index = e.index('Deposit')
            deposit_text = e[deposit_index - 1]
            
            # Extract deposit amount
            match = re.search(r'(\d+(\.\d+)?)', deposit_text)
            if match:
                dollar = float(match.group())
                inr = dollar * 83.12
                rounded = round(inr)
                deposit.append(f"{rounded} INR")
            else:
                deposit.append('-')
        else:
            deposit.append('-')  # If 'Deposit' is not found, append '-'
    except ValueError:
        deposit.append('-')    
    try:
        if e.index('Dogs') or e.index('Cats') >=0:
            pet.append(e[e.index('Allowed')])
    except ValueError:
        pet.append('-')
    
#     print(e)
    e.clear()# clear e for each itirations
        

In [70]:
y10=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'bed':bed,'bath':bath,'area':area,'pet':pet,'deposit':deposit})
y10

,city,country,name,address,bed,bath,area,pet,deposit
0,toronto,Canada,1 Bedroom Condo With Parking,"165 Legion Rd N, Etobicoke, ON M8Y 0B3, Canada",1,1,516.67,-,166 INR
1,toronto,Canada,Holistic Zen Home Share,"20 Brockton Ave, Toronto, ON M6K 1S5, Canada",1,2,-,Allowed,83 INR
2,toronto,Canada,"Cozy Studio Apartment In Toronto, Near Osgoode...","Studio 2, 199 Richmond Street West, Old Toront...",Studio,1,441.32,-,-
3,toronto,Canada,Sunny & Spacious 1-Bedroom Apartment In Toront...,"University Plaza, Richmond Street West, Old To...",1,1,613.54,-,-
4,toronto,Canada,"Nice & Bright 1-Bedroom In Toronto, Near Osgoo...","University Plaza, Richmond Street West, Old To...",1,1,581.25,-,-
...,...,...,...,...,...,...,...,...,...
325,toronto,Canada,Ravine View Getaway To Toronto,"Bendale, Scarborough, ON M1J 2B1, Canada",1,1,850,-,-
326,toronto,Canada,Great Room With Gym Near Downtown,"Toronto, Parkview Hills, ON, CA",1,1,-,-,14878 INR
327,toronto,Canada,Sunny Beaches Home - Kingston Rd Village,"East End Danforth, Toronto, ON M4E 3G4, Canada",3,2,-,Allowed,-
328,toronto,Canada,Executive Monthly 2BR/2Bath Luxury Condo - Dow...,"Niagara, Toronto, ON M5V 1T4, Canada",2,2,-,-,-


In [66]:
# Table 3 
city = []
country = []
name = []
address=[]
ac=[]
parking=[]
dishwasher=[]
fireplace=[]
Storage=[]
hardwood=[]
for i in urls:
    
    driver.get(i)
    time.sleep(1)
    city.append('toronto')
    country.append('Canada')
    
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
            

# capturing the amenities
    try:
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-rb-amenities/div/rb-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-mls-amenities/div/rb-mls-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
            
        try :
            if e.index('Air Conditioning')>=0:
                ac.append('Available')
        except ValueError:
            ac.append('Unavailable')

        try :
            if e.index('Parking')>=0:
                parking.append('Available')
        except ValueError:
            parking.append('Unavailable')

        try :
            if e.index('Dishwasher')>=0:
                dishwasher.append('Available')
        except ValueError:
            dishwasher.append('Unavailable')

        try :
            if e.index('Fireplace')>=0:
                fireplace.append('Available')
        except ValueError:
            fireplace.append('Unavailable')
        try :
            if e.index('Storage')>=0:
                Storage.append('Available')
        except ValueError:
            Storage.append('Unavailable')
        try :
            if e.index('Hardwood Floors')>=0:
                hardwood.append('Available')
        except ValueError:
            hardwood.append('Unavailable')
            
    except NoSuchElementException:
        ac.append('Unavailable')
        parking.append('Unavailable')
        dishwasher.append('Unavailable')
        fireplace.append('Unavailable')
        Storage.append('Unavailable')
        hardwood.append('Unavailable')
#         print('No emenities')
    e.clear()


In [20]:
u10= pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'ac': ac,'parking': parking,
                  'dishwasher': dishwasher,'fireplace': fireplace,'Storage':Storage,'hardwood_floor':hardwood})
u10.head()

,city,country,name,address,ac,parking,dishwasher,fireplace,Storage,hardwood_floor
0,ottawa,Canada,Bachelor/Studio Apartment For Rent,"255 York Street, Ottawa, ON K1N 5T9, Canada",Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable
1,ottawa,Canada,The Charming 3 Bedrooms River View Apartment G...,"Gatineau, QC J8P 1P3, Canada",Unavailable,Available,Available,Available,Unavailable,Unavailable
2,ottawa,Canada,"Rental, Apartment","250 Presland Road, Ottawa, ON K1J 9L9, Canada",Unavailable,Available,Available,Available,Unavailable,Available
3,ottawa,Canada,Great Room For Rent! Downtown Ottaw,"(Old) Ottawa, Capital, ON, CA",Unavailable,Available,Unavailable,Unavailable,Unavailable,Unavailable
4,ottawa,Canada,"Sophisticated 4 Beds, 2 Baths - Nepean - Ottawa","Whitehaven, Ottawa, ON K2C 2W1, Canada",Available,Available,Available,Unavailable,Unavailable,Unavailable


# brisbane

In [23]:
driver=webdriver.Chrome()
driver.get('https://rentberry.com/apartments/s/brisbane-qld-australia')
slct_type=driver.find_element(By.XPATH,'//*[@id="search-sidebar-wrapper"]/div[1]/rb-filter-bar/div[1]/div/div/div[2]/rb-filter-selection/rb-filter-selection-item/div/button')
slct_type.click()
time.sleep(1)
slct_apt=driver.find_element(By.XPATH,'//div[@class="filter-panel ng-star-inserted"]/div[1]/rb-filter-options/rb-filter-option[1]')
slct_apt.click()
slct_save=driver.find_element(By.XPATH,'//button[@class="rb-button w-100"]')
slct_save.click()
time.sleep(1)


In [24]:

urls = []
start_page = 0
end_page = 14  # Adjust the end page number as needed

for page in range(start_page, end_page):
    elements = driver.find_elements(By.XPATH, '//a[@class="property-card-bottom-address__text link-ghost font-xs one-line-string"]')
    for element in elements:
        urls.append(element.get_attribute('href'))
        
    try:
        next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//a[@class="pagination-item"]/span[@class="pagination-arrow-next"]')))
        next_button.click()
        time.sleep(2)
    except Exception as e:
        print("Exception occurred:", e)
        break  # Break the loop if unable to click next button


In [74]:
city = []
country = []
name = []
date = []
address = []
price = []
code = []

for i in urls:  # loop for each property card
    driver.get(i)
    time.sleep(1)
    
    # Append city and country
    city.append('brisbane')
    country.append('australia')
    
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
    
    try:
        # Find date
        date_element = driver.find_element(By.XPATH, '//div[@class="property-data-date font-xs"]')
        date_text = date_element.text
        match = re.search(r'\b\d{2}/\d{2}/\d{2}\b', date_text)
        if match:
            date_format = match.group()
            date.append(date_format)
        else:
            date.append('-')
    except NoSuchElementException:
        date.append('-')
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
        code.append('-')
    try:
        address_code = re.search(r'\b(?:ACT|NSW|NT|QLD|SA|TAS|VIC|WA)\b', address_text)
        if address_code:
            code_format = address_code.group()
            code.append(code_format)
        else:
            code.append('-')
    except NoSuchElementException:
        address.append('-')
        code.append('-')

 
    try:
        # Find price
        price_element = driver.find_element(By.XPATH, '//div[@class="price-value-container"]/div')
        price_text = price_element.text
        match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', price_text)
        if match:
            price_str = match.group().replace(',', '')
            price_usd = float(price_str)
            price_inr = price_usd * 83.12
            rounded = round(price_inr)
            price.append(f"{rounded} INR")
        else:
            price.append('-')
    except NoSuchElementException:
        price.append('-')


In [75]:
t11=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'date':date,'state_code':code,'price':price})
t11

,city,country,name,address,date,state_code,price
0,brisbane,australia,"Stylish City Getaway W/ Pool, Gym & Parking","Brisbane City, Queensland, Australia",12/25/23,-,976992 INR
1,brisbane,australia,New Farm|Scenic Views & Entertainment|Wifi,"Brisbane City QLD 4005, Australia",04/26/23,QLD,1080311 INR
2,brisbane,australia,Central Heights ~ 2Bed/2Bath/Car/Pool/Views ~ CBD,"Brisbane City, Queensland, Australia",04/25/23,-,905925 INR
3,brisbane,australia,The Brick Batch - Holiday Unit,"Bellara QLD 4507, Australia",04/25/23,QLD,239801 INR
4,brisbane,australia,CASSA ONE - Luxury River-View Apt In Brisbane,"Brisbane City, Queensland, Australia",08/01/23,-,595139 INR
...,...,...,...,...,...,...,...
303,brisbane,australia,Superb Apartment In 5* Zoetry Marigot Bay Reso...,"Saint Lucia, Brisbane, Australia",04/24/23,-,2003358 INR
304,brisbane,australia,Central 2-Bed Apartment With Gym And Rooftop Pool,"Brisbane City QLD 4006, Australia",04/25/23,QLD,636782 INR
305,brisbane,australia,Step Back Into Style ~ 2Bed/1Bath/1Car ~ New Farm,"Brisbane City QLD 4005, Australia",04/25/23,QLD,595139 INR
306,brisbane,australia,Lv80 River View Humble Abode + Free Parking On...,"Brisbane City, Queensland, Australia",07/04/23,-,564800 INR


In [76]:
# Table 2 code

city = []
country = []
name = []
address=[]
pet = []
bath = []
bed = []
area = []
deposit=[]
def measurement(sq_m):
    sq_ft = float(sq_m) * 10.7639  # 1 square meter = 10.7639 square feet
    return round(sq_ft,2)

for i in urls: # loop for each property card
    e = []
    driver.get(i)
    time.sleep(1)
    # Append city and country
    city.append('brisbane')
    country.append('australia')
    a = driver.find_element(By.XPATH, '//*[@id="main-info"]/section/div/div/rb-property-facilities/section')
    elements = a.find_elements(By.TAG_NAME, 'span')
    
    for i in elements:
        e.append(i.text)
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
    
    try:
        if e.index('Bed')>=0:
            bed.append(e[e.index('Bed')-1])
    except ValueError:
        bed.append('-')
    
    try:
        if e.index('Bath')>=0:
            bath.append(e[e.index('Bath')-1])
    except ValueError:
        bath.append('-')
    try:
        if 'Sq Ft' in e:
            area_index = e.index('Sq Ft')
            area_value = e[area_index - 1]
            area.append(area_value)
        elif 'Sq M' in e:
            area_index = e.index('Sq M')
            area_value_sq_m = e[area_index - 1]
            area_value = measurement(area_value_sq_m)
            area.append(area_value)
        else:
            area.append('-')
        
    except ValueError:
        area.append('-')
    try:
        # Check if 'Deposit' is in the list of property facilities
        if 'Deposit' in e:
            # Find the index of 'Deposit' and get the preceding element which contains the deposit amount
            deposit_index = e.index('Deposit')
            deposit_text = e[deposit_index - 1]
            
            # Extract deposit amount
            match = re.search(r'(\d+(\.\d+)?)', deposit_text)
            if match:
                dollar = float(match.group())
                inr = dollar * 83.12
                rounded = round(inr)
                deposit.append(f"{rounded} INR")
            else:
                deposit.append('-')
        else:
            deposit.append('-')  # If 'Deposit' is not found, append '-'
    except ValueError:
        deposit.append('-')    
    try:
        if e.index('Dogs') or e.index('Cats') >=0:
            pet.append(e[e.index('Allowed')])
    except ValueError:
        pet.append('-')
    
#     print(e)
    e.clear()# clear e for each itirations
        

In [77]:
y11=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'bed':bed,'bath':bath,'area':area,'pet':pet,'deposit':deposit})
y11

,city,country,name,address,bed,bath,area,pet,deposit
0,brisbane,australia,"Stylish City Getaway W/ Pool, Gym & Parking","Brisbane City, Queensland, Australia",Studio,-,-,-,-
1,brisbane,australia,New Farm|Scenic Views & Entertainment|Wifi,"Brisbane City QLD 4005, Australia",2,2,"1,023",-,-
2,brisbane,australia,Central Heights ~ 2Bed/2Bath/Car/Pool/Views ~ CBD,"Brisbane City, Queensland, Australia",2,2,958,-,-
3,brisbane,australia,The Brick Batch - Holiday Unit,"Bellara QLD 4507, Australia",1,1,-,-,-
4,brisbane,australia,CASSA ONE - Luxury River-View Apt In Brisbane,"Brisbane City, Queensland, Australia",1,1,-,-,-
...,...,...,...,...,...,...,...,...,...
303,brisbane,australia,Superb Apartment In 5* Zoetry Marigot Bay Reso...,"Saint Lucia, Brisbane, Australia",3,3,-,-,-
304,brisbane,australia,Central 2-Bed Apartment With Gym And Rooftop Pool,"Brisbane City QLD 4006, Australia",2,2,11,-,-
305,brisbane,australia,Step Back Into Style ~ 2Bed/1Bath/1Car ~ New Farm,"Brisbane City QLD 4005, Australia",2,1,915,-,-
306,brisbane,australia,Lv80 River View Humble Abode + Free Parking On...,"Brisbane City, Queensland, Australia",1,1,-,-,-


In [25]:
# Table 3 
city = []
country = []
name = []
address=[]
ac=[]
parking=[]
dishwasher=[]
fireplace=[]
Storage=[]
hardwood=[]
for i in urls:
    
    driver.get(i)
    time.sleep(1)
    city.append('brisbane')
    country.append('australia')
    
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
            

# capturing the amenities
    try:
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-rb-amenities/div/rb-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-mls-amenities/div/rb-mls-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
            
        try :
            if e.index('Air Conditioning')>=0:
                ac.append('Available')
        except ValueError:
            ac.append('Unavailable')

        try :
            if e.index('Parking')>=0:
                parking.append('Available')
        except ValueError:
            parking.append('Unavailable')

        try :
            if e.index('Dishwasher')>=0:
                dishwasher.append('Available')
        except ValueError:
            dishwasher.append('Unavailable')

        try :
            if e.index('Fireplace')>=0:
                fireplace.append('Available')
        except ValueError:
            fireplace.append('Unavailable')
        try :
            if e.index('Storage')>=0:
                Storage.append('Available')
        except ValueError:
            Storage.append('Unavailable')
        try :
            if e.index('Hardwood Floors')>=0:
                hardwood.append('Available')
        except ValueError:
            hardwood.append('Unavailable')
            
    except NoSuchElementException:
        ac.append('Unavailable')
        parking.append('Unavailable')
        dishwasher.append('Unavailable')
        fireplace.append('Unavailable')
        Storage.append('Unavailable')
        hardwood.append('Unavailable')
#         print('No emenities')
    e.clear()


In [26]:
u11= pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'ac': ac,'parking': parking,
                  'dishwasher': dishwasher,'fireplace': fireplace,'Storage':Storage,'hardwood':hardwood})
u11.head()

,city,country,name,address,ac,parking,dishwasher,fireplace,Storage,hardwood
0,brisbane,australia,"Stylish City Getaway W/ Pool, Gym & Parking","Brisbane City, Queensland, Australia",Available,Available,Available,Unavailable,Unavailable,Unavailable
1,brisbane,australia,New Farm|Scenic Views & Entertainment|Wifi,"Brisbane City QLD 4005, Australia",Available,Available,Available,Unavailable,Unavailable,Unavailable
2,brisbane,australia,Central Heights ~ 2Bed/2Bath/Car/Pool/Views ~ CBD,"Brisbane City, Queensland, Australia",Available,Available,Available,Unavailable,Unavailable,Unavailable
3,brisbane,australia,The Brick Batch - Holiday Unit,"Bellara QLD 4507, Australia",Available,Available,Unavailable,Unavailable,Unavailable,Unavailable
4,brisbane,australia,CASSA ONE - Luxury River-View Apt In Brisbane,"Brisbane City, Queensland, Australia",Available,Unavailable,Available,Unavailable,Unavailable,Unavailable


# melbourne

In [83]:
driver=webdriver.Chrome()
driver.get('https://rentberry.com/apartments/s/melbourne-vic-australia')
slct_type=driver.find_element(By.XPATH,'//*[@id="search-sidebar-wrapper"]/div[1]/rb-filter-bar/div[1]/div/div/div[2]/rb-filter-selection/rb-filter-selection-item/div/button')
slct_type.click()
time.sleep(2)
slct_apt=driver.find_element(By.XPATH,'//div[@class="filter-panel ng-star-inserted"]/div[1]/rb-filter-options/rb-filter-option[1]')
slct_apt.click()
time.sleep(2)
slct_save=driver.find_element(By.XPATH,'//button[@class="rb-button w-100"]')
slct_save.click()
time.sleep(2)

In [84]:

urls = []
start_page = 0
end_page = 13  # Adjust the end page number as needed

for page in range(start_page, end_page):
    elements = driver.find_elements(By.XPATH, '//a[@class="property-card-bottom-address__text link-ghost font-xs one-line-string"]')
    for element in elements:
        urls.append(element.get_attribute('href'))
        
    try:
        next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//a[@class="pagination-item"]/span[@class="pagination-arrow-next"]')))
        next_button.click()
        time.sleep(2)
    except Exception as e:
        print("Exception occurred:", e)
        break  # Break the loop if unable to click next button

In [86]:
city = []
country = []
name = []
date = []
address = []
price = []
code = []

for i in urls:  # loop for each property card
    driver.get(i)
    time.sleep(1)
    
    # Append city and country
    city.append('melbourne')
    country.append('australia')
    
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
    
    try:
        # Find date
        date_element = driver.find_element(By.XPATH, '//div[@class="property-data-date font-xs"]')
        date_text = date_element.text
        match = re.search(r'\b\d{2}/\d{2}/\d{2}\b', date_text)
        if match:
            date_format = match.group()
            date.append(date_format)
        else:
            date.append('-')
    except NoSuchElementException:
        date.append('-')
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
        code.append('-')
    try:
        address_code = re.search(r'\b(?:ACT|NSW|NT|QLD|SA|TAS|VIC|WA)\b', address_text)
        if address_code:
            code_format = address_code.group()
            code.append(code_format)
        else:
            code.append('-')
    except NoSuchElementException:
        address.append('-')
        code.append('-')

 
    try:
        # Find price
        price_element = driver.find_element(By.XPATH, '//div[@class="price-value-container"]/div')
        price_text = price_element.text
        match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', price_text)
        if match:
            price_str = match.group().replace(',', '')
            price_usd = float(price_str)
            price_inr = price_usd * 83.12
            rounded = round(price_inr)
            price.append(f"{rounded} INR")
        else:
            price.append('-')
    except NoSuchElementException:
        price.append('-')


In [87]:
t12=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'date':date,'state_code':code,'price':price})
t12

,city,country,name,address,date,state_code,price
0,melbourne,australia,Charming 1-Bed Central Studio Apartment,"Melbourne, Victoria, Australia",04/25/23,-,473784 INR
1,melbourne,australia,Martha Cove Apartment,"Safety Beach, Melbourne, Victoria, Australia",04/23/23,-,689896 INR
2,melbourne,australia,Corporate Keys Premier 2 Bedroom In The Hamptons,"Port Melbourne VIC 3207, Australia",04/23/23,VIC,689896 INR
3,melbourne,australia,Samma | Modern 2-BR King Bed Haven In Ivanhoe,"Ivanhoe, Norman Street, Ivanhoe VIC 3079, Aust...",02/05/24,VIC,747748 INR
4,melbourne,australia,A Lovely CBD Studio Right Next To Southern Cross,"Melbourne, Victoria, Australia",12/25/23,-,409117 INR
...,...,...,...,...,...,...,...
281,melbourne,australia,"A Lux 2BR Apt With Panoramic Views, FREE Parking","Southbank VIC 3006, Australia",07/04/23,VIC,833943 INR
282,melbourne,australia,Welcome To Your Sophisticated Oasis In The Hea...,"City of Melbourne, Victoria, Australia",09/16/23,-,564302 INR
283,melbourne,australia,Garfield · Stylish Apartment In Richmond,"Richmond VIC 3121, Australia",04/25/23,VIC,1999784 INR
284,melbourne,australia,"OCEANIC BEACH APARTMENT, PRIME POSITION TO ALL...","Sorrento VIC 3943, Australia",04/30/23,VIC,617831 INR


In [104]:
# Table 2 code

city = []
country = []
name = []
address=[]
pet = []
bath = []
bed = []
area = []
deposit=[]
def measurement(sq_m):
    sq_ft = float(sq_m) * 10.7639  # 1 square meter = 10.7639 square feet
    return round(sq_ft,2)

for i in urls: # loop for each property card
    e = []
    driver.get(i)
    time.sleep(1)
    # Append city and country
    city.append('melbourne')
    country.append('australia')
    a = driver.find_element(By.XPATH, '//*[@id="main-info"]/section/div/div/rb-property-facilities/section')
    elements = a.find_elements(By.TAG_NAME, 'span')
    
    for i in elements:
        e.append(i.text)
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
    
    try:
        if e.index('Bed')>=0:
            bed.append(e[e.index('Bed')-1])
    except ValueError:
        bed.append('-')
    
    try:
        if e.index('Bath')>=0:
            bath.append(e[e.index('Bath')-1])
    except ValueError:
        bath.append('-')
    try:
        if 'Sq Ft' in e:
            area_index = e.index('Sq Ft')
            area_value = e[area_index - 1]
            area.append(area_value)
        elif 'Sq M' in e:
            area_index = e.index('Sq M')
            area_value_sq_m = e[area_index - 1]
            area_value = measurement(area_value_sq_m)
            area.append(area_value)
        else:
            area.append('-')
        
    except ValueError:
        area.append('-')
    try:
        # Check if 'Deposit' is in the list of property facilities
        if 'Deposit' in e:
            # Find the index of 'Deposit' and get the preceding element which contains the deposit amount
            deposit_index = e.index('Deposit')
            deposit_text = e[deposit_index - 1]
            
            # Extract deposit amount
            match = re.search(r'(\d+(\.\d+)?)', deposit_text)
            if match:
                dollar = float(match.group())
                inr = dollar * 83.12
                rounded = round(inr)
                deposit.append(f"{rounded} INR")
            else:
                deposit.append('-')
        else:
            deposit.append('-')  # If 'Deposit' is not found, append '-'
    except ValueError:
        deposit.append('-')    
    try:
        if e.index('Dogs') or e.index('Cats') >=0:
            pet.append(e[e.index('Allowed')])
    except ValueError:
        pet.append('-')
    
#     print(e)
    e.clear()# clear e for each itirations
        

In [105]:
y12=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'bed':bed,'bath':bath,'area':area,'pet':pet,'deposit':deposit})
y12

,city,country,name,address,bed,bath,area,pet,deposit
0,melbourne,australia,Charming 1-Bed Central Studio Apartment,"Melbourne, Victoria, Australia",Studio,1,11,-,-
1,melbourne,australia,Martha Cove Apartment,"Safety Beach, Melbourne, Victoria, Australia",3,2,"1,442",-,-
2,melbourne,australia,Corporate Keys Premier 2 Bedroom In The Hamptons,"Port Melbourne VIC 3207, Australia",2,2,-,-,-
3,melbourne,australia,Samma | Modern 2-BR King Bed Haven In Ivanhoe,"Ivanhoe, Norman Street, Ivanhoe VIC 3079, Aust...",2,1,-,-,-
4,melbourne,australia,A Lovely CBD Studio Right Next To Southern Cross,"Melbourne, Victoria, Australia",Studio,1,431,-,-
...,...,...,...,...,...,...,...,...,...
281,melbourne,australia,"A Lux 2BR Apt With Panoramic Views, FREE Parking","Southbank VIC 3006, Australia",2,2,840,-,-
282,melbourne,australia,Welcome To Your Sophisticated Oasis In The Hea...,"City of Melbourne, Victoria, Australia",2,2,-,-,-
283,melbourne,australia,Garfield · Stylish Apartment In Richmond,"Richmond VIC 3121, Australia",1,1,538,-,-
284,melbourne,australia,"OCEANIC BEACH APARTMENT, PRIME POSITION TO ALL...","Sorrento VIC 3943, Australia",1,1,-,-,-


In [106]:
# Table 3 
city = []
country = []
name = []
address=[]
ac=[]
parking=[]
dishwasher=[]
fireplace=[]
Storage=[]
hardwood=[]
for i in urls:
    
    driver.get(i)
    time.sleep(1)
    city.append('melbourne')
    country.append('australia')
    
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
            

# capturing the amenities
    try:
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-rb-amenities/div/rb-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-mls-amenities/div/rb-mls-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
            
        try :
            if e.index('Air Conditioning')>=0:
                ac.append('Available')
        except ValueError:
            ac.append('Unavailable')

        try :
            if e.index('Parking')>=0:
                parking.append('Available')
        except ValueError:
            parking.append('Unavailable')

        try :
            if e.index('Dishwasher')>=0:
                dishwasher.append('Available')
        except ValueError:
            dishwasher.append('Unavailable')

        try :
            if e.index('Fireplace')>=0:
                fireplace.append('Available')
        except ValueError:
            fireplace.append('Unavailable')
        try :
            if e.index('Storage')>=0:
                Storage.append('Available')
        except ValueError:
            Storage.append('Unavailable')
        try :
            if e.index('Hardwood Floors')>=0:
                hardwood.append('Available')
        except ValueError:
            hardwood.append('Unavailable')
            
    except NoSuchElementException:
        ac.append('Unavailable')
        parking.append('Unavailable')
        dishwasher.append('Unavailable')
        fireplace.append('Unavailable')
        Storage.append('Unavailable')
        hardwood.append('Unavailable')
#         print('No emenities')
    e.clear()


In [107]:
u12= pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'ac': ac,'parking': parking,
                  'dishwasher': dishwasher,'fireplace': fireplace,'Storage':Storage,'hardwood':hardwood})
u12.head()

,city,country,name,address,ac,parking,dishwasher,fireplace,Storage,hardwood
0,melbourne,australia,Charming 1-Bed Central Studio Apartment,"Melbourne, Victoria, Australia",Available,Unavailable,Available,Unavailable,Unavailable,Unavailable
1,melbourne,australia,Martha Cove Apartment,"Safety Beach, Melbourne, Victoria, Australia",Available,Available,Available,Unavailable,Unavailable,Unavailable
2,melbourne,australia,Corporate Keys Premier 2 Bedroom In The Hamptons,"Port Melbourne VIC 3207, Australia",Available,Available,Available,Unavailable,Unavailable,Unavailable
3,melbourne,australia,Samma | Modern 2-BR King Bed Haven In Ivanhoe,"Ivanhoe, Norman Street, Ivanhoe VIC 3079, Aust...",Available,Available,Available,Unavailable,Unavailable,Unavailable
4,melbourne,australia,A Lovely CBD Studio Right Next To Southern Cross,"Melbourne, Victoria, Australia",Available,Unavailable,Available,Unavailable,Unavailable,Unavailable


# sydney

In [48]:
driver=webdriver.Chrome()
driver.get('https://rentberry.com/apartments/s/sydney-nsw-australia')
slct_type=driver.find_element(By.XPATH,'//*[@id="search-sidebar-wrapper"]/div[1]/rb-filter-bar/div[1]/div/div/div[2]/rb-filter-selection/rb-filter-selection-item/div/button')
slct_type.click()
time.sleep(2)
slct_apt=driver.find_element(By.XPATH,'//div[@class="filter-panel ng-star-inserted"]/div[1]/rb-filter-options/rb-filter-option[1]')
slct_apt.click()
time.sleep(2)
slct_save=driver.find_element(By.XPATH,'//button[@class="rb-button w-100"]')
slct_save.click()
time.sleep(2)

In [50]:

urls = []
start_page = 0
end_page = 15  # Adjust the end page number as needed

for page in range(start_page, end_page):
    elements = driver.find_elements(By.XPATH, '//a[@class="property-card-bottom-address__text link-ghost font-xs one-line-string"]')
    for element in elements:
        urls.append(element.get_attribute('href'))
        
    try:
        next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//a[@class="pagination-item"]/span[@class="pagination-arrow-next"]')))
        next_button.click()
        time.sleep(2)
    except Exception as e:
        print("Exception occurred:", e)
        break  # Break the loop if unable to click next button

In [51]:
city = []
country = []
name = []
date = []
address = []
price = []
code = []

for i in urls:  # loop for each property card
    driver.get(i)
    time.sleep(1)
    
    # Append city and country
    city.append('sydney')
    country.append('australia')
    
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
    
    try:
        # Find date
        date_element = driver.find_element(By.XPATH, '//div[@class="property-data-date font-xs"]')
        date_text = date_element.text
        match = re.search(r'\b\d{2}/\d{2}/\d{2}\b', date_text)
        if match:
            date_format = match.group()
            date.append(date_format)
        else:
            date.append('-')
    except NoSuchElementException:
        date.append('-')
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
        code.append('-')
    try:
        address_code = re.search(r'\b(?:ACT|NSW|NT|QLD|SA|TAS|VIC|WA)\b', address_text)
        if address_code:
            code_format = address_code.group()
            code.append(code_format)
        else:
            code.append('-')
    except NoSuchElementException:
        address.append('-')
        code.append('-')

 
    try:
        # Find price
        price_element = driver.find_element(By.XPATH, '//div[@class="price-value-container"]/div')
        price_text = price_element.text
        match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', price_text)
        if match:
            price_str = match.group().replace(',', '')
            price_usd = float(price_str)
            price_inr = price_usd * 83.12
            rounded = round(price_inr)
            price.append(f"{rounded} INR")
        else:
            price.append('-')
    except NoSuchElementException:
        price.append('-')


In [52]:
t13=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'date':date,'state_code':code,'price':price})
t13.head(2)
print(t13.shape)

(330, 7)


In [54]:
# Table 2 code

city = []
country = []
name = []
address=[]
pet = []
bath = []
bed = []
area = []
deposit=[]
def measurement(sq_m):
    sq_ft = float(sq_m) * 10.7639  # 1 square meter = 10.7639 square feet
    return round(sq_ft,2)

for i in urls: # loop for each property card
    e = []
    driver.get(i)
    time.sleep(1)
    # Append city and country
    city.append('sydney')
    country.append('australia')
    a = driver.find_element(By.XPATH, '//*[@id="main-info"]/section/div/div/rb-property-facilities/section')
    elements = a.find_elements(By.TAG_NAME, 'span')
    
    for i in elements:
        e.append(i.text)
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
    
    try:
        if e.index('Bed')>=0:
            bed.append(e[e.index('Bed')-1])
    except ValueError:
        bed.append('-')
    
    try:
        if e.index('Bath')>=0:
            bath.append(e[e.index('Bath')-1])
    except ValueError:
        bath.append('-')
    try:
        if 'Sq Ft' in e:
            area_index = e.index('Sq Ft')
            area_value = e[area_index - 1]
            area.append(area_value)
        elif 'Sq M' in e:
            area_index = e.index('Sq M')
            area_value_sq_m = e[area_index - 1]
            area_value = measurement(area_value_sq_m)
            area.append(area_value)
        else:
            area.append('-')
        
    except ValueError:
        area.append('-')
    try:
        # Check if 'Deposit' is in the list of property facilities
        if 'Deposit' in e:
            # Find the index of 'Deposit' and get the preceding element which contains the deposit amount
            deposit_index = e.index('Deposit')
            deposit_text = e[deposit_index - 1]
            
            # Extract deposit amount
            match = re.search(r'(\d+(\.\d+)?)', deposit_text)
            if match:
                dollar = float(match.group())
                inr = dollar * 83.12
                rounded = round(inr)
                deposit.append(f"{rounded} INR")
            else:
                deposit.append('-')
        else:
            deposit.append('-')  # If 'Deposit' is not found, append '-'
    except ValueError:
        deposit.append('-')    
    try:
        if e.index('Dogs') or e.index('Cats') >=0:
            pet.append(e[e.index('Allowed')])
    except ValueError:
        pet.append('-')
    
#     print(e)
    e.clear()# clear e for each itirations
        

In [55]:
y13=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'bed':bed,'bath':bath,'area':area,'pet':pet,'deposit':deposit})
y13.head()
print(y13.shape)

(330, 9)


In [57]:
# Table 3 
city = []
country = []
name = []
address=[]
ac=[]
parking=[]
dishwasher=[]
fireplace=[]
Storage=[]
hardwood=[]
for i in urls:
    
    driver.get(i)
    time.sleep(1)
    city.append('sydney')
    country.append('australia')
    
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
            

# capturing the amenities
    try:
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-rb-amenities/div/rb-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-mls-amenities/div/rb-mls-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
            
        try :
            if e.index('Air Conditioning')>=0:
                ac.append('Available')
        except ValueError:
            ac.append('Unavailable')

        try :
            if e.index('Parking')>=0:
                parking.append('Available')
        except ValueError:
            parking.append('Unavailable')

        try :
            if e.index('Dishwasher')>=0:
                dishwasher.append('Available')
        except ValueError:
            dishwasher.append('Unavailable')

        try :
            if e.index('Fireplace')>=0:
                fireplace.append('Available')
        except ValueError:
            fireplace.append('Unavailable')
        try :
            if e.index('Storage')>=0:
                Storage.append('Available')
        except ValueError:
            Storage.append('Unavailable')
        try :
            if e.index('Hardwood Floors')>=0:
                hardwood.append('Available')
        except ValueError:
            hardwood.append('Unavailable')
            
    except NoSuchElementException:
        ac.append('Unavailable')
        parking.append('Unavailable')
        dishwasher.append('Unavailable')
        fireplace.append('Unavailable')
        Storage.append('Unavailable')
        hardwood.append('Unavailable')
#         print('No emenities')
    e.clear()


In [58]:
u13= pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'ac': ac,'parking': parking,
                  'dishwasher': dishwasher,'fireplace': fireplace,'Storage':Storage,'hardwood':hardwood})
u13.head()
print(u13.shape)

(330, 10)


# okyo-japan

In [33]:
driver=webdriver.Chrome()
driver.get('https://rentberry.com/apartments/s/tokyo-japan')
slct_type=driver.find_element(By.XPATH,'//*[@id="search-sidebar-wrapper"]/div[1]/rb-filter-bar/div[1]/div/div/div[2]/rb-filter-selection/rb-filter-selection-item/div/button')
slct_type.click()
time.sleep(1)


In [34]:
slct_apt=driver.find_element(By.XPATH,'//div[@class="filter-panel ng-star-inserted"]/div[1]/rb-filter-options/rb-filter-option[1]')
slct_apt.click()
time.sleep(1)
slct_save=driver.find_element(By.XPATH,'//button[@class="rb-button w-100"]')
slct_save.click()
time.sleep(1)

In [37]:

urls = []
start_page = 0
end_page = 15  # Adjust the end page number as needed

for page in range(start_page, end_page):
    elements = driver.find_elements(By.XPATH, '//a[@class="property-card-bottom-address__text link-ghost font-xs one-line-string"]')
    for element in elements:
        urls.append(element.get_attribute('href'))
        
    try:
        next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//a[@class="pagination-item"]/span[@class="pagination-arrow-next"]')))
        next_button.click()
        time.sleep(2)
    except Exception as e:
        print("Exception occurred:", e)
        break  # Break the loop if unable to click next button

In [38]:

city = []
country = []
name = []
address=[]
date=[]
pet = []
bath = []
bed = []
area = []
deposit=[]
price = []
code = []
ac=[]
parking=[]
dishwasher=[]
fireplace=[]
Storage=[]
hardwood=[]

def measurement(sq_m):
    sq_ft = float(sq_m) * 10.7639  # 1 square meter = 10.7639 square feet
    return round(sq_ft,2)

for i in urls[:10]: # loop for each property card
    e = []
    driver.get(i)
    time.sleep(1)
    # Append city and country
    city.append('Austin')
    country.append('US')
#     try:
#         # Find price
#         price_element = driver.find_element(By.XPATH, '//div[@class="price-value-container"]/div')
#         price_text = price_element.text
#         match = re.search(r'¥\s*([\d,.]+)([KM]?)', price_text)
#         if match:
#             price_str = match.group(1).replace(',', '')  # Remove commas
#             multiplier = match.group(2)
#             if multiplier == 'K':
#                 price_yen = float(price_str) * 1000
#             elif multiplier == 'M':
#                 price_yen = float(price_str) * 1000000
#             else:
#                 price_yen = float(price_str)
#             # Updated exchange rate: 1 JPY = 0.55 INR
#             price_inr = price_yen * 0.55
#             rounded = round(price_inr)
#             price.append(f"{rounded} INR")
#         else:
#             price.append('-')
#     except NoSuchElementException:
#         price.append('-')    
 
    
    try:
        # Find price
        price_element = driver.find_element(By.XPATH, '//div[@class="price-value-container"]/div')
        price_text = price_element.text
        matches = re.finditer(r'¥\s*([\d,.]+)M?', price_text)
        if matches:
            for match in matches:
                price_str = match.group(1).replace(',', '')
                price_jpy = float(price_str) * 1000000  # converting to JPY
                price_usd = price_jpy * 0.0091  # assuming 1 JPY = 0.0091 USD
                price.append(round(price_usd))
        else:
            price.append('-')
    except NoSuchElementException:
        price.append('-')
    try:
        # Find date
        date_element = driver.find_element(By.XPATH, '//div[@class="property-data-date font-xs"]')
        date_text = date_element.text
        match = re.search(r'\b\d{2}/\d{2}/\d{2}\b', date_text)
        if match:
            date_format = match.group()
            date.append(date_format)
        else:
            date.append('-')
    except NoSuchElementException:
        date.append('-')
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
        code.append('-')
    try:
        address_code = re.search(r'\b(?:AL|AK|AZ|AR|CA|CO|CT|DE|FL|GA|HI|ID|IL|IN|IA|KS|KY|LA|ME|MD|MA|MI|MN|MS|MO|MT|NE|NV|NH|NJ|NM|NY|NC|ND|OH|OK|OR|PA|RI|SC|SD|TN|TX|UT|VT|VA|WA|WV|WI|WY)\b', address_text)
        if address_code:
            code_format = address_code.group()
            code.append(code_format)
        else:
            code.append('-')
    except NoSuchElementException:
        address.append('-')
        code.append('-')
        
    a = driver.find_element(By.XPATH, '//*[@id="main-info"]/section/div/div/rb-property-facilities/section')
    elements = a.find_elements(By.TAG_NAME, 'span')
    
    for i in elements:
        e.append(i.text)
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    
    try:
        if e.index('Bed')>=0:
            bed.append(e[e.index('Bed')-1])
    except ValueError:
        bed.append('-')
    
    try:
        if e.index('Bath')>=0:
            bath.append(e[e.index('Bath')-1])
    except ValueError:
        bath.append('-')
    try:
        if 'Sq Ft' in e:
            area_index = e.index('Sq Ft')
            area_value = e[area_index - 1]
            area.append(area_value)
        elif 'Sq M' in e:
            area_index = e.index('Sq M')
            area_value_sq_m = e[area_index - 1]
            area_value = measurement(area_value_sq_m)
            area.append(area_value)
        else:
            area.append('-')
        
    except ValueError:
        area.append('-')
    try:
        # Check if 'Deposit' is in the list of property facilities
        if 'Deposit' in e:
            # Find the index of 'Deposit' and get the preceding element which contains the deposit amount
            deposit_index = e.index('Deposit')
            deposit_text = e[deposit_index - 1]
            
            # Extract deposit amount
            match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', deposit_text)
            if match:
                price_str = match.group().replace(',', '')
                price_usd = float(price_str)
                price_inr = price_usd * 83.12
                rounded = round(price_inr)
                deposit.append(f"{rounded}")
            else:
                deposit.append('-')
        else:
            deposit.append('-')  # If 'Deposit' is not found, append '-'
    except ValueError:
        deposit.append('-') 

    try:
        # Check if 'Deposit' is in the list of property facilities
        if 'Deposit' in e:
            # Find the index of 'Deposit' and get the preceding element which contains the deposit amount
            deposit_index = e.index('Deposit')
            deposit_text = e[deposit_index - 1]
            # Extract deposit amount
            matches = re.finditer(r'¥\s*([\d,.]+)M?', deposit_text)
            if matches:
                for match in matches:
                    deposit_str = match.group(1).replace(',', '')  # Remove commas
                    deposit_jpy = float(deposit_str) * 1000000  # Convert to JPY
                    # Updated exchange rate: 1 JPY = 0.55 INR
                    deposit_inr = deposit_jpy * 0.55
                    rounded = round(deposit_inr)
                    deposit.append(f"{rounded} INR")
            else:
                deposit.append('-')
        else:
            deposit.append('-')  # If 'Deposit' is not found, append '-'
    except ValueError:
        deposit.append('-')
        
        
        
        
        
    try:
        if e.index('Dogs') or e.index('Cats') >=0:
            pet.append(e[e.index('Allowed')])
    except ValueError:
        pet.append('-')
# capturing the amenities
    try:
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-rb-amenities/div/rb-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-mls-amenities/div/rb-mls-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
            
        try :
            if e.index('Air Conditioning')>=0:
                ac.append('Available')
        except ValueError:
            ac.append('Unavailable')

        try :
            if e.index('Parking')>=0:
                parking.append('Available')
        except ValueError:
            parking.append('Unavailable')

        try :
            if e.index('Dishwasher')>=0:
                dishwasher.append('Available')
        except ValueError:
            dishwasher.append('Unavailable')

        try :
            if e.index('Fireplace')>=0:
                fireplace.append('Available')
        except ValueError:
            fireplace.append('Unavailable')
        try :
            if e.index('Storage')>=0:
                Storage.append('Available')
        except ValueError:
            Storage.append('Unavailable')
        try :
            if e.index('Hardwood Floors')>=0:
                hardwood.append('Available')
        except ValueError:
            hardwood.append('Unavailable')
            
    except NoSuchElementException:
        ac.append('Unavailable')
        parking.append('Unavailable')
        dishwasher.append('Unavailable')
        fireplace.append('Unavailable')
        Storage.append('Unavailable')
        hardwood.append('Unavailable')
    
#     print(e)
    e.clear()# clear e for each itirations
        

In [39]:
tok=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'bed':bed,'bath':bath,'area':area,'pet':pet,
                 'ac': ac,'parking': parking,'state_code':code,'price':price,'date':date,
                  'dishwasher': dishwasher,'fireplace': fireplace,'storage':Storage,'hardwood_floor':hardwood,'deposit':deposit})
tok


ValueError: All arrays must be of the same length

In [79]:
city = []
country = []
name = []
date = []
address = []
price = []
code = []

for i in urls:  # loop for each property card
    driver.get(i)
    time.sleep(1)
    
    # Append city and country
    city.append('tokyo')
    country.append('japan')
    
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
    
    try:
        # Find date
        date_element = driver.find_element(By.XPATH, '//div[@class="property-data-date font-xs"]')
        date_text = date_element.text
        match = re.search(r'\b\d{2}/\d{2}/\d{2}\b', date_text)
        if match:
            date_format = match.group()
            date.append(date_format)
        else:
            date.append('-')
    except NoSuchElementException:
        date.append('-')
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
        code.append('-')
    try:
        address_code = re.search(r'\b(?:Hokkaido|Aomori|Iwate|Miyagi|Akita|Yamagata|Fukushima|Ibaraki|Tochigi|Gunma|Saitama|Chiba|Tokyo|Kanagawa|Niigata|Toyama|Ishikawa|Fukui|Yamanashi|Nagano|Gifu|Shizuoka|Aichi|Mie|Shiga|Kyoto|Osaka|Hyogo|Nara|Wakayama|Tottori|Shimane|Okayama|Hiroshima|Yamaguchi|Tokushima|Kagawa|Ehime|Kochi|Fukuoka|Saga|Nagasaki|Kumamoto|Oita|Miyazaki|Kagoshima|Okinawa)\b', address_text)
        if address_code:
            code_format = address_code.group()
            code.append(code_format)
        else:
            code.append('-')
    except NoSuchElementException:
        address.append('-')
        code.append('-')

 
    try:
        # Find price
        price_element = driver.find_element(By.XPATH, '//div[@class="price-value-container"]/div')
        price_text = price_element.text
        match = re.search(r'¥\s*([\d,.]+)([KM]?)', price_text)
        if match:
            price_str = match.group(1).replace(',', '')  # Remove commas
            multiplier = match.group(2)
            if multiplier == 'K':
                price_yen = float(price_str) * 1000
            elif multiplier == 'M':
                price_yen = float(price_str) * 1000000
            else:
                price_yen = float(price_str)
            # Updated exchange rate: 1 JPY = 0.55 INR
            price_inr = price_yen * 0.55
            rounded = round(price_inr)
            price.append(f"{rounded} INR")
        else:
            price.append('-')
    except NoSuchElementException:
        price.append('-')


In [80]:
t14=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'date':date,'state_code':code,'price':price})
t14
# 'print(t14.shape)

,city,country,name,address,date,state_code,price
0,tokyo,japan,SPICA 105 / Kawaguchi Saitama,"Kawaguchi, Saitama Prefecture, Japan",04/23/23,Saitama,229656 INR
1,tokyo,japan,There Are 206 Rooms Where You Can Enjoy Movies...,"Shinagawa, Japan",04/30/23,-,220087 INR
2,tokyo,japan,Riverside Home In SHINJUKU,"Shinjuku, Kamiochiai 1-chome, Shinjuku, JP",03/08/19,-,66000 INR
3,tokyo,japan,Condominium That Wins The Exam Corona Disinfec...,"Setagaya, Japan",04/30/23,-,459313 INR
4,tokyo,japan,Vrbo Property,"Setagaya, 154-0011, Japan",10/30/23,-,270324 INR
...,...,...,...,...,...,...,...
325,tokyo,japan,Studio Apartment In Minato,MQ87+X3 Tokyo,-,Tokyo,81433 INR
326,tokyo,japan,NOMAD Bellheim,27-20-303 Asahicho,04/24/23,-,157888 INR
327,tokyo,japan,LA302 新宿の新しく快適なお部屋。都内観光地にもアクセス良好で、便利な環境です。,"Shibuya, 151-0071, Japan",04/25/23,-,478451 INR
328,tokyo,japan,Near UenoAsakusa3mins To Skytree60 Cozy Room /...,"Sumida, Japan",04/25/23,-,545433 INR


In [132]:
# Table 2 code

city = []
country = []
name = []
address=[]
pet = []
bath = []
bed = []
area = []
deposit=[]
def measurement(sq_m):
    sq_ft = float(sq_m) * 10.7639  # 1 square meter = 10.7639 square feet
    return round(sq_ft,2)

for i in urls: # loop for each property card
    e = []
    driver.get(i)
    time.sleep(1)
    # Append city and country
    city.append('tokyo')
    country.append('japan')
    a = driver.find_element(By.XPATH, '//*[@id="main-info"]/section/div/div/rb-property-facilities/section')
    elements = a.find_elements(By.TAG_NAME, 'span')
    
    for i in elements:
        e.append(i.text)
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
    
    try:
        if e.index('Bed')>=0:
            bed.append(e[e.index('Bed')-1])
    except ValueError:
        bed.append('-')
    
    try:
        if e.index('Bath')>=0:
            bath.append(e[e.index('Bath')-1])
    except ValueError:
        bath.append('-')
    try:
        if 'Sq Ft' in e:
            area_index = e.index('Sq Ft')
            area_value = e[area_index - 1]
            area.append(area_value)
        elif 'Sq M' in e:
            area_index = e.index('Sq M')
            area_value_sq_m = e[area_index - 1]
            area_value = measurement(area_value_sq_m)
            area.append(area_value)
        else:
            area.append('-')
        
    except ValueError:
        area.append('-')


    try:
        # Check if 'Deposit' is in the list of property facilities
        if 'Deposit' in e:
            # Find the index of 'Deposit' and get the preceding element which contains the deposit amount
            deposit_index = e.index('Deposit')
            deposit_text = e[deposit_index - 1]
            # Extract deposit amount
            match = re.search(r'¥\s*([\d,.]+)([KM]?)', deposit_text)
            if match:
                deposit_str = match.group(1).replace(',', '')  # Remove commas
                deposit_yen = float(deposit_str)
                # Updated exchange rate: 1 JPY = 0.55 INR
                deposit_inr = deposit_yen * 0.55
                rounded = round(deposit_inr)
                deposit.append(f"{rounded} INR")
            else:
                deposit.append('-')
        else:
            deposit.append('-')  # If 'Deposit' is not found, append '-'
    except ValueError:
        deposit.append('-')
    
    try:
        if e.index('Dogs') or e.index('Cats') >=0:
            pet.append(e[e.index('Allowed')])
    except ValueError:
        pet.append('-')
    
#     print(e)
    e.clear()# clear e for each itirations
        

In [133]:
y14=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'bed':bed,'bath':bath,'area':area,'pet':pet,'deposit':deposit})
y14

,city,country,name,address,bed,bath,area,pet,deposit
0,tokyo,japan,SPICA 105 / Kawaguchi Saitama,"Kawaguchi, Saitama Prefecture, Japan",2,1,377,-,-
1,tokyo,japan,There Are 206 Rooms Where You Can Enjoy Movies...,"Shinagawa, Japan",2,1,431,-,-
2,tokyo,japan,Riverside Home In SHINJUKU,"Shinjuku, Kamiochiai 1-chome, Shinjuku, JP",1,1,-,-,9900 INR
3,tokyo,japan,Condominium That Wins The Exam Corona Disinfec...,"Setagaya, Japan",1,1,280,-,-
4,tokyo,japan,Vrbo Property,"Setagaya, 154-0011, Japan",2,1,366,-,-
...,...,...,...,...,...,...,...,...,...
325,tokyo,japan,Twin Room / Shinjuku-Ku Tokyo,"Shinjuku, Japan",1,1,118,-,-
326,tokyo,japan,Studio Apartment In Minato,MQ87+X3 Tokyo,Studio,-,172.22,-,-
327,tokyo,japan,NOMAD Bellheim,27-20-303 Asahicho,Studio,-,-,-,-
328,tokyo,japan,LA302 新宿の新しく快適なお部屋。都内観光地にもアクセス良好で、便利な環境です。,"Shibuya, 151-0071, Japan",1,2,269,-,-


In [134]:
# Table 3 
city = []
country = []
name = []
address=[]
ac=[]
parking=[]
dishwasher=[]
fireplace=[]
Storage=[]
hardwood=[]
for i in urls:
    
    driver.get(i)
    time.sleep(1)
    city.append('tokyo')
    country.append('japan')
    
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
            

# capturing the amenities
    try:
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-rb-amenities/div/rb-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-mls-amenities/div/rb-mls-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
            
        try :
            if e.index('Air Conditioning')>=0:
                ac.append('Available')
        except ValueError:
            ac.append('Unavailable')

        try :
            if e.index('Parking')>=0:
                parking.append('Available')
        except ValueError:
            parking.append('Unavailable')

        try :
            if e.index('Dishwasher')>=0:
                dishwasher.append('Available')
        except ValueError:
            dishwasher.append('Unavailable')

        try :
            if e.index('Fireplace')>=0:
                fireplace.append('Available')
        except ValueError:
            fireplace.append('Unavailable')
        try :
            if e.index('Storage')>=0:
                Storage.append('Available')
        except ValueError:
            Storage.append('Unavailable')
        try :
            if e.index('Hardwood Floors')>=0:
                hardwood.append('Available')
        except ValueError:
            hardwood.append('Unavailable')
            
    except NoSuchElementException:
        ac.append('Unavailable')
        parking.append('Unavailable')
        dishwasher.append('Unavailable')
        fireplace.append('Unavailable')
        Storage.append('Unavailable')
        hardwood.append('Unavailable')
#         print('No emenities')
    e.clear()


In [135]:
u14= pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'ac': ac,'parking': parking,
                  'dishwasher': dishwasher,'fireplace': fireplace,'Storage':Storage,'hardwood':hardwood})
u14.head()

,city,country,name,address,ac,parking,dishwasher,fireplace,Storage,hardwood
0,tokyo,japan,SPICA 105 / Kawaguchi Saitama,"Kawaguchi, Saitama Prefecture, Japan",Available,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable
1,tokyo,japan,There Are 206 Rooms Where You Can Enjoy Movies...,"Shinagawa, Japan",Available,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable
2,tokyo,japan,Riverside Home In SHINJUKU,"Shinjuku, Kamiochiai 1-chome, Shinjuku, JP",Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable
3,tokyo,japan,Condominium That Wins The Exam Corona Disinfec...,"Setagaya, Japan",Available,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable
4,tokyo,japan,Vrbo Property,"Setagaya, 154-0011, Japan",Available,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable


# Perth

In [121]:
driver=webdriver.Chrome()
driver.get('https://rentberry.com/apartments/s/perth-wa-australia')
slct_type=driver.find_element(By.XPATH,'//*[@id="search-sidebar-wrapper"]/div[1]/rb-filter-bar/div[1]/div/div/div[2]/rb-filter-selection/rb-filter-selection-item/div/button')
slct_type.click()
time.sleep(2)


In [122]:
slct_apt=driver.find_element(By.XPATH,'//div[@class="filter-panel ng-star-inserted"]/div[1]/rb-filter-options/rb-filter-option[1]')
slct_apt.click()
time.sleep(2)
slct_save=driver.find_element(By.XPATH,'//button[@class="rb-button w-100"]')
slct_save.click()
time.sleep(2)

In [124]:

urls = []
start_page = 0
end_page = 15  # Adjust the end page number as needed

for page in range(start_page, end_page):
    elements = driver.find_elements(By.XPATH, '//a[@class="property-card-bottom-address__text link-ghost font-xs one-line-string"]')
    for element in elements:
        urls.append(element.get_attribute('href'))
        
    try:
        next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//a[@class="pagination-item"]/span[@class="pagination-arrow-next"]')))
        next_button.click()
        time.sleep(2)
    except Exception as e:
        print("Exception occurred:", e)
        break  # Break the loop if unable to click next button

In [99]:
city = []
country = []
name = []
date = []
address = []
price = []
code = []

for i in urls:  # loop for each property card
    driver.get(i)
    time.sleep(1)
    
    # Append city and country
    city.append('perth')
    country.append('australia')
    
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
    
    try:
        # Find date
        date_element = driver.find_element(By.XPATH, '//div[@class="property-data-date font-xs"]')
        date_text = date_element.text
        match = re.search(r'\b\d{2}/\d{2}/\d{2}\b', date_text)
        if match:
            date_format = match.group()
            date.append(date_format)
        else:
            date.append('-')
    except NoSuchElementException:
        date.append('-')
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
        code.append('-')
    try:
        address_code = re.search(r'\b(?:ACT|NSW|NT|QLD|SA|TAS|VIC|WA)\b', address_text)
        if address_code:
            code_format = address_code.group()
            code.append(code_format)
        else:
            code.append('-')
    except NoSuchElementException:
        address.append('-')
        code.append('-')

 
    try:
        # Find price
        price_element = driver.find_element(By.XPATH, '//div[@class="price-value-container"]/div')
        price_text = price_element.text
        match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', price_text)
        if match:
            price_str = match.group().replace(',', '')
            price_usd = float(price_str)
            price_inr = price_usd * 83.12
            rounded = round(price_inr)
            price.append(f"{rounded} INR")
        else:
            price.append('-')
    except NoSuchElementException:
        price.append('-')


In [100]:
t15=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'date':date,'state_code':code,'price':price})
t15.head(2)
print(t15.shape)

(330, 7)


In [127]:
# Table 2 code

city = []
country = []
name = []
address=[]
pet = []
bath = []
bed = []
area = []
deposit=[]
def measurement(sq_m):
    sq_ft = float(sq_m) * 10.7639  # 1 square meter = 10.7639 square feet
    return round(sq_ft,2)

for i in urls: # loop for each property card
    e = []
    driver.get(i)
    time.sleep(1)
    # Append city and country
    city.append('perth')
    country.append('australia')
    a = driver.find_element(By.XPATH, '//*[@id="main-info"]/section/div/div/rb-property-facilities/section')
    elements = a.find_elements(By.TAG_NAME, 'span')
    
    for i in elements:
        e.append(i.text)
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
    
    try:
        if e.index('Bed')>=0:
            bed.append(e[e.index('Bed')-1])
    except ValueError:
        bed.append('-')
    
    try:
        if e.index('Bath')>=0:
            bath.append(e[e.index('Bath')-1])
    except ValueError:
        bath.append('-')
    try:
        if 'Sq Ft' in e:
            area_index = e.index('Sq Ft')
            area_value = e[area_index - 1]
            area.append(area_value)
        elif 'Sq M' in e:
            area_index = e.index('Sq M')
            area_value_sq_m = e[area_index - 1]
            area_value = measurement(area_value_sq_m)
            area.append(area_value)
        else:
            area.append('-')
        
    except ValueError:
        area.append('-')
    try:
        # Check if 'Deposit' is in the list of property facilities
        if 'Deposit' in e:
            # Find the index of 'Deposit' and get the preceding element which contains the deposit amount
            deposit_index = e.index('Deposit')
            deposit_text = e[deposit_index - 1]
            
            # Extract deposit amount
            match = re.search(r'(\d+(\.\d+)?)', deposit_text)
            if match:
                dollar = float(match.group())
                inr = dollar * 83.12
                rounded = round(inr)
                deposit.append(f"{rounded} INR")
            else:
                deposit.append('-')
        else:
            deposit.append('-')  # If 'Deposit' is not found, append '-'
    except ValueError:
        deposit.append('-')    
    try:
        if e.index('Dogs') or e.index('Cats') >=0:
            pet.append(e[e.index('Allowed')])
    except ValueError:
        pet.append('-')
    
#     print(e)
    e.clear()# clear e for each itirations
        

In [128]:
y15=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'bed':bed,'bath':bath,'area':area,'pet':pet,'deposit':deposit})
y15.head()

,city,country,name,address,bed,bath,area,pet,deposit
0,perth,australia,The Lemon Tree Studio,"Kensington WA 6151, Australia",1,1,258,-,-
1,perth,australia,Beach Sunsets @Scarborough Beach,"Scarborough WA 6019, Australia",2,1,635,-,-
2,perth,australia,Beautifully Renovated Studio Apartment In Frem...,"Fremantle, City of Fremantle, Australia",Studio,1,538,-,-
3,perth,australia,Bambino 438 - Modern 2 Bedroom Apartment Close...,"Subiaco WA 6008, Australia",2,1,-,-,-
4,perth,australia,"Spacious Lofty Apartment, Stunning Views Overl...","Guildford WA 6935, Australia",2,1,"1,076",-,-


In [103]:
# Table 3 
city = []
country = []
name = []
address=[]
ac=[]
parking=[]
dishwasher=[]
fireplace=[]
Storage=[]
hardwood=[]
for i in urls:
    
    driver.get(i)
    time.sleep(1)
    city.append('perth')
    country.append('australia')
    
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
            

# capturing the amenities
    try:
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-rb-amenities/div/rb-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-mls-amenities/div/rb-mls-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
            
        try :
            if e.index('Air Conditioning')>=0:
                ac.append('Available')
        except ValueError:
            ac.append('Unavailable')

        try :
            if e.index('Parking')>=0:
                parking.append('Available')
        except ValueError:
            parking.append('Unavailable')

        try :
            if e.index('Dishwasher')>=0:
                dishwasher.append('Available')
        except ValueError:
            dishwasher.append('Unavailable')

        try :
            if e.index('Fireplace')>=0:
                fireplace.append('Available')
        except ValueError:
            fireplace.append('Unavailable')
        try :
            if e.index('Storage')>=0:
                Storage.append('Available')
        except ValueError:
            Storage.append('Unavailable')
        try :
            if e.index('Hardwood Floors')>=0:
                hardwood.append('Available')
        except ValueError:
            hardwood.append('Unavailable')
            
    except NoSuchElementException:
        ac.append('Unavailable')
        parking.append('Unavailable')
        dishwasher.append('Unavailable')
        fireplace.append('Unavailable')
        Storage.append('Unavailable')
        hardwood.append('Unavailable')
#         print('No emenities')
    e.clear()


In [104]:
u15= pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'ac': ac,'parking': parking,
                  'dishwasher': dishwasher,'fireplace': fireplace,'Storage':Storage,'hardwood':hardwood})
u15.head()
print(u15.shape)

(330, 10)


# amsterdam-netherlands

In [37]:
driver=webdriver.Chrome()
driver.get('https://rentberry.com/apartments/s/amsterdam-netherlands')
slct_type=driver.find_element(By.XPATH,'//*[@id="search-sidebar-wrapper"]/div[1]/rb-filter-bar/div[1]/div/div/div[2]/rb-filter-selection/rb-filter-selection-item/div/button')
slct_type.click()
time.sleep(2)


In [38]:
slct_apt=driver.find_element(By.XPATH,'//div[@class="filter-panel ng-star-inserted"]/div[1]/rb-filter-options/rb-filter-option[1]')
slct_apt.click()
time.sleep(2)
slct_save=driver.find_element(By.XPATH,'//button[@class="rb-button w-100"]')
slct_save.click()
time.sleep(2)

In [45]:

urls = []
start_page = 0
end_page = 15  # Adjust the end page number as needed

for page in range(start_page, end_page):
    elements = driver.find_elements(By.XPATH, '//a[@class="property-card-bottom-address__text link-ghost font-xs one-line-string"]')
    for element in elements:
        urls.append(element.get_attribute('href'))
        
    try:
        next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//a[@class="pagination-item"]/span[@class="pagination-arrow-next"]')))
        next_button.click()
        time.sleep(2)
    except Exception as e:
        print("Exception occurred:", e)
        break  # Break the loop if unable to click next button

In [101]:
city = []
country = []
name = []
date = []
address = []
price = []
code = []

for i in urls:  # loop for each property card
    driver.get(i)
    time.sleep(0.7)
    
    # Append city and country
    city.append('Amsterdam')
    country.append('Netherland')
    
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
    
    try:
        # Find date
        date_element = driver.find_element(By.XPATH, '//div[@class="property-data-date font-xs"]')
        date_text = date_element.text
        match = re.search(r'\b\d{2}/\d{2}/\d{2}\b', date_text)
        if match:
            date_format = match.group()
            date.append(date_format)
        else:
            date.append('-')
    except NoSuchElementException:
        date.append('-')
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
        code.append('-')
    try:
        # Modify the regular expression to match any two capital letters as state codes
        address_code = re.search(r'\b[A-Z]{2}\b', address_text)
        if address_code:
            code_format = address_code.group()
            code.append(code_format)
        else:
            code.append('-')
    except NoSuchElementException:
        address.append('-')
        code.append('-')

 
    try:
        # Find price
        price_element = driver.find_element(By.XPATH, '//div[@class="price-value-container"]/div')
        price_text = price_element.text
        match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', price_text)
        if match:
            price_str = match.group().replace(',', '')
            price_usd = float(price_str)
            price_inr = price_usd * 90.73
            rounded = round(price_inr)
            price.append(f"{rounded} INR")
        else:
            price.append('-')
    except NoSuchElementException:
        price.append('-')


In [46]:
# Table 2 code

city = []
country = []
name = []
address=[]
date=[]
pet = []
bath = []
bed = []
area = []
deposit=[]
price = []
code = []
ac=[]
parking=[]
dishwasher=[]
fireplace=[]
Storage=[]
hardwood=[]

def measurement(sq_m):
    sq_ft = float(sq_m) * 10.7639  # 1 square meter = 10.7639 square feet
    return round(sq_ft,2)

for i in urls: # loop for each property card
    e = []
    driver.get(i)
    time.sleep(1.5)
    # Append city and country
    city.append('Amsterdam')
    country.append('Netherland')
    try:
        # Find price
        price_element = driver.find_element(By.XPATH, '//div[@class="price-value-container"]/div')
        price_text = price_element.text
        match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', price_text)
        if match:
            price_str = match.group().replace(',', '')
            price_usd = float(price_str)
            price_inr = price_usd * 90.73
            rounded = round(price_inr)
            price.append(f"{rounded}")
        else:
            price.append('-')
    except NoSuchElementException:
        price.append('-')
    try:
        # Find date
        date_element = driver.find_element(By.XPATH, '//div[@class="property-data-date font-xs"]')
        date_text = date_element.text
        match = re.search(r'\b\d{2}/\d{2}/\d{2}\b', date_text)
        if match:
            date_format = match.group()
            date.append(date_format)
        else:
            date.append('-')
    except NoSuchElementException:
        date.append('-')
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
        code.append('-')
    try:
        address_code = re.search(r'\b[A-Z]{2}\b', address_text)
        if address_code:
            code_format = address_code.group()
            code.append(code_format)
        else:
            code.append('-')
    except NoSuchElementException:
        address.append('-')
        code.append('-')
        
    a = driver.find_element(By.XPATH, '//*[@id="main-info"]/section/div/div/rb-property-facilities/section')
    elements = a.find_elements(By.TAG_NAME, 'span')
    
    for i in elements:
        e.append(i.text)
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    
    try:
        if e.index('Bed')>=0:
            bed.append(e[e.index('Bed')-1])
    except ValueError:
        bed.append('-')
    
    try:
        if e.index('Bath')>=0:
            bath.append(e[e.index('Bath')-1])
    except ValueError:
        bath.append('-')
    try:
        if 'Sq Ft' in e:
            area_index = e.index('Sq Ft')
            area_value = e[area_index - 1]
            area.append(area_value)
        elif 'Sq M' in e:
            area_index = e.index('Sq M')
            area_value_sq_m = e[area_index - 1]
            area_value = measurement(area_value_sq_m)
            area.append(area_value)
        else:
            area.append('-')
        
    except ValueError:
        area.append('-')
    try:
        # Check if 'Deposit' is in the list of property facilities
        if 'Deposit' in e:
            # Find the index of 'Deposit' and get the preceding element which contains the deposit amount
            deposit_index = e.index('Deposit')
            deposit_text = e[deposit_index - 1]
            
            # Extract deposit amount
            match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', deposit_text)
            if match:
                price_str = match.group().replace(',', '')
                price_usd = float(price_str)
                price_inr = price_usd * 90.73
                rounded = round(price_inr)
                deposit.append(f"{rounded}")
            else:
                deposit.append('-')
        else:
            deposit.append('-')  # If 'Deposit' is not found, append '-'
    except ValueError:
        deposit.append('-')    
    try:
        if e.index('Dogs') or e.index('Cats') >=0:
            pet.append(e[e.index('Allowed')])
    except ValueError:
        pet.append('-')
# capturing the amenities
    try:
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-rb-amenities/div/rb-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-mls-amenities/div/rb-mls-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
            
        try :
            if e.index('Air Conditioning')>=0:
                ac.append('Available')
        except ValueError:
            ac.append('Unavailable')

        try :
            if e.index('Parking')>=0:
                parking.append('Available')
        except ValueError:
            parking.append('Unavailable')

        try :
            if e.index('Dishwasher')>=0:
                dishwasher.append('Available')
        except ValueError:
            dishwasher.append('Unavailable')

        try :
            if e.index('Fireplace')>=0:
                fireplace.append('Available')
        except ValueError:
            fireplace.append('Unavailable')
        try :
            if e.index('Storage')>=0:
                Storage.append('Available')
        except ValueError:
            Storage.append('Unavailable')
        try :
            if e.index('Hardwood Floors')>=0:
                hardwood.append('Available')
        except ValueError:
            hardwood.append('Unavailable')
            
    except NoSuchElementException:
        ac.append('Unavailable')
        parking.append('Unavailable')
        dishwasher.append('Unavailable')
        fireplace.append('Unavailable')
        Storage.append('Unavailable')
        hardwood.append('Unavailable')
    
#     print(e)
    e.clear()# clear e for each itirations
        

In [47]:
amster=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'bed':bed,'bath':bath,'area':area,'pet':pet,
                 'ac': ac,'parking': parking,'state_code':code,'price':price,'date':date,
                  'dishwasher': dishwasher,'fireplace': fireplace,'storage':Storage,'hardwood_floor':hardwood,'deposit':deposit})
amster


,city,country,name,address,bed,bath,area,pet,ac,parking,state_code,price,date,dishwasher,fireplace,storage,hardwood_floor,deposit
0,Amsterdam,Netherland,Compact Apartment On Canal!,"Korte Prinsengracht 48-H, 1013 GT Amsterdam, N...",2,2,592.01,Allowed,Unavailable,Available,GT,405654,09/25/21,Available,Unavailable,Unavailable,Unavailable,-
1,Amsterdam,Netherland,Bright Double Bedroom Near Kraaiennest Metro S...,"Kantershof 612, 1104 HH Amsterdam, Netherlands",6,1,1442.36,-,Unavailable,Unavailable,HH,63420,11/16/20,Unavailable,Unavailable,Unavailable,Unavailable,-
2,Amsterdam,Netherland,Luminous Double Bedroom In Amstelveen,"Maarten Lutherweg 284, 1185 AX Amstelveen, Net...",4,1,1259.38,-,Unavailable,Unavailable,AX,78844,11/16/20,Unavailable,Unavailable,Unavailable,Unavailable,-
3,Amsterdam,Netherland,Holiday Home With Sauna Near Amsterdam,"1154 PP Uitdam, Netherlands",6,3,"1,615",Allowed,Unavailable,Available,PP,1024070,05/31/23,Available,Unavailable,Unavailable,Unavailable,-
4,Amsterdam,Netherland,Great Family Apartment Amsterdam,"1097 CG Amsterdam, Netherlands",3,1,"1,475",-,Unavailable,Available,CG,816661,05/09/23,Available,Available,Unavailable,Unavailable,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,Amsterdam,Netherland,Chalet With 2 Bedrooms And A Terrace,"1165 NA Halfweg, Netherlands",2,1,-,-,Unavailable,Unavailable,NA,510719,04/24/23,Available,Unavailable,Unavailable,Unavailable,-
326,Amsterdam,Netherland,Off The Strand Gzira 3-Bedroom Apartment,"1050 North Holland, Netherlands",3,2,-,-,Available,Unavailable,-,359019,05/01/23,Available,Unavailable,Unavailable,Unavailable,-
327,Amsterdam,Netherland,Holiday House Noordwijk Aan Zee For 1 - 4 Pers...,"2202, Van Riebeeckhavenweg, 1041 AD Amsterdam,...",2,1,753,-,Unavailable,Unavailable,AD,445031,04/24/23,Unavailable,Unavailable,Unavailable,Unavailable,-
328,Amsterdam,Netherland,Aaaasssssssccvb Erysipelas Dfhii,"1183 HV Amstelveen, Netherlands",2,2,915,-,Unavailable,Unavailable,HV,462723,04/24/23,Unavailable,Unavailable,Unavailable,Unavailable,-


In [102]:
t16=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'date':date,'state_code':code,'price':price})
t16
# print(t16.shape)

,city,country,name,address,date,state_code,price
0,Amsterdam,Netherland,New Luxury One-Bedroom Apartment In The Centre...,"Frens Haringhandel, Singel, 1017 AW Amsterdam,...",08/27/23,AW,381066 INR
1,Amsterdam,Netherland,Comfortable Studio 45m2 With Extra Services,"Htel, Mont Blanc, 1186 AP Amstelveen, Netherlands",02/15/24,AP,321910 INR
2,Amsterdam,Netherland,Newly Renovated 1 Bedroom In Amsterdam South/C...,"Pieter Cornelisz. Hooftstraat 13, 1071 BL Amst...",10/12/22,BL,381066 INR
3,Amsterdam,Netherland,2 Bedrooms Apartment With Balcony In Amsterdam...,"1012 BP Amsterdam, Netherlands",04/23/23,BP,1699101 INR
4,Amsterdam,Netherland,Villeleynah Holiday Home,"1012 North Holland, Netherlands",04/24/23,-,718128 INR
...,...,...,...,...,...,...,...
303,Amsterdam,Netherland,Woning Te Huur: Osdorper Ban 736 1069ZR Amsterdam,"Osdorper Ban 736, 1069 ZR Amsterdam, Netherlands",02/03/24,ZR,172387 INR
304,Amsterdam,Netherland,MVGM - Verhuurd Onder Voorbehoud: Gerard Doust...,"Gerard Doustraat 79A, 1072 VM Amsterdam, Nethe...",12/09/23,VM,117949 INR
305,Amsterdam,Netherland,"Te Huur: Veldhuizenstraat, 1106 DH Amsterdam -...","Veldhuizenstraat 62, 1106 DH Amsterdam, Nether...",02/09/24,DH,145168 INR
306,Amsterdam,Netherland,Studio Te Huur Nieuwezijds Voorburgwal In Amst...,"Nieuwezijds Voorburgwal 18C, 1012 RZ Amsterdam...",02/19/24,RZ,122486 INR


In [94]:
# Table 2 code

city = []
country = []
name = []
address=[]
pet = []
bath = []
bed = []
area = []
deposit=[]
def measurement(sq_m):
    sq_ft = float(sq_m) * 10.7639  # 1 square meter = 10.7639 square feet
    return round(sq_ft,2)

for i in urls: # loop for each property card
    e = []
    driver.get(i)
    time.sleep(1)
    # Append city and country
    city.append('Amsterdam')
    country.append('Netherland')
    a = driver.find_element(By.XPATH, '//*[@id="main-info"]/section/div/div/rb-property-facilities/section')
    elements = a.find_elements(By.TAG_NAME, 'span')
    
    for i in elements:
        e.append(i.text)
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
    
    try:
        if e.index('Bed')>=0:
            bed.append(e[e.index('Bed')-1])
    except ValueError:
        bed.append('-')
    
    try:
        if e.index('Bath')>=0:
            bath.append(e[e.index('Bath')-1])
    except ValueError:
        bath.append('-')
    try:
        if 'Sq Ft' in e:
            area_index = e.index('Sq Ft')
            area_value = e[area_index - 1]
            area.append(area_value)
        elif 'Sq M' in e:
            area_index = e.index('Sq M')
            area_value_sq_m = e[area_index - 1]
            area_value = measurement(area_value_sq_m)
            area.append(area_value)
        else:
            area.append('-')
        
    except ValueError:
        area.append('-')
    try:
        # Check if 'Deposit' is in the list of property facilities
        if 'Deposit' in e:
            # Find the index of 'Deposit' and get the preceding element which contains the deposit amount
            deposit_index = e.index('Deposit')
            deposit_text = e[deposit_index - 1]
            
            # Extract deposit amount
            match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', deposit_text)
            if match:
                price_str = match.group().replace(',', '')
                price_usd = float(price_str)
                price_inr = price_usd * 90.73
                rounded = round(price_inr)
                deposit.append(f"{rounded} INR")
            else:
                deposit.append('-')
        else:
            deposit.append('-')  # If 'Deposit' is not found, append '-'
    except ValueError:
        deposit.append('-')    
    try:
        if e.index('Dogs') or e.index('Cats') >=0:
            pet.append(e[e.index('Allowed')])
    except ValueError:
        pet.append('-')
    
#     print(e)
    e.clear()# clear e for each itirations
        

In [95]:
y16=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'bed':bed,'bath':bath,'area':area,'pet':pet,'deposit':deposit})
y16


,city,country,name,address,bed,bath,area,pet,deposit
0,Amsterdam,Netherland,New Luxury One-Bedroom Apartment In The Centre...,"Frens Haringhandel, Singel, 1017 AW Amsterdam,...",2,1,699.65,-,381066 INR
1,Amsterdam,Netherland,Comfortable Studio 45m2 With Extra Services,"Htel, Mont Blanc, 1186 AP Amstelveen, Netherlands",Studio,1,484.38,Allowed,-
2,Amsterdam,Netherland,Newly Renovated 1 Bedroom In Amsterdam South/C...,"Pieter Cornelisz. Hooftstraat 13, 1071 BL Amst...",2,1,538.19,-,181460 INR
3,Amsterdam,Netherland,2 Bedrooms Apartment With Balcony In Amsterdam...,"1012 BP Amsterdam, Netherlands",2,2,807,-,-
4,Amsterdam,Netherland,Villeleynah Holiday Home,"1012 North Holland, Netherlands",5,6,"13,024",Allowed,-
...,...,...,...,...,...,...,...,...,...
303,Amsterdam,Netherland,Woning Te Huur: Osdorper Ban 736 1069ZR Amsterdam,"Osdorper Ban 736, 1069 ZR Amsterdam, Netherlands",2,1,775.0,-,-
304,Amsterdam,Netherland,MVGM - Verhuurd Onder Voorbehoud: Gerard Doust...,"Gerard Doustraat 79A, 1072 VM Amsterdam, Nethe...",1,1,322.92,-,-
305,Amsterdam,Netherland,"Te Huur: Veldhuizenstraat, 1106 DH Amsterdam -...","Veldhuizenstraat 62, 1106 DH Amsterdam, Nether...",3,1,807.29,-,-
306,Amsterdam,Netherland,Studio Te Huur Nieuwezijds Voorburgwal In Amst...,"Nieuwezijds Voorburgwal 18C, 1012 RZ Amsterdam...",1,1,150.69,-,-


In [96]:
# Table 3 
city = []
country = []
name = []
address=[]
ac=[]
parking=[]
dishwasher=[]
fireplace=[]
Storage=[]
hardwood=[]
for i in urls:
    
    driver.get(i)
    time.sleep(1)
    city.append('Amsterdam')
    country.append('Netherland')
    
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
            

# capturing the amenities
    try:
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-rb-amenities/div/rb-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-mls-amenities/div/rb-mls-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
            
        try :
            if e.index('Air Conditioning')>=0:
                ac.append('Available')
        except ValueError:
            ac.append('Unavailable')

        try :
            if e.index('Parking')>=0:
                parking.append('Available')
        except ValueError:
            parking.append('Unavailable')

        try :
            if e.index('Dishwasher')>=0:
                dishwasher.append('Available')
        except ValueError:
            dishwasher.append('Unavailable')

        try :
            if e.index('Fireplace')>=0:
                fireplace.append('Available')
        except ValueError:
            fireplace.append('Unavailable')
        try :
            if e.index('Storage')>=0:
                Storage.append('Available')
        except ValueError:
            Storage.append('Unavailable')
        try :
            if e.index('Hardwood Floors')>=0:
                hardwood.append('Available')
        except ValueError:
            hardwood.append('Unavailable')
            
    except NoSuchElementException:
        ac.append('Unavailable')
        parking.append('Unavailable')
        dishwasher.append('Unavailable')
        fireplace.append('Unavailable')
        Storage.append('Unavailable')
        hardwood.append('Unavailable')
#         print('No emenities')
    e.clear()


In [97]:
u16= pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'ac': ac,'parking': parking,
                  'dishwasher': dishwasher,'fireplace': fireplace,'Storage':Storage,'hardwood':hardwood})
u16.head()

,city,country,name,address,ac,parking,dishwasher,fireplace,Storage,hardwood
0,Amsterdam,Netherland,New Luxury One-Bedroom Apartment In The Centre...,"Frens Haringhandel, Singel, 1017 AW Amsterdam,...",Unavailable,Unavailable,Available,Unavailable,Unavailable,Unavailable
1,Amsterdam,Netherland,Comfortable Studio 45m2 With Extra Services,"Htel, Mont Blanc, 1186 AP Amstelveen, Netherlands",Unavailable,Available,Unavailable,Unavailable,Available,Unavailable
2,Amsterdam,Netherland,Newly Renovated 1 Bedroom In Amsterdam South/C...,"Pieter Cornelisz. Hooftstraat 13, 1071 BL Amst...",Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable
3,Amsterdam,Netherland,2 Bedrooms Apartment With Balcony In Amsterdam...,"1012 BP Amsterdam, Netherlands",Available,Unavailable,Available,Unavailable,Unavailable,Unavailable
4,Amsterdam,Netherland,Villeleynah Holiday Home,"1012 North Holland, Netherlands",Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable


# munich-germany

In [66]:
driver=webdriver.Chrome()
driver.get('https://rentberry.com/apartments/s/munich-germany')
slct_type=driver.find_element(By.XPATH,'//*[@id="search-sidebar-wrapper"]/div[1]/rb-filter-bar/div[1]/div/div/div[2]/rb-filter-selection/rb-filter-selection-item/div/button')
slct_type.click()
time.sleep(2)


In [67]:
slct_apt=driver.find_element(By.XPATH,'//div[@class="filter-panel ng-star-inserted"]/div[1]/rb-filter-options/rb-filter-option[1]')
slct_apt.click()
time.sleep(2)
slct_save=driver.find_element(By.XPATH,'//button[@class="rb-button w-100"]')
slct_save.click()
time.sleep(2)

In [72]:

urls = []
start_page = 0
end_page = 15  # Adjust the end page number as needed

for page in range(start_page, end_page):
    elements = driver.find_elements(By.XPATH, '//a[@class="property-card-bottom-address__text link-ghost font-xs one-line-string"]')
    for element in elements:
        urls.append(element.get_attribute('href'))
        
    try:
        next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//a[@class="pagination-item"]/span[@class="pagination-arrow-next"]')))
        next_button.click()
        time.sleep(2)
    except Exception as e:
        print("Exception occurred:", e)
        break  # Break the loop if unable to click next button

In [140]:
city = []
country = []
name = []
date = []
address = []
price = []
code = []

for i in urls:  # loop for each property card
    driver.get(i)
    time.sleep(1)
    
    # Append city and country
    city.append('munich')
    country.append('germany')
    
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
    
    try:
        # Find date
        date_element = driver.find_element(By.XPATH, '//div[@class="property-data-date font-xs"]')
        date_text = date_element.text
        match = re.search(r'\b\d{2}/\d{2}/\d{2}\b', date_text)
        if match:
            date_format = match.group()
            date.append(date_format)
        else:
            date.append('-')
    except NoSuchElementException:
        date.append('-')
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
        code.append('-')
    try:
        # Modify the regular expression to match any two capital letters as state codes
        address_code = re.search(r'\b(?:Germany|DE)\b', address_text)
        if address_code:
            code_format = address_code.group()
            code.append(code_format)
        else:
            code.append('-')
    except NoSuchElementException:
        address.append('-')
        code.append('-')

 
    try:
        # Find price
        price_element = driver.find_element(By.XPATH, '//div[@class="price-value-container"]/div')
        price_text = price_element.text
        match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', price_text)
        if match:
            price_str = match.group().replace(',', '')
            price_usd = float(price_str)
            price_inr = price_usd * 90.73
            rounded = round(price_inr)
            price.append(f"{rounded} INR")
        else:
            price.append('-')
    except NoSuchElementException:
        price.append('-')


In [141]:
t17=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'date':date,'state_code':code,'price':price})
t17
# print(t16.shape)

,city,country,name,address,date,state_code,price
0,munich,germany,Cosy And Beautiful Designed Apartment,"M-Suites, Hofmannstraße, 81379 Munich, Germany",02/23/21,Germany,293965 INR
1,munich,germany,Beautiful 1-Room Apartment In Munich'S Westend,"Ridlerstraße 19, 80339 Munich, Germany",02/15/24,Germany,131468 INR
2,munich,germany,Komfort Studio Apartment,"H.ome Serviced Apartments, Am Oberwiesenfeld 7...",07/03/19,Germany,138726 INR
3,munich,germany,Furnished Single Family House,"Gareisstraße 1, 80937 Munich, Germany",03/28/20,Germany,244971 INR
4,munich,germany,"The LOFT 2 Room Apartment For 1-4 People, Appr...","Munich, Bavaria, Germany",04/22/23,Germany,245243 INR
...,...,...,...,...,...,...,...
325,munich,germany,München Olschewskibogen Suite,"Olschewskibogen 18, 80935 Munich, Germany",02/26/21,Germany,188174 INR
326,munich,germany,Quiet Apartment With Covered Balcony And Duple...,"Münchner Straße 3, 82008 Unterhaching, Germany",02/16/24,Germany,135188 INR
327,munich,germany,Beautiful Studio Apartment In Munich'S Westend...,"Ridlerstraße 19, 80339 Munich, Germany",06/27/19,Germany,125207 INR
328,munich,germany,Homely Stay - Studio P1,"Munich, Bavaria, Germany",08/01/23,Germany,558806 INR


In [73]:
# Table 2 code

city = []
country = []
name = []
address=[]
pet = []
bath = []
bed = []
area = []
deposit=[]
def measurement(sq_m):
    sq_ft = float(sq_m) * 10.7639  # 1 square meter = 10.7639 square feet
    return round(sq_ft,2)

for i in urls: # loop for each property card
    e = []
    driver.get(i)
    time.sleep(1)
    # Append city and country
    city.append('munich')
    country.append('germany')
    a = driver.find_element(By.XPATH, '//*[@id="main-info"]/section/div/div/rb-property-facilities/section')
    elements = a.find_elements(By.TAG_NAME, 'span')
    
    for i in elements:
        e.append(i.text)
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
    
    try:
        if e.index('Bed')>=0:
            bed.append(e[e.index('Bed')-1])
    except ValueError:
        bed.append('-')
    
    try:
        if e.index('Bath')>=0:
            bath.append(e[e.index('Bath')-1])
    except ValueError:
        bath.append('-')
    try:
        if 'Sq Ft' in e:
            area_index = e.index('Sq Ft')
            area_value = e[area_index - 1]
            area.append(area_value)
        elif 'Sq M' in e:
            area_index = e.index('Sq M')
            area_value_sq_m = e[area_index - 1]
            area_value = measurement(area_value_sq_m)
            area.append(area_value)
        else:
            area.append('-')
        
    except ValueError:
        area.append('-')
    try:
        # Check if 'Deposit' is in the list of property facilities
        if 'Deposit' in e:
            # Find the index of 'Deposit' and get the preceding element which contains the deposit amount
            deposit_index = e.index('Deposit')
            deposit_text = e[deposit_index - 1]
            match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', deposit_text)
            if match:
                price_str = match.group().replace(',', '')
                dollar = float(price_str)
                inr = dollar * 90.73
                rounded = round(inr)
                deposit.append(f"{rounded}")
            else:
                deposit.append('-')
        else:
            deposit.append('-')  # If 'Deposit' is not found, append '-'
    except ValueError:
        deposit.append('-')  
        
    try:
        if e.index('Dogs') or e.index('Cats') >=0:
            pet.append(e[e.index('Allowed')])
    except ValueError:
        pet.append('-')
    
#     print(e)
    e.clear()# clear e for each itirations
        

In [75]:
y17=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'bed':bed,'bath':bath,'area':area,'pet':pet,'deposit':deposit})
y17

,city,country,name,address,bed,bath,area,pet,deposit
0,munich,germany,Balancing Between Hotel And Living,"SMARTments business, Walter-Gropius-Straße 11,...",1,1,376.74,-,18146
1,munich,germany,Comfortable 1-Room Apartment In Munich,"Leica Geosystems GmbH Vertrieb Deutschland, Tr...",1,1,204.51,-,-
2,munich,germany,Sunny Studio,"Westermühlstraße 37, 80469 Munich, Germany",Studio,1,312.15,-,127022
3,munich,germany,Furnished Apartment Ideal For Long-Term Living...,"Herrenchiemseestraße 30, 81669 Munich, Germany",2,1,409.03,-,153334
4,munich,germany,High Quality Studio - First Time Use,"Ottobrunner Straße 12a, 81737 Munich, Germany",Studio,1,473.61,-,90730
...,...,...,...,...,...,...,...,...,...
325,munich,germany,"AK1-L, One Bedroom Apartment","Munich, Bavaria, Germany",1,1,753,-,-
326,munich,germany,2 Room Apartment In Munich Schwabing/BMW/ Olym...,"Schleißheimer Straße 188, 80797 Munich, Germany",1,1,538.19,-,-
327,munich,germany,Business Apartment With Kitchenette In Munich-...,"Soulmade, Mühlfeldweg 46, 85748 Garching bei M...",Studio,1,226.04,-,-
328,munich,germany,Single Wohnung In München,"Leica Geosystems GmbH Vertrieb Deutschland, Tr...",1,1,-,-,-


In [156]:
# Table 3 
city = []
country = []
name = []
address=[]
ac=[]
parking=[]
dishwasher=[]
fireplace=[]
Storage=[]
hardwood=[]
for i in urls:
    e=[]
    driver.get(i)
    time.sleep(1)
    city.append('munich')
    country.append('germany')
    
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
            

# capturing the amenities
    try:
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-rb-amenities/div/rb-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-mls-amenities/div/rb-mls-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
            
        try :
            if e.index('Air Conditioning')>=0:
                ac.append('Available')
        except ValueError:
            ac.append('Unavailable')

        try :
            if e.index('Parking')>=0:
                parking.append('Available')
        except ValueError:
            parking.append('Unavailable')

        try :
            if e.index('Dishwasher')>=0:
                dishwasher.append('Available')
        except ValueError:
            dishwasher.append('Unavailable')

        try :
            if e.index('Fireplace')>=0:
                fireplace.append('Available')
        except ValueError:
            fireplace.append('Unavailable')
        try :
            if e.index('Storage')>=0:
                Storage.append('Available')
        except ValueError:
            Storage.append('Unavailable')
        try :
            if e.index('Hardwood Floors')>=0:
                hardwood.append('Available')
        except ValueError:
            hardwood.append('Unavailable')
            
    except NoSuchElementException:
        ac.append('Unavailable')
        parking.append('Unavailable')
        dishwasher.append('Unavailable')
        fireplace.append('Unavailable')
        Storage.append('Unavailable')
        hardwood.append('Unavailable')
#         print('No emenities')
    e.clear()


In [157]:
u17= pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'ac': ac,'parking': parking,
                  'dishwasher': dishwasher,'fireplace': fireplace,'Storage':Storage,'hardwood':hardwood})
u17

,city,country,name,address,ac,parking,dishwasher,fireplace,Storage,hardwood
0,munich,germany,Cosy And Beautiful Designed Apartment,"M-Suites, Hofmannstraße, 81379 Munich, Germany",Available,Unavailable,Available,Unavailable,Available,Unavailable
1,munich,germany,Beautiful 1-Room Apartment In Munich'S Westend,"Ridlerstraße 19, 80339 Munich, Germany",Unavailable,Unavailable,Available,Unavailable,Unavailable,Unavailable
2,munich,germany,Komfort Studio Apartment,"H.ome Serviced Apartments, Am Oberwiesenfeld 7...",Available,Available,Available,Unavailable,Unavailable,Unavailable
3,munich,germany,Furnished Single Family House,"Gareisstraße 1, 80937 Munich, Germany",Unavailable,Available,Available,Unavailable,Unavailable,Unavailable
4,munich,germany,"The LOFT 2 Room Apartment For 1-4 People, Appr...","Munich, Bavaria, Germany",Unavailable,Unavailable,Available,Unavailable,Unavailable,Unavailable
...,...,...,...,...,...,...,...,...,...,...
325,munich,germany,München Olschewskibogen Suite,"Olschewskibogen 18, 80935 Munich, Germany",Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable
326,munich,germany,Quiet Apartment With Covered Balcony And Duple...,"Münchner Straße 3, 82008 Unterhaching, Germany",Unavailable,Unavailable,Available,Unavailable,Unavailable,Unavailable
327,munich,germany,Beautiful Studio Apartment In Munich'S Westend...,"Ridlerstraße 19, 80339 Munich, Germany",Unavailable,Unavailable,Available,Unavailable,Unavailable,Unavailable
328,munich,germany,Homely Stay - Studio P1,"Munich, Bavaria, Germany",Unavailable,Unavailable,Available,Unavailable,Unavailable,Unavailable


# glasgow-uk

In [76]:
driver=webdriver.Chrome()
driver.get('https://rentberry.com/apartments/s/glasgow-uk')
slct_type=driver.find_element(By.XPATH,'//*[@id="search-sidebar-wrapper"]/div[1]/rb-filter-bar/div[1]/div/div/div[2]/rb-filter-selection/rb-filter-selection-item/div/button')
slct_type.click()
time.sleep(2)


In [77]:
slct_apt=driver.find_element(By.XPATH,'//div[@class="filter-panel ng-star-inserted"]/div[1]/rb-filter-options/rb-filter-option[1]')
slct_apt.click()
time.sleep(2)
slct_save=driver.find_element(By.XPATH,'//button[@class="rb-button w-100"]')
slct_save.click()
time.sleep(2)

In [78]:

urls = []
start_page = 0
end_page = 13  # Adjust the end page number as needed

for page in range(start_page, end_page):
    elements = driver.find_elements(By.XPATH, '//a[@class="property-card-bottom-address__text link-ghost font-xs one-line-string"]')
    for element in elements:
        urls.append(element.get_attribute('href'))
        
    try:
        next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//a[@class="pagination-item"]/span[@class="pagination-arrow-next"]')))
        next_button.click()
        time.sleep(2)
    except Exception as e:
        print("Exception occurred:", e)
        break  # Break the loop if unable to click next button

In [43]:
city = []
country = []
name = []
date = []
address = []
price = []
code = []

for i in urls:  # loop for each property card
    driver.get(i)
    time.sleep(1)
    
    # Append city and country
    city.append('glasgow')
    country.append('UK')
    
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
    
    try:
        # Find date
        date_element = driver.find_element(By.XPATH, '//div[@class="property-data-date font-xs"]')
        date_text = date_element.text
        match = re.search(r'\b\d{2}/\d{2}/\d{2}\b', date_text)
        if match:
            date_format = match.group()
            date.append(date_format)
        else:
            date.append('-')
    except NoSuchElementException:
        date.append('-')
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
        code.append('-')
    try:
        # Modify the regular expression to match any two capital letters as state codes
        address_code = re.search(r'\b(?:UK|[A-Z]{2,3})\b', address_text)
        if address_code:
            code_format = address_code.group()
            code.append(code_format)
        else:
            code.append('-')
    except NoSuchElementException:
        address.append('-')
        code.append('-')

 
    try:
        # Find price
        price_element = driver.find_element(By.XPATH, '//div[@class="price-value-container"]/div')
        price_text = price_element.text
        match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', price_text)
        if match:
            price_str = match.group().replace(',', '')
            price_usd = float(price_str)
            price_inr = price_usd * 104.82
            rounded = round(price_inr)
            price.append(f"{rounded}")
        else:
            price.append('-')
    except NoSuchElementException:
        price.append('-')


In [44]:
t19=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'date':date,'state_code':code,'price':price})
t19

,city,country,name,address,date,state_code,price
0,glasgow,UK,2 Bedroom Flat To Rent,"Blackfriars Car Park, Parsonage Square, Glasgo...",-,-,120543
1,glasgow,UK,3 Bedroom Flat To Rent,"46 Buccleuch Street, Glasgow, G3 6PQ, United K...",02/07/24,-,156706
2,glasgow,UK,2 Bedroom In Glasgow,"Maverton Building, Fenwick Road, Giffnock, G46...",08/05/23,-,395800
3,glasgow,UK,1 Bedroom In Glasgow,"98 Wilton Gardens, Queen's Cross, Glasgow, G20...",08/05/23,-,359742
4,glasgow,UK,2 Bedroom Flat To Rent,"13 Millroad Gardens, Glasgow, G40 2LU, United ...",02/15/24,-,99579
...,...,...,...,...,...,...,...
281,glasgow,UK,3 Bedroom Flat To Rent,"22 Mavisbank Gardens, Glasgow, G51 1HF, United...",11/17/23,-,146748
282,glasgow,UK,2 Bedroom In Glasgow,"301-305 Glasgow Harbour Terraces, Thornwood, G...",08/05/23,-,359742
283,glasgow,UK,"2 Bedroom Flat For Rent In Townhead, Glasgow","Spar, Port Dundas Road, Glasgow, G4 0HF, Unite...",02/07/24,-,131025
284,glasgow,UK,"3 Bedroom Flat For Rent In Woodlands, Glasgow","Dunearn Street, Glasgow, G4 9ED, United Kingdom",12/27/23,-,259429


In [80]:
# Table 2 code

city = []
country = []
name = []
address=[]
pet = []
bath = []
bed = []
area = []
deposit=[]
def measurement(sq_m):
    sq_ft = float(sq_m) * 10.7639  # 1 square meter = 10.7639 square feet
    return round(sq_ft,2)

for i in urls: # loop for each property card
    e = []
    driver.get(i)
    time.sleep(1)
    # Append city and country
    city.append('glasgow')
    country.append('UK')
    a = driver.find_element(By.XPATH, '//*[@id="main-info"]/section/div/div/rb-property-facilities/section')
    elements = a.find_elements(By.TAG_NAME, 'span')
    
    for i in elements:
        e.append(i.text)
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
    
    try:
        if e.index('Bed')>=0:
            bed.append(e[e.index('Bed')-1])
    except ValueError:
        bed.append('-')
    
    try:
        if e.index('Bath')>=0:
            bath.append(e[e.index('Bath')-1])
    except ValueError:
        bath.append('-')
    try:
        if 'Sq Ft' in e:
            area_index = e.index('Sq Ft')
            area_value = e[area_index - 1]
            area.append(area_value)
        elif 'Sq M' in e:
            area_index = e.index('Sq M')
            area_value_sq_m = e[area_index - 1]
            area_value = measurement(area_value_sq_m)
            area.append(area_value)
        else:
            area.append('-')
        
    except ValueError:
        area.append('-')
    try:
        # Check if 'Deposit' is in the list of property facilities
        if 'Deposit' in e:
            # Find the index of 'Deposit' and get the preceding element which contains the deposit amount
            deposit_index = e.index('Deposit')
            deposit_text = e[deposit_index - 1]
            
            # Extract deposit amount
            match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', deposit_text)
            if match:
                price_str = match.group().replace(',', '')
                dollar = float(price_str)
                inr = dollar * 104.82
                rounded = round(inr)
                deposit.append(f"{rounded}")
            else:
                deposit.append('-')
        else:
            deposit.append('-')  # If 'Deposit' is not found, append '-'
    except ValueError:
        deposit.append('-')
    
    try:
        if e.index('Dogs') or e.index('Cats') >=0:
            pet.append(e[e.index('Allowed')])
    except ValueError:
        pet.append('-')
    
#     print(e)
    e.clear()# clear e for each itirations
        

In [81]:
y19=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'bed':bed,'bath':bath,'area':area,'pet':pet,'deposit':deposit})
y19

,city,country,name,address,bed,bath,area,pet,deposit
0,glasgow,UK,2 Bedroom Flat To Rent,"Blackfriars Car Park, Parsonage Square, Glasgo...",2,1,-,-,-
1,glasgow,UK,3 Bedroom Flat To Rent,"46 Buccleuch Street, Glasgow, G3 6PQ, United K...",3,1,-,-,-
2,glasgow,UK,2 Bedroom In Glasgow,"Maverton Building, Fenwick Road, Giffnock, G46...",3,1,699.65,-,26205
3,glasgow,UK,1 Bedroom In Glasgow,"98 Wilton Gardens, Queen's Cross, Glasgow, G20...",2,1,645.83,-,26205
4,glasgow,UK,2 Bedroom Flat To Rent,"13 Millroad Gardens, Glasgow, G40 2LU, United ...",2,1,-,-,-
...,...,...,...,...,...,...,...,...,...
281,glasgow,UK,3 Bedroom Flat To Rent,"22 Mavisbank Gardens, Glasgow, G51 1HF, United...",3,2,-,-,-
282,glasgow,UK,2 Bedroom In Glasgow,"301-305 Glasgow Harbour Terraces, Thornwood, G...",3,2,699.65,-,26205
283,glasgow,UK,"2 Bedroom Flat For Rent In Townhead, Glasgow","Spar, Port Dundas Road, Glasgow, G4 0HF, Unite...",2,2,-,-,-
284,glasgow,UK,"3 Bedroom Flat For Rent In Woodlands, Glasgow","Dunearn Street, Glasgow, G4 9ED, United Kingdom",3,1,-,-,-


In [47]:
# Table 3 
city = []
country = []
name = []
address=[]
ac=[]
parking=[]
dishwasher=[]
fireplace=[]
Storage=[]
hardwood=[]
for i in urls:
    e=[]
    driver.get(i)
    time.sleep(1)
    city.append('glasgow')
    country.append('UK')
    
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
            

# capturing the amenities
    try:
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-rb-amenities/div/rb-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-mls-amenities/div/rb-mls-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
            
        try :
            if e.index('Air Conditioning')>=0:
                ac.append('Available')
        except ValueError:
            ac.append('Unavailable')

        try :
            if e.index('Parking')>=0:
                parking.append('Available')
        except ValueError:
            parking.append('Unavailable')

        try :
            if e.index('Dishwasher')>=0:
                dishwasher.append('Available')
        except ValueError:
            dishwasher.append('Unavailable')

        try :
            if e.index('Fireplace')>=0:
                fireplace.append('Available')
        except ValueError:
            fireplace.append('Unavailable')
        try :
            if e.index('Storage')>=0:
                Storage.append('Available')
        except ValueError:
            Storage.append('Unavailable')
        try :
            if e.index('Hardwood Floors')>=0:
                hardwood.append('Available')
        except ValueError:
            hardwood.append('Unavailable')
            
    except NoSuchElementException:
        ac.append('Unavailable')
        parking.append('Unavailable')
        dishwasher.append('Unavailable')
        fireplace.append('Unavailable')
        Storage.append('Unavailable')
        hardwood.append('Unavailable')
#         print('No emenities')
    e.clear()


In [51]:
u19= pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'ac': ac,'parking': parking,
                  'dishwasher': dishwasher,'fireplace': fireplace,'Storage':Storage,'hardwood':hardwood})
u19

,city,country,name,address,ac,parking,dishwasher,fireplace,Storage,hardwood
0,glasgow,UK,2 Bedroom Flat To Rent,"Blackfriars Car Park, Parsonage Square, Glasgo...",Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable
1,glasgow,UK,3 Bedroom Flat To Rent,"46 Buccleuch Street, Glasgow, G3 6PQ, United K...",Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable
2,glasgow,UK,2 Bedroom In Glasgow,"Maverton Building, Fenwick Road, Giffnock, G46...",Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable
3,glasgow,UK,1 Bedroom In Glasgow,"98 Wilton Gardens, Queen's Cross, Glasgow, G20...",Unavailable,Available,Available,Unavailable,Available,Unavailable
4,glasgow,UK,2 Bedroom Flat To Rent,"13 Millroad Gardens, Glasgow, G40 2LU, United ...",Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable
...,...,...,...,...,...,...,...,...,...,...
281,glasgow,UK,3 Bedroom Flat To Rent,"22 Mavisbank Gardens, Glasgow, G51 1HF, United...",Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable
282,glasgow,UK,2 Bedroom In Glasgow,"301-305 Glasgow Harbour Terraces, Thornwood, G...",Unavailable,Available,Unavailable,Unavailable,Unavailable,Unavailable
283,glasgow,UK,"2 Bedroom Flat For Rent In Townhead, Glasgow","Spar, Port Dundas Road, Glasgow, G4 0HF, Unite...",Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable
284,glasgow,UK,"3 Bedroom Flat For Rent In Woodlands, Glasgow","Dunearn Street, Glasgow, G4 9ED, United Kingdom",Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable


In [104]:
merged_df = pd.merge(t19,y19, on=['city', 'name', 'address'], suffixes=('_t19', '_y19'))
new_glass = pd.merge(merged_df,u19, on=['city', 'name', 'address'])
new_glass

,city,country_t19,name,address,date,state_code,price,country_y19,bed,bath,area,pet,deposit,country,ac,parking,dishwasher,fireplace,Storage,hardwood
0,glasgow,UK,2 Bedroom Flat To Rent,"Blackfriars Car Park, Parsonage Square, Glasgo...",-,-,120543,UK,2,1,-,-,-,UK,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable
1,glasgow,UK,3 Bedroom Flat To Rent,"46 Buccleuch Street, Glasgow, G3 6PQ, United K...",02/07/24,-,156706,UK,3,1,-,-,-,UK,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable
2,glasgow,UK,2 Bedroom In Glasgow,"Maverton Building, Fenwick Road, Giffnock, G46...",08/05/23,-,395800,UK,3,1,699.65,-,26205,UK,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable
3,glasgow,UK,1 Bedroom In Glasgow,"98 Wilton Gardens, Queen's Cross, Glasgow, G20...",08/05/23,-,359742,UK,2,1,645.83,-,26205,UK,Unavailable,Available,Available,Unavailable,Available,Unavailable
4,glasgow,UK,2 Bedroom Flat To Rent,"13 Millroad Gardens, Glasgow, G40 2LU, United ...",02/15/24,-,99579,UK,2,1,-,-,-,UK,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,glasgow,UK,3 Bedroom Flat To Rent,"22 Mavisbank Gardens, Glasgow, G51 1HF, United...",11/17/23,-,146748,UK,3,2,-,-,-,UK,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable
336,glasgow,UK,2 Bedroom In Glasgow,"301-305 Glasgow Harbour Terraces, Thornwood, G...",08/05/23,-,359742,UK,3,2,699.65,-,26205,UK,Unavailable,Available,Unavailable,Unavailable,Unavailable,Unavailable
337,glasgow,UK,"2 Bedroom Flat For Rent In Townhead, Glasgow","Spar, Port Dundas Road, Glasgow, G4 0HF, Unite...",02/07/24,-,131025,UK,2,2,-,-,-,UK,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable
338,glasgow,UK,"3 Bedroom Flat For Rent In Woodlands, Glasgow","Dunearn Street, Glasgow, G4 9ED, United Kingdom",12/27/23,-,259429,UK,3,1,-,-,-,UK,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable


In [103]:
t19.head(2)

,city,country,name,address,date,state_code,price
0,glasgow,UK,2 Bedroom Flat To Rent,"Blackfriars Car Park, Parsonage Square, Glasgo...",-,-,120543
1,glasgow,UK,3 Bedroom Flat To Rent,"46 Buccleuch Street, Glasgow, G3 6PQ, United K...",02/07/24,-,156706


In [105]:
new_glass.duplicated().sum()

54

In [62]:
gg.drop_duplicates(inplace=True)

In [60]:
gg.shape

(340, 20)

# frankfurt-germany

In [82]:
driver=webdriver.Chrome()
driver.get('https://rentberry.com/apartments/s/frankfurt-germany')
slct_type=driver.find_element(By.XPATH,'//*[@id="search-sidebar-wrapper"]/div[1]/rb-filter-bar/div[1]/div/div/div[2]/rb-filter-selection/rb-filter-selection-item/div/button')
slct_type.click()
time.sleep(2)


In [83]:
slct_apt=driver.find_element(By.XPATH,'//div[@class="filter-panel ng-star-inserted"]/div[1]/rb-filter-options/rb-filter-option[1]')
slct_apt.click()
time.sleep(2)
slct_save=driver.find_element(By.XPATH,'//button[@class="rb-button w-100"]')
slct_save.click()
time.sleep(2)

In [84]:

urls = []
start_page = 0
end_page = 15  # Adjust the end page number as needed

for page in range(start_page, end_page):
    elements = driver.find_elements(By.XPATH, '//a[@class="property-card-bottom-address__text link-ghost font-xs one-line-string"]')
    for element in elements:
        urls.append(element.get_attribute('href'))
        
    try:
        next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//a[@class="pagination-item"]/span[@class="pagination-arrow-next"]')))
        next_button.click()
        time.sleep(2)
    except Exception as e:
        print("Exception occurred:", e)
        break  # Break the loop if unable to click next button

In [5]:
city = []
country = []
name = []
date = []
address = []
price = []
code = []

for i in urls:  # loop for each property card
    driver.get(i)
    time.sleep(1)
    
    # Append city and country
    city.append('frankfurt')
    country.append('germany')
    
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
    
    try:
        # Find date
        date_element = driver.find_element(By.XPATH, '//div[@class="property-data-date font-xs"]')
        date_text = date_element.text
        match = re.search(r'\b\d{2}/\d{2}/\d{2}\b', date_text)
        if match:
            date_format = match.group()
            date.append(date_format)
        else:
            date.append('-')
    except NoSuchElementException:
        date.append('-')
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
        code.append('-')
    try:
        # Modify the regular expression to match any two capital letters as state codes
        address_code = re.search(r'\b(?:Germany|DE)\b', address_text)
        if address_code:
            code_format = address_code.group()
            code.append(code_format)
        else:
            code.append('-')
    except NoSuchElementException:
        address.append('-')
        code.append('-')

 
    try:
        # Find price
        price_element = driver.find_element(By.XPATH, '//div[@class="price-value-container"]/div')
        price_text = price_element.text
        match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', price_text)
        if match:
            price_str = match.group().replace(',', '')
            price_usd = float(price_str)
            price_inr = price_usd * 90.73
            rounded = round(price_inr)
            price.append(f"{rounded} INR")
        else:
            price.append('-')
    except NoSuchElementException:
        price.append('-')


In [6]:
t18=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'date':date,'state_code':code,'price':price})
t18

,city,country,name,address,date,state_code,price
0,frankfurt,germany,"Bright, Modern, Furnished 3 Bedrooms In Steinb...","Wingertstraße 7, 61449 Steinbach (Taunus), Ger...",04/04/22,Germany,241705 INR
1,frankfurt,germany,Chic Studio Apartment With Kitchenette In Sach...,"Mailänder Straße 8, 60598 Frankfurt, Germany",07/10/19,Germany,200604 INR
2,frankfurt,germany,Senior 2 Bedroom Apartment,"Gutleutstraße 45, 60329 Frankfurt, Germany",06/27/19,Germany,396762 INR
3,frankfurt,germany,Large Luxurious Suite Apartment With Balcony/T...,"Gerbermühlstraße, 60599 Frankfurt, Germany",03/03/22,Germany,154876 INR
4,frankfurt,germany,Design Serviced Apartment In Frankfurt Airport,"Amelia-Mary-Earhart-Straße 5, 60549 Frankfurt,...",12/05/19,Germany,146166 INR
...,...,...,...,...,...,...,...
325,frankfurt,germany,Galerie S,"Kleyerstraße 1, 60326 Frankfurt, Germany",07/24/23,Germany,167850 INR
326,frankfurt,germany,Modern Studio In Frankfurt Sachsenhausen,"STAYERY Frankfurt, Länderweg 15-17, 60599 Fran...",07/08/21,Germany,194071 INR
327,frankfurt,germany,"New Longstay Boarding Apartment For 1 Person, ...","Platz der Deutschen Einheit 5, 63065 Offenbach...",11/11/23,Germany,114320 INR
328,frankfurt,germany,Fantastic And New Home Located In Frankfurt Am...,"Friedlebenstraße 36, 60433 Frankfurt, Germany",12/09/22,Germany,126115 INR


In [86]:
# Table 2 code

city = []
country = []
name = []
address=[]
pet = []
bath = []
bed = []
area = []
deposit=[]
def measurement(sq_m):
    sq_ft = float(sq_m) * 10.7639  # 1 square meter = 10.7639 square feet
    return round(sq_ft,2)

for i in urls: # loop for each property card
    e = []
    driver.get(i)
    time.sleep(1)
    # Append city and country
    city.append('frankfurt')
    country.append('germany')
    a = driver.find_element(By.XPATH, '//*[@id="main-info"]/section/div/div/rb-property-facilities/section')
    elements = a.find_elements(By.TAG_NAME, 'span')
    
    for i in elements:
        e.append(i.text)
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
    
    try:
        if e.index('Bed')>=0:
            bed.append(e[e.index('Bed')-1])
    except ValueError:
        bed.append('-')
    
    try:
        if e.index('Bath')>=0:
            bath.append(e[e.index('Bath')-1])
    except ValueError:
        bath.append('-')
    try:
        if 'Sq Ft' in e:
            area_index = e.index('Sq Ft')
            area_value = e[area_index - 1]
            area.append(area_value)
        elif 'Sq M' in e:
            area_index = e.index('Sq M')
            area_value_sq_m = e[area_index - 1]
            area_value = measurement(area_value_sq_m)
            area.append(area_value)
        else:
            area.append('-')
        
    except ValueError:
        area.append('-')
    try:
        # Check if 'Deposit' is in the list of property facilities
        if 'Deposit' in e:
            # Find the index of 'Deposit' and get the preceding element which contains the deposit amount
            deposit_index = e.index('Deposit')
            deposit_text = e[deposit_index - 1]
            
            # Extract deposit amount
            match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', deposit_text)
            if match:
                price_str = match.group().replace(',', '')
                price_usd = float(price_str)
                price_inr = price_usd * 90.73
                rounded = round(price_inr)
                deposit.append(f"{rounded}")
            else:
                deposit.append('-')
        else:
            deposit.append('-')  # If 'Deposit' is not found, append '-'
    except ValueError:
        deposit.append('-') 
        
        
    try:
        if e.index('Dogs') or e.index('Cats') >=0:
            pet.append(e[e.index('Allowed')])
    except ValueError:
        pet.append('-')
    
#     print(e)
    e.clear()# clear e for each itirations
        

In [87]:
y18=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'bed':bed,'bath':bath,'area':area,'pet':pet,'deposit':deposit})
y18

,city,country,name,address,bed,bath,area,pet,deposit
0,frankfurt,germany,"Bright, Modern, Furnished 3 Bedrooms In Steinb...","Wingertstraße 7, 61449 Steinbach (Taunus), Ger...",3,1,645.83,-,127022
1,frankfurt,germany,Chic Studio Apartment With Kitchenette In Sach...,"Mailänder Straße 8, 60598 Frankfurt, Germany",Studio,1,484.38,-,-
2,frankfurt,germany,Senior 2 Bedroom Apartment,"Gutleutstraße 45, 60329 Frankfurt, Germany",3,1,505.9,-,-
3,frankfurt,germany,Large Luxurious Suite Apartment With Balcony/T...,"Gerbermühlstraße, 60599 Frankfurt, Germany",Studio,1,462.85,-,90730
4,frankfurt,germany,Design Serviced Apartment In Frankfurt Airport,"Amelia-Mary-Earhart-Straße 5, 60549 Frankfurt,...",Studio,1,462.85,-,90730
...,...,...,...,...,...,...,...,...,...
325,frankfurt,germany,Fantastic And New Home Located In Frankfurt Am...,"Friedlebenstraße 36, 60433 Frankfurt, Germany",2,1,484.38,Allowed,-
326,frankfurt,germany,Apartment Facing The Front Garden,"Niedenau 7, 60325 Frankfurt, Germany",1,1,-,-,45365
327,frankfurt,germany,Comfort Room Facing The Quiet Inner Courtyard ...,"The Flag West M., Bockenheimer Landstraße 38-4...",Studio,1,247.57,Allowed,-
328,frankfurt,germany,Bright And Quiet Apartment,"Niedenau 7, 60325 Frankfurt, Germany",1,1,-,-,45365


In [9]:
# Table 3 
city = []
country = []
name = []
address=[]
ac=[]
parking=[]
dishwasher=[]
fireplace=[]
Storage=[]
hardwood=[]
for i in urls:
    e=[]
    driver.get(i)
    time.sleep(1)
    city.append('frankfurt')
    country.append('germany')
    
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
            

# capturing the amenities
    try:
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-rb-amenities/div/rb-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-mls-amenities/div/rb-mls-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
            
        try :
            if e.index('Air Conditioning')>=0:
                ac.append('Available')
        except ValueError:
            ac.append('Unavailable')

        try :
            if e.index('Parking')>=0:
                parking.append('Available')
        except ValueError:
            parking.append('Unavailable')

        try :
            if e.index('Dishwasher')>=0:
                dishwasher.append('Available')
        except ValueError:
            dishwasher.append('Unavailable')

        try :
            if e.index('Fireplace')>=0:
                fireplace.append('Available')
        except ValueError:
            fireplace.append('Unavailable')
        try :
            if e.index('Storage')>=0:
                Storage.append('Available')
        except ValueError:
            Storage.append('Unavailable')
        try :
            if e.index('Hardwood Floors')>=0:
                hardwood.append('Available')
        except ValueError:
            hardwood.append('Unavailable')
            
    except NoSuchElementException:
        ac.append('Unavailable')
        parking.append('Unavailable')
        dishwasher.append('Unavailable')
        fireplace.append('Unavailable')
        Storage.append('Unavailable')
        hardwood.append('Unavailable')
#         print('No emenities')
    e.clear()


In [10]:
u18= pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'ac': ac,'parking': parking,
                  'dishwasher': dishwasher,'fireplace': fireplace,'Storage':Storage,'hardwood':hardwood})
u18

,city,country,name,address,ac,parking,dishwasher,fireplace,Storage,hardwood
0,frankfurt,germany,"Bright, Modern, Furnished 3 Bedrooms In Steinb...","Wingertstraße 7, 61449 Steinbach (Taunus), Ger...",Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable
1,frankfurt,germany,Chic Studio Apartment With Kitchenette In Sach...,"Mailänder Straße 8, 60598 Frankfurt, Germany",Available,Available,Unavailable,Unavailable,Available,Unavailable
2,frankfurt,germany,Senior 2 Bedroom Apartment,"Gutleutstraße 45, 60329 Frankfurt, Germany",Available,Unavailable,Available,Unavailable,Unavailable,Unavailable
3,frankfurt,germany,Large Luxurious Suite Apartment With Balcony/T...,"Gerbermühlstraße, 60599 Frankfurt, Germany",Available,Available,Unavailable,Unavailable,Unavailable,Unavailable
4,frankfurt,germany,Design Serviced Apartment In Frankfurt Airport,"Amelia-Mary-Earhart-Straße 5, 60549 Frankfurt,...",Unavailable,Unavailable,Available,Unavailable,Available,Unavailable
...,...,...,...,...,...,...,...,...,...,...
325,frankfurt,germany,Galerie S,"Kleyerstraße 1, 60326 Frankfurt, Germany",Unavailable,Available,Unavailable,Unavailable,Unavailable,Unavailable
326,frankfurt,germany,Modern Studio In Frankfurt Sachsenhausen,"STAYERY Frankfurt, Länderweg 15-17, 60599 Fran...",Unavailable,Available,Unavailable,Unavailable,Available,Unavailable
327,frankfurt,germany,"New Longstay Boarding Apartment For 1 Person, ...","Platz der Deutschen Einheit 5, 63065 Offenbach...",Unavailable,Unavailable,Available,Unavailable,Unavailable,Unavailable
328,frankfurt,germany,Fantastic And New Home Located In Frankfurt Am...,"Friedlebenstraße 36, 60433 Frankfurt, Germany",Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable


# birmingham-uk

In [2]:
driver=webdriver.Chrome()
driver.get('https://rentberry.com/apartments/s/birmingham-uk')
slct_type=driver.find_element(By.XPATH,'//*[@id="search-sidebar-wrapper"]/div[1]/rb-filter-bar/div[1]/div/div/div[2]/rb-filter-selection/rb-filter-selection-item/div/button')
slct_type.click()
time.sleep(2)


In [3]:
slct_apt=driver.find_element(By.XPATH,'//div[@class="filter-panel ng-star-inserted"]/div[1]/rb-filter-options/rb-filter-option[1]')
slct_apt.click()
time.sleep(2)
slct_save=driver.find_element(By.XPATH,'//button[@class="rb-button w-100"]')
slct_save.click()
time.sleep(2)

In [5]:

urls = []
start_page = 0
end_page = 14  # Adjust the end page number as needed

for page in range(start_page, end_page):
    elements = driver.find_elements(By.XPATH, '//a[@class="property-card-bottom-address__text link-ghost font-xs one-line-string"]')
    for element in elements:
        urls.append(element.get_attribute('href'))
        
    try:
        next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//a[@class="pagination-item"]/span[@class="pagination-arrow-next"]')))
        next_button.click()
        time.sleep(2)
    except Exception as e:
        print("Exception occurred:", e)
        break  # Break the loop if unable to click next button

In [6]:
city = []
country = []
name = []
date = []
address = []
price = []
code = []

for i in urls:  # loop for each property card
    driver.get(i)
    time.sleep(0.7)
    
    # Append city and country
    city.append('birmingham')
    country.append('UK')
    
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
    
    try:
        # Find date
        date_element = driver.find_element(By.XPATH, '//div[@class="property-data-date font-xs"]')
        date_text = date_element.text
        match = re.search(r'\b\d{2}/\d{2}/\d{2}\b', date_text)
        if match:
            date_format = match.group()
            date.append(date_format)
        else:
            date.append('-')
    except NoSuchElementException:
        date.append('-')
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
        code.append('-')
    try:
        # Modify the regular expression to match any two capital letters as state codes
        address_code = re.search(r'\b(?:UK|[A-Z]{2,3})\b', address_text)
        if address_code:
            code_format = address_code.group()
            code.append(code_format)
        else:
            code.append('-')
    except NoSuchElementException:
        address.append('-')
        code.append('-')

 
    try:
        # Find price
        price_element = driver.find_element(By.XPATH, '//div[@class="price-value-container"]/div')
        price_text = price_element.text
        match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', price_text)
        if match:
            price_str = match.group().replace(',', '')
            price_usd = float(price_str)
            price_inr = price_usd * 104.82
            rounded = round(price_inr)
            price.append(f"{rounded}")
        else:
            price.append('-')
    except NoSuchElementException:
        price.append('-')


In [7]:
t20=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'date':date,'state_code':code,'price':price})
t20

,city,country,name,address,date,state_code,price
0,birmingham,UK,Super King Parkside City Centre Apartment,"Midford Grove, Park Central, B15 2EE, United K...",06/16/22,-,409846
1,birmingham,UK,1 Bedroom Coach House To Rent,"574 Pershore Road, Kings Heath, B29 7EN, Unite...",01/17/24,-,115302
2,birmingham,UK,2 Bedroom Flat To Rent,"Arosa Drive, Harborne, B17 0SE, United Kingdom",11/09/23,-,131759
3,birmingham,UK,2 Bedroom Apartment To Rent,"Hemisphere, The Boulevard, Balsall Heath, B5 7...",02/03/24,-,112681
4,birmingham,UK,3 Bedroom Flat To Rent,"New Hampton Lofts, 87-99 Branston Street, Asto...",01/13/24,-,193917
...,...,...,...,...,...,...,...
303,birmingham,UK,2 Bedroom Apartment To Rent,"The Cube, 196 Washington Street, Park Central,...",01/05/24,-,167712
304,birmingham,UK,Studio To Rent,"Latitude, Bromsgrove Street, Attwood Green, B5...",01/17/24,-,94338
305,birmingham,UK,2 Bedroom Flat To Rent,"Bristol Road South / Hole Lane, Bristol Road S...",02/15/24,-,115302
306,birmingham,UK,1 Bedroom Flat To Rent,"Vanguard, St John's Walk, Attwood Green, B5 4T...",02/11/24,-,99579


In [9]:
# Table 2 code

city = []
country = []
name = []
address=[]
pet = []
bath = []
bed = []
area = []
deposit=[]
def measurement(sq_m):
    sq_ft = float(sq_m) * 10.7639  # 1 square meter = 10.7639 square feet
    return round(sq_ft,2)

for i in urls: # loop for each property card
    e = []
    driver.get(i)
    time.sleep(1)
    # Append city and country
    city.append('birmingham')
    country.append('UK')
    a = driver.find_element(By.XPATH, '//*[@id="main-info"]/section/div/div/rb-property-facilities/section')
    elements = a.find_elements(By.TAG_NAME, 'span')
    
    for i in elements:
        e.append(i.text)
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
    
    try:
        if e.index('Bed')>=0:
            bed.append(e[e.index('Bed')-1])
    except ValueError:
        bed.append('-')
    
    try:
        if e.index('Bath')>=0:
            bath.append(e[e.index('Bath')-1])
    except ValueError:
        bath.append('-')
    try:
        if 'Sq Ft' in e:
            area_index = e.index('Sq Ft')
            area_value = e[area_index - 1]
            area.append(area_value)
        elif 'Sq M' in e:
            area_index = e.index('Sq M')
            area_value_sq_m = e[area_index - 1]
            area_value = measurement(area_value_sq_m)
            area.append(area_value)
        else:
            area.append('-')
        
    except ValueError:
        area.append('-')
    try:
        # Check if 'Deposit' is in the list of property facilities
        if 'Deposit' in e:
            # Find the index of 'Deposit' and get the preceding element which contains the deposit amount
            deposit_index = e.index('Deposit')
            deposit_text = e[deposit_index - 1]
            
            # Extract deposit amount
            match = re.search(r'(\d{1,3}(,\d{3})*(\.\d+)?)', deposit_text)
            if match:
                price_str = match.group().replace(',', '')
                dollar = float(price_str)
                inr = dollar * 104.82
                rounded = round(inr)
                deposit.append(f"{rounded}")
            else:
                deposit.append('-')
        else:
            deposit.append('-')  # If 'Deposit' is not found, append '-'
    except ValueError:
        deposit.append('-')
    
    try:
        if e.index('Dogs') or e.index('Cats') >=0:
            pet.append(e[e.index('Allowed')])
    except ValueError:
        pet.append('-')
    
#     print(e)
    e.clear()# clear e for each itirations
        

In [10]:
y20=pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'bed':bed,'bath':bath,'area':area,'pet':pet,'deposit':deposit})
y20

,city,country,name,address,bed,bath,area,pet,deposit
0,birmingham,UK,Super King Parkside City Centre Apartment,"Midford Grove, Park Central, B15 2EE, United K...",2,1,699.65,-,15723
1,birmingham,UK,1 Bedroom Coach House To Rent,"574 Pershore Road, Kings Heath, B29 7EN, Unite...",1,1,-,-,-
2,birmingham,UK,2 Bedroom Flat To Rent,"Arosa Drive, Harborne, B17 0SE, United Kingdom",2,1,-,-,-
3,birmingham,UK,2 Bedroom Apartment To Rent,"Hemisphere, The Boulevard, Balsall Heath, B5 7...",2,2,-,-,-
4,birmingham,UK,3 Bedroom Flat To Rent,"New Hampton Lofts, 87-99 Branston Street, Asto...",3,2,-,-,-
...,...,...,...,...,...,...,...,...,...
303,birmingham,UK,2 Bedroom Apartment To Rent,"The Cube, 196 Washington Street, Park Central,...",2,2,-,-,-
304,birmingham,UK,Studio To Rent,"Latitude, Bromsgrove Street, Attwood Green, B5...",Studio,1,-,-,-
305,birmingham,UK,2 Bedroom Flat To Rent,"Bristol Road South / Hole Lane, Bristol Road S...",2,2,-,-,-
306,birmingham,UK,1 Bedroom Flat To Rent,"Vanguard, St John's Walk, Attwood Green, B5 4T...",1,1,-,-,-


In [12]:
# Table 3 
city = []
country = []
name = []
address=[]
ac=[]
parking=[]
dishwasher=[]
fireplace=[]
Storage=[]
hardwood=[]
for i in urls:
    e=[]
    driver.get(i)
    time.sleep(0.7)
    city.append('birmingham')
    country.append('UK')
    
    try:
        # Find property name
        name_element = driver.find_element(By.XPATH, '//h1[@class="property-name"]')
        name_text = name_element.text
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    try:
        # Find address
        address_element = driver.find_element(By.XPATH, '//span[@class="property-data-address-name"]')
        address_text = address_element.text
        address.append(address_text)
    except NoSuchElementException:
        address.append('-')
            

# capturing the amenities
    try:
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-rb-amenities/div/rb-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
        UL_ele = driver.find_element(By.XPATH,'//*[@id="property-amenities"]/div/div/div/rb-property-mls-amenities/div/rb-mls-amenities/ul')
        LI_ele = UL_ele.find_elements(By.TAG_NAME, 'span')
        for ele in LI_ele:
#             print(ele.text)
            e.append(ele.text)
            
        try :
            if e.index('Air Conditioning')>=0:
                ac.append('Available')
        except ValueError:
            ac.append('Unavailable')

        try :
            if e.index('Parking')>=0:
                parking.append('Available')
        except ValueError:
            parking.append('Unavailable')

        try :
            if e.index('Dishwasher')>=0:
                dishwasher.append('Available')
        except ValueError:
            dishwasher.append('Unavailable')

        try :
            if e.index('Fireplace')>=0:
                fireplace.append('Available')
        except ValueError:
            fireplace.append('Unavailable')
        try :
            if e.index('Storage')>=0:
                Storage.append('Available')
        except ValueError:
            Storage.append('Unavailable')
        try :
            if e.index('Hardwood Floors')>=0:
                hardwood.append('Available')
        except ValueError:
            hardwood.append('Unavailable')
            
    except NoSuchElementException:
        ac.append('Unavailable')
        parking.append('Unavailable')
        dishwasher.append('Unavailable')
        fireplace.append('Unavailable')
        Storage.append('Unavailable')
        hardwood.append('Unavailable')
#         print('No emenities')
    e.clear()


In [13]:
u20= pd.DataFrame({'city':city,'country':country,'name':name,'address':address,'ac': ac,'parking': parking,
                  'dishwasher': dishwasher,'fireplace': fireplace,'Storage':Storage,'hardwood':hardwood})
u20

,city,country,name,address,ac,parking,dishwasher,fireplace,Storage,hardwood
0,birmingham,UK,Super King Parkside City Centre Apartment,"Midford Grove, Park Central, B15 2EE, United K...",Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable
1,birmingham,UK,1 Bedroom Coach House To Rent,"574 Pershore Road, Kings Heath, B29 7EN, Unite...",Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable
2,birmingham,UK,2 Bedroom Flat To Rent,"Arosa Drive, Harborne, B17 0SE, United Kingdom",Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable
3,birmingham,UK,2 Bedroom Apartment To Rent,"Hemisphere, The Boulevard, Balsall Heath, B5 7...",Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable
4,birmingham,UK,3 Bedroom Flat To Rent,"New Hampton Lofts, 87-99 Branston Street, Asto...",Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable
...,...,...,...,...,...,...,...,...,...,...
303,birmingham,UK,2 Bedroom Apartment To Rent,"The Cube, 196 Washington Street, Park Central,...",Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable
304,birmingham,UK,Studio To Rent,"Latitude, Bromsgrove Street, Attwood Green, B5...",Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable
305,birmingham,UK,2 Bedroom Flat To Rent,"Bristol Road South / Hole Lane, Bristol Road S...",Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable
306,birmingham,UK,1 Bedroom Flat To Rent,"Vanguard, St John's Walk, Attwood Green, B5 4T...",Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable


In [158]:
first=pd.concat([df1,t4,t5,t6,t7,t13,t14,t15,t16,t17],ignore_index=True)
first

,city,country,name,address,date,state_code,price
0,Austin,USA,"36 Comal Street Apt #B4-2-168, Downtown Austin...",36 Comal Street,02/11/24,-,233567 INR
1,Austin,USA,"301 Brazos Street Apt #B1, Austin, TX 78701 - ...",301 Brazos Street,02/11/24,-,252934 INR
2,Austin,USA,"3233 Harmon Avenue, UT, Austin, TX 78705 - 1 B...",3233 Harmon Avenue,02/11/24,-,116036 INR
3,Austin,USA,"201 Lavaca Street Apt #Neches, Austin, TX 7870...",201 Lavaca Street,02/11/24,-,213951 INR
4,Austin,USA,"Property For Rent In Austin, Texas","914 Taulbee Lane, Austin, TX 78757, USA",02/07/24,TX,290920 INR
...,...,...,...,...,...,...,...
5561,munich,germany,München Olschewskibogen Suite,"Olschewskibogen 18, 80935 Munich, Germany",02/26/21,Germany,188174 INR
5562,munich,germany,Quiet Apartment With Covered Balcony And Duple...,"Münchner Straße 3, 82008 Unterhaching, Germany",02/16/24,Germany,135188 INR
5563,munich,germany,Beautiful Studio Apartment In Munich'S Westend...,"Ridlerstraße 19, 80339 Munich, Germany",06/27/19,Germany,125207 INR
5564,munich,germany,Homely Stay - Studio P1,"Munich, Bavaria, Germany",08/01/23,Germany,558806 INR


In [169]:
second=pd.concat([sec,y13,y14,y15,y16,y17],ignore_index=True)
second

,city,country,name,address,bed,bath,area,pet,deposit
0,Austin,US,"5350 Burnet Road, UT, Austin, TX 78756 - 1 BR ...",5350 Burnet Road,1,1,572,Allowed,NaN
1,Austin,US,"111 Sandra Muraida Way Apt #A10, Austin, TX 78...",111 Sandra Muraida Way,1,1,"1,079",Allowed,NaN
2,Austin,US,"Duval Road, Austin, TX 78759 - 1 BR 1 BA Apart...","Austin, The Centrum, TX, US",1,1,513,Allowed,NaN
3,Austin,US,8 Weeks Free + $1K OFF,"8504 S Congress Ave, Unit B",2,2,"1,080",Allowed,NaN
4,Austin,US,"Metric Boulevard, Austin, TX 78758 - 1 BR 1 BA...","Austin, Village at Walnut Creek, TX, US",1,1,-,Allowed,NaN
...,...,...,...,...,...,...,...,...,...
5957,munich,germany,München Olschewskibogen Suite,"Olschewskibogen 18, 80935 Munich, Germany",Studio,1,247.57,-,-
5958,munich,germany,Quiet Apartment With Covered Balcony And Duple...,"Münchner Straße 3, 82008 Unterhaching, Germany",3,1,484.38,Allowed,-
5959,munich,germany,Beautiful Studio Apartment In Munich'S Westend...,"Ridlerstraße 19, 80339 Munich, Germany",Studio,1,204.51,-,181 INR
5960,munich,germany,Homely Stay - Studio P1,"Munich, Bavaria, Germany",Studio,1,431,-,-


In [29]:
y18.to_csv('seconddd_data.csv', index=False)
y18.shape

(330, 9)

In [162]:
sec=pd.read_csv('second_data.csv')
sec.shape

(4312, 9)

In [167]:
third=pd.concat([df3,u13,u14,u15,u16,u17],ignore_index=True)
third.head(2)

,city,country,name,address,ac,parking,dishwasher,fireplace,storage,hardwood_floor,Storage,hardwood
0,Austin,US,"5350 Burnet Road, UT, Austin, TX 78756 - 1 BR ...",5350 Burnet Road,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,NaN,NaN,NaN
1,Austin,US,"111 Sandra Muraida Way Apt #A10, Austin, TX 78...",111 Sandra Muraida Way,Unavailable,Unavailable,Unavailable,Unavailable,Unavailable,NaN,NaN,NaN


In [30]:
u18.to_csv('thirddd_data.csv',index=False)
u18.shape

(330, 10)

In [53]:
final=pd.read_csv('rent_data.csv')
final.shape

(5914, 21)

In [54]:
final=pd.concat([final,orlando],ignore_index=True)
final.shape

(6354, 21)

In [59]:
final.to_csv('rent_data.csv')